<a href="https://colab.research.google.com/github/d-mcfarland/2022Spring/blob/main/old_main_FinAcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pip install --will exit --must restart

In [ ]:
#### Since a restart runtime is required (via, exit()... you will need to "Run After" on the next code block)
import sys
#!pip install --pre pycaret


#pip install --after restart runtime

In [ ]:
#Throw an error to remind me to restart the runtime
#MUST RESTART the runtime


In [ ]:
!pip install scikit-learn
!pip install pyyaml
!pip install markupsafe==2.0.1

!pip install graphviz 
!pip install -q pydot
!pip install country_converter 

!pip install jenkspy
!pip install bioinfokit
!pip install git+https://github.com/jamesdj/tobit/blob/ba91c92471eb41e63ea27bdb56b99f72fcc5c0cc/tobit.py

!pip install Pingouin
!pip install Jinja2




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install twine 
#!pip install git+https://github.com/d-mcfarland/tobit

#import tobit *

#imports and mount gdrive

In [ ]:
import pandas as pd
import jenkspy
import pingouin as pg
import copy
import io
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
from scipy.stats import f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.stats.api as sms
from pandas.api.types import is_numeric_dtype
from pandas.api.types import CategoricalDtype
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from itertools import permutations
import itertools 
import traceback

'''#using pycaret in Google Colab requires the following two statements
from pycaret.utils import enable_colab
enable_colab()
from pycaret.classification import *
'''
# show all rows & columns (wrap, truncate, see all, view)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)

from google.colab import drive
drive.mount('/gdrive')

#Read dataframes from .csv generated from LoadRawData.ipynb

In [ ]:
# If RAW data changed, run LoadRawData.ipynb to create the following .csv files:
#    df_ifdindex.csv --this file has the Financial Development Indexes from the IMF
#    df_demograph.csv
#    df_incgrp.csv
#    df_iset.csv

###df_relig = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_relig.csv')
###df_country = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_country.csv')
###df_iset = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_iset.csv')
df_merged = pd.read_excel('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_merged.xlsx',index_col='FIPS')
###df_all_religions = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_all_religions.csv')
###df_iset_subset = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data/df_iset_subset.csv')

#df_iset['year']=pd.to_numeric(df_iset['year'], errors='coerce')
### to end
'''
df_iset.reset_index(inplace=True)
df_iset.drop(columns=['Unnamed: 0','index'],inplace=True)
df_country.drop(columns=['Unnamed: 0'],inplace=True)
df_relig.drop(columns=['Unnamed: 0'],inplace=True)
'''


In [ ]:
#drop unused columns
df_merged = df_merged[df_merged.columns.drop(list(df_merged.filter(regex='(?:_y|_x_x|Unnamed|AMERICA|ASIA|AFRICA|AUSTR|EUROP|COUNTRY)')))]



##not used... Inspect & Summarize data (appending df in a loop) & Debug (extra columns)

---



In [ ]:
# Upadate a dataframe in a loop
# create a summary showing the columns in each df
#   and showing the unique columns pulled from each df
'''
#first define the dataframe with column names
df_all_cols = ['df_name','col_desc']
df_all = pd.DataFrame(columns = df_all_cols)

#in loop append rows using "dictionary" input 

for i in df_lst:
  currdf = get_df_name(i) #retrieve the name of the current dataframe
  print(currdf, i.columns, i.shape,'\n')
  print(len(i['country'].unique()))
  #save column names into a new dataframe
  try:
    my_unique = i['desc'].unique()
  except:
    my_unique = i.columns
  for unq_col in my_unique:
    df_all=df_all.append({'df_name':currdf, 'col_desc': unq_col},ignore_index=True)

print ('\n',df_all)
'''

##NOT USED: Manual Updates based on:
###Anomaly analysis (remove outliers)
###Review OIC members


In [ ]:
# After a manual review, update the following countries to OIC = 'Muslim'
'''mylist=['South Sudan', 'Brunei', 'Egypt', 'French Guinana', 'Gambia', 'Iran', 'Kyrgyzstan', 'Syria', 'Yemen']
for my_col in mylist:
  df_demograph.loc[df_demograph['country']==my_col,'OIC']=1

'''
'''# Results of an anomaly analysis, the following countries will be excluded:
mylist=['Singapore','Luxembourg','San Marino','Hong Kong SAR, China','Vietnam','United Arab Emirates','Malaysia'] 
df_demograph=df_demograph.loc[~df_demograph['country'].isin(mylist)]

'''
# to slice a dataframe based on values in a string
'''
search_values = ['Iran']
try: 
  df0=df_demograph[df_demograph['country'].str.contains('|'.join(search_values ))]
  print(df0.transpose())
except:
  True
'''



#DEFINE functions:

3##Jitter funct to add a small amount of noise

In [ ]:
def jitter(a_series, noise_reduction=1000000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))


## regress_table(df_a, predictors, iv_list): Format Regression Table 


In [ ]:
def regress_table(df_a, predictors, iv_list):
    temp=[]
    cnt=0
    for curr_fd in predictors:
        # define DV
        Y = df_a[curr_fd]  
        print('\n', Y.head())
        if(len(Y)>0):
            for curr_iv in iv_list:
                #try:
                X = df_a[curr_iv]
                n=X['FIPS'].nunique()
                print('\nn=',n,'\n',X.head(),'\n',Y.head())

                #except:
                #    X = []
                if(len(X) > 0): 
                    X = sm.add_constant(X)  
                    pglm = pg.linear_regression(X, Y)
                    pglm=pglm[['names', 'coef','pval','r2']]
                    pglm=pglm.loc[pglm['names']!='Intercept']
                    
                    #assign pval designator
                    select_conditions = [
                        (pglm['pval'] <=.01),
                        ((pglm['pval'] <=.05)&(pglm['pval'] >.01)),
                        ((pglm['pval'] <=.1)&(pglm['pval'] >.05)),
                        (pglm['pval'] >.1)
                        ]
                    select_values = ['a', 'b', 'c','']
                    pglm['my_level'] = np.select(select_conditions, select_values)

                    pglm.drop(columns=['pval'],inplace=True)
                    pglm['run_nbr']=cnt
                    pglm['n']=n
                    pglm['dv']=curr_fd

                    #transpose data for output table 
                    #### No longer done for the Excel format!!!!
                    ### pglm=pglm.transpose()
                    temp.append(pglm.round(3))
                    cnt+=1

    df_out = pd.concat(temp)

    df_out['ind']= range(1, len(df_out) + 1)
    df_out.set_index('ind')

    #print(X.rcorr())

    #concat the coef beta-value and sig-level 
    df_out['outcol'] = [str(x) + ' ' + y for x, y in zip(df_out['coef'], df_out['my_level'])]

    piv=df_out.pivot(index=['run_nbr','dv','r2','n'],columns=['names'],values=['outcol'])
    return(piv)



In [ ]:
'''def regress_table(df_a, predictors, iv_list):
    temp=[]
    cnt=0
    for curr_fd in predictors:
        if(curr_fd=='Financial Development Index'):
            short_dv='FD'
        elif(curr_fd=='Financial Institutions Index'):
            short_dv='FI'
        elif(curr_fd=='Financial Markets Index'):
            short_dv='FM'

        # Only look at current FD value
        df_loop=df_a.loc[df_a['desc']==curr_fd].copy(deep=True)
        # define DV
        Y = df_loop['value']  
        for curr_iv in iv_list:
            try:
                X = df_loop[curr_iv]
                n=df_loop['FIPS'].nunique()
            except:
                X = []
            if(len(X) > 0): 
                X = sm.add_constant(X)  
                pglm = pg.linear_regression(X, Y)
                pglm=pglm[['names', 'coef','pval','r2']]
                pglm=pglm.loc[pglm['names']!='Intercept']
                
                #assign pval designator
                select_conditions = [
                    (pglm['pval'] <=.01),
                    ((pglm['pval'] <=.05)&(pglm['pval'] >.01)),
                    ((pglm['pval'] <=.1)&(pglm['pval'] >.05)),
                    (pglm['pval'] >.1)
                    ]
                select_values = ['a', 'b', 'c','']
                pglm['my_level'] = np.select(select_conditions, select_values)

                pglm.drop(columns=['pval'],inplace=True)
                pglm['run_nbr']=cnt
                pglm['n']=n
                pglm['dv']=short_dv

                #transpose data for output table 
                #### No longer done for the Excel format!!!!
                ### pglm=pglm.transpose()
                temp.append(pglm.round(3))
                cnt+=1

    df_out = pd.concat(temp)

    df_out['ind']= range(1, len(df_out) + 1)
    df_out.set_index('ind')

    #print(X.rcorr())

    #concat the coef beta-value and sig-level 
    df_out['outcol'] = [str(x) + ' ' + y for x, y in zip(df_out['coef'], df_out['my_level'])]

    piv=df_out.pivot(index=['run_nbr','dv','r2','n'],columns=['names'],values=['outcol'])
    return(piv)

'''

##Sub-total pivot table

In [ ]:
def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
    '''
    Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
    
    Args:
    - df - dataframe used in pivot table
    - values - values used to aggregrate
    - indices - ordered list of indices to aggregrate by
    - columns - columns to aggregrate by
    - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
    - fill_value - value used to in place of empty cells
    
    Returns:
    -flat table with data aggregrated and tabulated
    
    '''
    listOfTable = []
    for indexNumber in range(len(indices)):
        n = indexNumber+1
        if n == 1:
            table = pd.pivot_table(df, values=values, index=indices[:n], columns=columns, aggfunc=aggfunc, fill_value=fill_value, margins=True)
        else:
            table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value)
        table = table.reset_index()
        for column in indices[n:]:
            table[column] = ''
        listOfTable.append(table)
    concatTable = pd.concat(listOfTable).sort_index()
    concatTable = concatTable.set_index(keys=indices)
    return concatTable.sort_index(axis=0,ascending=True)
#pivot_table_w_subtotals(df=df,values='Value',indices=['Store','Department','Type'],columns=[],aggfunc='sum',fill_value='')

##T-Test / Comparison of means

In [ ]:
#### loops thru row_order
### need to modify this to work for:
###     ['desc'] = passed_dv (long format)
###     OR   [passed_dv] (wide format)
'''
############   timeframe defines DV for the ttests
        #comp_means=comp_means[['country_relig','desc','overall','1980_89','1990_99','2000_09','2010_20']]
        time_frame=['overall','1980_89','1990_99','2000_09','2010_20']
'''   

def compare_means_long(passed_df, passed_dv):
    comp_means =passed_df.copy(deep=True)
    comp_means.reset_index(inplace=True)
    #comp_means=comp_means[['country_relig','desc',passed_dv]]
    comp_means=comp_means.dropna()
    try:
        comp_means.drop(columns='level_0',inplace=True)
    except:
        pass    

    for curr_col in row_order:         
        b=comp_means.loc[(['desc']==curr_col)].copy(deep=True)
        ### Games Howell
        try:
            print("\n\nGames Howell multiple comparison of means for unequal variances and/or unequal sample sizes\n")
            res=pg.pairwise_gameshowell(dv=passed_dv, between='country_relig', data=b).round(3)
            signif=res.loc[res['pval']<.05]
            insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
            print ('Signif Different Religions (Games Howell):', passed_dv, '/', curr_col,':\n', signif[['A','B','pval']])
            print ('Insignif Religions (Games Howell):', passed_dv, '/', curr_col,':\n', insignif[['A','B','pval']])
            
        except Exception:
            print('Games Howell failed\n')
            traceback.print_exc()
            pass

        ### Penguin pairwise ttest
        try:
            print("\n\nttest of means (pairwise): adjusts heteroscedasticity \n")

            # only compare country_relig countries if count > 1
            #gbtmp=b.groupby(['country_relig']).filter(lambda x: len(x)>1).reset_index()
            res=pg.pairwise_ttests(dv=passed_dv, between='country_relig', data=b)
            signif=res.loc[res['p-unc']<.05]
            insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
            print ('Signif Different Religions (ttest of means --pairwise):', passed_dv,'\n', signif[['A','B','pval']])
            print ('Insignif Religions (ttest of means --pairwise)):', passed_dv,'\n', insignif[['A','B','pval']])

        except Exception:
            print('ttest of means (pairwise) failed\n')
            traceback.print_exc()
            pass

        ### Statsmodel ttest
        print("\n\nttest of means (baseline is CIANoRelAll): \n")
        base=b.loc[b['country_relig']=='CIANoRelAll'].copy(deep=True)
        for curr_relig in relig_list:
            if curr_relig != 'CIANoRelAll':
                try:
                    c=b.loc[b['country_relig']==curr_relig].copy(deep=True)
                    t_stat, p_val = stats.ttest_ind(c[curr_dv], base[curr_dv], equal_var=True)
                    print('ttest of means (baseline is CIANoRelAll), p_val=',p_val,'\n')                            
                    if p_val < .05:
                        print('\nSignif:', passed_dv, '/', curr_dv, '/', curr_relig, 'CIANoRelAll',p_val)
                    else:
                        print('\nNOT Signif:',passed_dv, '/', curr_dv, '/', curr_relig, 'CIANoRelAll',p_val)
                except:
                    print('ttest of means (baseline is CIANoRelAll) failed')
                    traceback.print_exc()
                    pass



##Tobit regression functions

In [ ]:
import math
import warnings
from scipy.optimize import minimize
import scipy.stats
from scipy.special import log_ndtr
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score, explained_variance_score
def split_left_right_censored(x, y, cens):
    counts = cens.value_counts()
    if -1 not in counts and 1 not in counts:
        warnings.warn("No censored observations; use regression methods for uncensored data")
    xs = []
    ys = []

    for value in [-1, 0, 1]:
        if value in counts:
            split = cens == value
            y_split = np.squeeze(y[split].values)
            x_split = x[split].values

        else:
            y_split, x_split = None, None
        xs.append(x_split)
        ys.append(y_split)
    return xs, ys


def tobit_neg_log_likelihood(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1])
    s = params[-1]

    to_cat = []

    cens = False
    if y_left is not None:
        cens = True
        left = (y_left - np.dot(x_left, b))
        to_cat.append(left)
    if y_right is not None:
        cens = True
        right = (np.dot(x_right, b) - y_right)
        to_cat.append(right)
    if cens:
        concat_stats = np.concatenate(to_cat, axis=0) / s
        log_cum_norm = scipy.stats.norm.logcdf(concat_stats)  # log_ndtr(concat_stats)
        cens_sum = log_cum_norm.sum()
    else:
        cens_sum = 0

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        mid = scipy.stats.norm.logpdf(mid_stats) - math.log(max(np.finfo('float').resolution, s))
        mid_sum = mid.sum()
    else:
        mid_sum = 0

    loglik = cens_sum + mid_sum

    return - loglik


def tobit_neg_log_likelihood_der(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1]) # in censReg, not using chain rule as below; they optimize in terms of log(s)
    s = params[-1]

    beta_jac = np.zeros(len(b))
    sigma_jac = 0

    if y_left is not None:
        left_stats = (y_left - np.dot(x_left, b)) / s
        l_pdf = scipy.stats.norm.logpdf(left_stats)
        l_cdf = log_ndtr(left_stats)
        left_frac = np.exp(l_pdf - l_cdf)
        beta_left = np.dot(left_frac, x_left / s)
        beta_jac -= beta_left

        left_sigma = np.dot(left_frac, left_stats)
        sigma_jac -= left_sigma

    if y_right is not None:
        right_stats = (np.dot(x_right, b) - y_right) / s
        r_pdf = scipy.stats.norm.logpdf(right_stats)
        r_cdf = log_ndtr(right_stats)
        right_frac = np.exp(r_pdf - r_cdf)
        beta_right = np.dot(right_frac, x_right / s)
        beta_jac += beta_right

        right_sigma = np.dot(right_frac, right_stats)
        sigma_jac -= right_sigma

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        beta_mid = np.dot(mid_stats, x_mid / s)
        beta_jac += beta_mid

        mid_sigma = (np.square(mid_stats) - 1).sum()
        sigma_jac += mid_sigma

    combo_jac = np.append(beta_jac, sigma_jac / s)  # by chain rule, since the expression above is dloglik/dlogsigma

    return -combo_jac


class TobitModel:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.ols_coef_ = None
        self.ols_intercept = None
        self.coef_ = None
        self.intercept_ = None
        self.sigma_ = None

    def fit(self, x, y, cens, verbose=False):
        """
        Fit a maximum-likelihood Tobit regression
        :param x: Pandas DataFrame (n_samples, n_features): Data
        :param y: Pandas Series (n_samples,): Target
        :param cens: Pandas Series (n_samples,): -1 indicates left-censored samples, 0 for uncensored, 1 for right-censored
        :param verbose: boolean, show info from minimization
        :return:
        """
        x_copy = x.copy()
        if self.fit_intercept:
            x_copy.insert(0, 'intercept', 1.0)
        else:
            x_copy.scale(with_mean=True, with_std=False, copy=False)
        init_reg = LinearRegression(fit_intercept=False).fit(x_copy, y)
        b0 = init_reg.coef_
        y_pred = init_reg.predict(x_copy)
        resid = y - y_pred
        resid_var = np.var(resid)
        s0 = np.sqrt(resid_var)
        params0 = np.append(b0, s0)
        xs, ys = split_left_right_censored(x_copy, y, cens)

        result = minimize(lambda params: tobit_neg_log_likelihood(xs, ys, params), params0, method='BFGS',
                          jac=lambda params: tobit_neg_log_likelihood_der(xs, ys, params), options={'disp': verbose})
        if verbose:
            print(result)
        self.ols_coef_ = b0[1:]
        self.ols_intercept = b0[0]
        if self.fit_intercept:
            self.intercept_ = result.x[1]
            self.coef_ = result.x[1:-1]
        else:
            self.coef_ = result.x[:-1]
            self.intercept_ = 0
        self.sigma_ = result.x[-1]
        return self

    def predict(self, x):
        return self.intercept_ + np.dot(x, self.coef_)

    def score(self, x, y, scoring_function=mean_absolute_error):
        y_pred = np.dot(x, self.coef_)
        return scoring_function(y, y_pred)

##Define function to split records into bins: make_bins()

In [ ]:
def make_bins(df2split, split_into, val_col, col_name, labels):
    #debugging, if bin size issue, set with_label = False
    with_label = True

    split_into=int(split_into)

    '''
    #binning based on equal distance (essentially, one divides the range by nbr of desired segments)
    my_min=float(df2split[val_col].min())
    my_max=float(df2split[val_col].max())
    equ_dist_bins = np.linspace(my_min,my_max,split_into+1)
    if with_label:
        df2split['bin_'+col_name+'_dist'] = pd.cut(df2split[col_name], bins= equ_dist_bins, labels=labels, duplicates='drop', include_lowest=True)
    else:
        df2split['bin_'+col_name+'_dist'] = pd.cut(df2split[col_name], bins= equ_dist_bins, duplicates='drop', include_lowest=True)
    #binning based on natural breaks in data (using Jenkspy algorithm/library) 
    jenkspy_breaks = jenkspy.jenks_breaks(df2split[col_name], nb_class=split_into)
    if with_label:
        df2split['bin_'+col_name+'_jenkspy'] = pd.cut(df2split[col_name], bins=jenkspy_breaks, labels=labels, duplicates='drop',  include_lowest=True)
    else:
        df2split['bin_'+col_name+'_jenkspy'] = pd.cut(df2split[col_name], bins=jenkspy_breaks, duplicates='drop',  include_lowest=True)
    '''    
    #binning based on equal number of rows in each bin (using qcut)
    if with_label:
        df2split['bin_'+col_name+'_freq'] = pd.qcut(df2split[col_name].rank(method = 'first'), q=split_into, precision=1, duplicates='drop', labels=labels)
    else:
        df2split['bin_'+col_name+'_freq'] = pd.qcut(df2split[col_name].rank(method = 'first'), q=split_into, precision=1, duplicates='drop')

    #print('equ_dist_bins:',equ_dist_bins, 'jenkspy_breaks:',jenkspy_breaks)

#pd.qcut(df2split[col_name].rank(method = 'first')
#df2split[col_name] + jitter(df2split[col_name]) 



##Define function to test Linear Regression Assumptions:
###1. Linearity: observed & predicted
###2. Normally distributed residuals
###3. Multicolinearity of IV
###4. Autocorrelation of residuals
###5. Heteroscedasticity of residuals


Sources: 
https://jeffmacaluso.github.io/post/LinearRegressionAssumptions/

https://people.duke.edu/~rnau/testing.htm#linearity 

In [ ]:
def linear_regression_assumptions(features, label, feature_names=None):
    """
    Tests a linear regression on the model to see if assumptions are being met
    """
    from sklearn.linear_model import LinearRegression
    from statsmodels.graphics.gofplots import qqplot
    
    # Setting feature names to x1, x2, x3, etc. if they are not defined
    if feature_names is None:
        feature_names = ['X'+str(feature+1) for feature in range(features.shape[1])]
    
    print('Fitting linear regression')
    # Multi-threading if the dataset is a size where doing so is beneficial
    if features.shape[0] < 100000:
        model = LinearRegression(n_jobs=-1)
    else:
        model = LinearRegression()
        
    model.fit(features, label)
    
    # Returning linear regression R^2 and coefficients before performing diagnostics
    r2 = model.score(features, label)
    print()
    print('R^2:', r2, '\n')
    print('Coefficients')
    print('-------------------------------------')
    print('Intercept:', model.intercept_)
    
    for feature in range(len(model.coef_)):
        print('{0}: {1}'.format(feature_names[feature], model.coef_[feature]))

    print('\nPerforming linear regression assumption testing')
    
    # Creating predictions and calculating residuals for assumption tests
    predictions = model.predict(features)
    df_results = pd.DataFrame({'Actual': label, 'Predicted': predictions})
    df_results['Residuals'] = abs(df_results['Actual']) - abs(df_results['Predicted'])

    ########################################################
    #def linear_assumption():
    """
 
    Linearity: Extremely serious. Assumes there is a linear relationship between the predictors and
                the response variable. The points should be symmetrically distributed 
                around a diagonal line with a roughly constant variance.  
                Look carefully for evidence of a "bowed" pattern, indicating that the model 
                makes systematic errors whenever it is making unusually large or small predictions.  
                In multiple regression models, nonlinearity or nonadditivity may also be 
                revealed by systematic patterns in plots of the 
                residuals versus individual independent variables.

    How to fix: 
            Consider applying a nonlinear transformation to the DV and/or IV 
            if you can think of a transformation seems appropriate. 
            (Don’t just make something up!) 
            
            For example, if the data are strictly positive, the log transformation is an option.  
            (The logarithm base does not matter--all log functions are same up to linear scaling
            --although the NL is usually preferred because small changes in the  
            NL are equivalent to percentage changes.
            If a log transformation is applied to the DV only, this is equivalent 
            to assuming that it grows (or decays) exponentially as a function of the IV.  
            If a log transformation is applied to both the DV and the IV, this is equivalent 
            to assuming that the effects of the IV are multiplicative rather than additive in 
            their original units. This means that, on the margin, a small percentage change in 
            one of the IV induces a proportional percentage change in the expected value of the DV, 
            other things being equal. Models of this kind are commonly used in modeling 
            price-demand relationships. 

            Another possibility to consider is adding another regressor that is a nonlinear function 
            of one of the other variables. For example, if you have regressed Y on X, and the graph 
            of residuals versus predicted values suggests a parabolic curve, then it may make sense to 
            regress Y on both X and X^2 (i.e., X-squared). The latter transformation is possible even 
            when X and/or Y have negative values, whereas logging is not.  Higher-order terms of this 
            kind (cubic, etc.) might also be considered in some cases.  But don’t get carried away!
            This sort of "polynomial curve fitting" can be a nice way to draw a smooth curve through 
            a wavy pattern of points (in fact, it is a trend-line option on scatterplots on Excel), 
            but it is usually a terrible way to extrapolate outside the range of the sample data. 

            Finally, it may be that you have overlooked some entirely different IV that explains or
            corrects for the nonlinear pattern or interactions among variables that you are seeing 
            in your residual plots. In that case the shape of the pattern, together with economic 
            or physical reasoning, may suggest some likely suspects.  For example, if the strength 
            of the linear relationship between Y and X1 depends on the level of some other variable 
            X2, this could perhaps be addressed by creating a new IV that is the product of X1 and X2.
            In the case of time series data, if the trend in Y is believed to have changed at a 
            particular point in time, then the addition of a piecewise linear trend variable 
            (one whose string of values looks like 0, 0, …, 0, 1, 2, 3, … ) could be used to fit 
            the kink in the data.  Such a variable can be considered as the product of a trend 
            variable and a dummy variable.  Again, though, you need to beware of overfitting the 
            sample data by throwing in artificially constructed variables that are poorly motivated.  
            At the end of the day you need to be able to interpret the model and explain (or sell) 
            it to others. 
    """
    print('\n=======================================================================================')
    print('Assumption 1: Linear Relationship between the Target and the Features')
    
    print('Checking with a scatter plot of actual vs. predicted. Predictions should follow the diagonal line.')
    
    # Expected mean of the residual term should be close to zero
    print('Expected mean of the residual term should be close to zero:', df_results['Residuals'].mean())

    # Plotting the actual vs predicted values
    sns.lmplot(x='Actual', y='Predicted', data=df_results, fit_reg=False, size=7)
    # Plotting the diagonal line
    line_coords = np.arange(df_results.min().min(), df_results.max().max())
    plt.plot(line_coords, line_coords,  # X and y points
                color='darkorange', linestyle='--')
    plt.title('Actual vs. Predicted')
    plt.show()
    print('If non-linearity is apparent, consider adding a polynomial term')
    
    sns.lmplot(x='Residuals', y='Predicted', data=df_results, fit_reg=False, size=7)
    # Plotting the diagonal line
    line_coords = np.arange(df_results.min().min(), df_results.max().max())
    plt.plot(line_coords, line_coords,  # X and y points
                color='darkorange', linestyle='--')
    plt.title('Residuals vs. Predicted')
    plt.show()
    print('If non-linearity is apparent, consider adding a polynomial term')


    ########################################################
    #def normal_errors_assumption(p_value_thresh=0.05):
    p_value_thresh=0.05
    """
    Normality: Assumes that the error terms are normally distributed. If they are not,
    nonlinear transformations of variables may solve this.
            
    This assumption being violated primarily causes issues with the confidence intervals

    Violations of normality create problems for determining whether model coefficients are significantly
    different from zero and for calculating confidence intervals for forecasts. Sometimes the error 
    distribution is "skewed" by the presence of a few large outliers. Since parameter estimation is 
    based on the minimization of squared error, a few extreme observations can exert a disproportionate 
    influence on parameter estimates. Calculation of confidence intervals and various significance 
    tests for coefficients are all based on the assumptions of normally distributed errors. If the error 
    distribution is significantly non-normal, confidence intervals may be too wide or too narrow.

    Technically, the normal distribution assumption is not necessary if you are willing to assume 
    the model equation is correct and your only goal is to estimate its coefficients and generate 
    predictions in such a way as to minimize mean squared error.  The formulas for estimating 
    coefficients require no more than that, and some references on regression analysis do not list 
    normally distributed errors among the key assumptions.  But generally we are interested in making 
    inferences about the model and/or estimating the probability that a given forecast error will 
    exceed some threshold in a particular direction, in which case distributional assumptions are 
    important.  Also, a significant violation of the normal distribution assumption is often a 
    "red flag" indicating that there is some other problem with the model assumptions and/or that 
    there are a few unusual data points that should be studied closely and/or that a better model 
    is still waiting out there somewhere.

    How to diagnose: 
        The best test for normally distributed errors is a normal probability plot 
        or normal quantile plot of the residuals. These are plots of the fractiles of error distribution 
        versus the fractiles of a normal distribution having the same mean and variance. If the distribution 
        is normal, the points on such a plot should fall close to the diagonal reference line. A 
        bow-shaped pattern of deviations from the diagonal indicates that the residuals have excessive 
        skewness (i.e., they are not symmetrically distributed, with too many large errors in one 
        direction). An S-shaped pattern of deviations indicates that the residuals have excessive 
        kurtosis--i.e., there are either too many or two few large errors in both directions. 
        Sometimes the problem is revealed to be that there are a few data points on one or both ends 
        that deviate significantly from the reference line ("outliers"), in which case they should get 
        close attention. 

        There are also a variety of statistical tests for normality, including the Kolmogorov-Smirnov 
        test, the Shapiro-Wilk test, the Jarque-Bera test, and the Anderson-Darling test.  The 
        Anderson-Darling test is generally considered to be the best, because it is specific to the 
        normal distribution (unlike the K-S test) and it looks at the whole distribution rather than 
        just the skewness and kurtosis (like the J-B test).  But all of these tests are excessively 
        "picky" in this author’s opinion.  Real data rarely has errors that are perfectly normally 
        distributed, and it may not be possible to fit your data with a model whose errors do not 
        violate the normality assumption at the 0.05 level of significance.  It is usually better 
        to focus more on violations of the other assumptions and/or the influence of a few outliers 
        (which may be mainly responsible for violations of normality anyway) and to look at a normal 
        probability plot or normal quantile plot and draw your own conclusions about whether the 
        problem is serious and whether it is systematic.

    How to fix: 
        Violations of normality often arise either because (a) the distributions of the dependent
         and/or independent variables are themselves significantly non-normal, and/or 
         (b) the linearity assumption is violated. In such cases, a nonlinear transformation of 
         variables might cure both problems. In the case of the two normal quantile plots above, 
         the second model was obtained applying a natural log transformation to the variables in 
         the first one. 

        The dependent and independent variables in a regression model do not need to be normally 
        distributed by themselves--only the prediction errors need to be normally distributed.  
        (In fact, independent variables do not even need to be random, as in the case of trend 
        or dummy or treatment or pricing variables.)  But if the distributions of some of the 
        variables that are random are extremely asymmetric or long-tailed, it may be hard to fit 
        them into a linear model whose errors will be normally distributed, and explaining the 
        shape of their distributions may be an interesting topic all by itself.  Keep in mind 
        that the normal error assumption is usually justified by appeal to the central limit 
        theorem, which holds in the case where many random variations are added together.  If 
        the underlying sources of randomness are not interacting additively, this argument fails 
        to hold.

        Another possibility is that there are two or more subsets of the data having different 
        statistical properties, in which case separate models should be built, or else some data 
        should merely be excluded, provided that there is some a priori criterion that can be 
        applied to make this determination.

        In some cases, the problem with the error distribution is mainly due to one or two very 
        large errors. Such values should be scrutinized closely: are they genuine (i.e., not the 
        result of data entry errors), are they explainable, are similar events likely to occur 
        again in the future, and how influential are they in your model-fitting results? If they 
        are merely errors or if they can be explained as unique events not likely to be repeated, 
        then you may have cause to remove them. In some cases, however, it may be that the extreme 
        values in the data provide the most useful information about values of some of the 
        coefficients and/or provide the most realistic guide to the magnitudes of forecast errors.  

    """
    from statsmodels.stats.diagnostic import normal_ad
    print('\n=======================================================================================')
    print('Assumption 2: The error terms are normally distributed')
    print()

    print('Using the Anderson-Darling test for normal distribution')

    # Performing the test on the residuals
    p_value = normal_ad(df_results['Residuals'])[1]
    print('p-value from the test - below 0.05 generally means non-normal:', p_value)

    # Reporting the normality of the residuals
    if p_value < p_value_thresh:
        print('Residuals are not normally distributed')
    else:
        print('Residuals are normally distributed')

    # Plotting the residuals distribution
    plt.subplots(figsize=(12, 6))
    plt.title('Distribution of Residuals')
    sns.distplot(df_results['Residuals'])
    plt.show()

    print()
    if p_value > p_value_thresh:
        print('Assumption satisfied')
    else:
        print('Assumption not satisfied')
        print()
        print('Confidence intervals will likely be affected')
        print('Try performing nonlinear transformations on variables')

    # qqplot
    print('\nQQ Plot... normally distrib data will appears as a straight line along the diagonal.')
    qqplot(label, line='s')
    plt.show()

    
    
    ########################################
    #def multicollinearity_assumption():
    """
    Multicollinearity: Assumes that predictors are not correlated with each other. If there is
                        correlation among the predictors, then either remove prepdictors with high
                        Variance Inflation Factor (VIF) values or perform dimensionality reduction
                        
                        This assumption being violated causes issues with interpretability of the 
                        coefficients and the standard errors of the coefficients.
    """
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    print('\n=======================================================================================')
    print('Assumption 3: Little to no multicollinearity among predictors')
    
    # Plotting the heatmap
    plt.figure(figsize = (10,8))
    sns.heatmap(pd.DataFrame(features, columns=feature_names).corr(), annot=True)
    plt.title('Correlation of Variables')
    plt.show()
    
    print('Variance Inflation Factors (VIF)')
    print('> 10: An indication that multicollinearity may be present')
    print('> 100: Certain multicollinearity among the variables')
    print('-------------------------------------')
    
    # Gathering the VIF for each variable
    VIF = [variance_inflation_factor(features, i) for i in range(features.shape[1])]
    for idx, vif in enumerate(VIF):
        print('{0}: {1}'.format(feature_names[idx], vif))
    
    # Gathering and printing total cases of possible or definite multicollinearity
    possible_multicollinearity = sum([1 for vif in VIF if vif > 10])
    definite_multicollinearity = sum([1 for vif in VIF if vif > 100])
    print()
    print('{0} cases of possible multicollinearity'.format(possible_multicollinearity))
    print('{0} cases of definite multicollinearity'.format(definite_multicollinearity))
    print()

    if definite_multicollinearity == 0:
        if possible_multicollinearity == 0:
            print('Assumption satisfied')
        else:
            print('Assumption possibly satisfied')
            print()
            print('Coefficient interpretability may be problematic')
            print('Consider removing variables with a high Variance Inflation Factor (VIF)')
    else:
        print('Assumption not satisfied')
        print()
        print('Coefficient interpretability will be problematic')
        print('Consider removing variables with a high Variance Inflation Factor (VIF)')
    
    
    ###########################################
    #def autocorrelation_assumption():
    """
    ## autocorrelation of residuals indicates a voilation of independence

    Autocorrelation: 
            Assumes that there is no autocorrelation in the residuals. If there is
            autocorrelation, then there is a pattern that is not explained due to
            the current value being dependent on the previous value.
            This may be resolved by adding a lag variable of either the dependent
            variable or some of the predictors.

            To test for non-time-series violations of independence, you can look at plots of the 
            residuals versus independent variables or plots of residuals versus row number in 
            situations where the rows have been sorted or grouped in some way that depends (only) 
            on the values of the independent variables.  The residuals should be randomly and 
            symmetrically distributed around zero under all conditions, and in particular there 
            should be no correlation between consecutive errors no matter how the rows are sorted, 
            as long as it is on some criterion that does not involve the dependent variable.  If 
            this is not true, it could be due to a violation of the linearity assumption or due to 
            bias that is explainable by omitted variables (say, interaction terms or dummies for 
            identifiable conditions).

        Interpret: 
            Ideally, most of the residual autocorrelations should fall within the 95% 
            confidence bands around zero, which are located at roughly plus-or-minus 
            2-over-the-square-root-of-n, where n is the sample size. Thus, if the sample size is 
            50, the autocorrelations should be between +/- 0.3. If the sample size is 100, they 
            should be between +/- 0.2. Pay especially close attention to significant correlations 
            at the first couple of lags and in the vicinity of the seasonal period, because these 
            are probably not due to mere chance and are also fixable. The Durbin-Watson statistic 
            provides a test for significant residual autocorrelation at lag 1: the DW stat is 
            approximately equal to 2(1-a) where a is the lag-1 residual autocorrelation, so ideally 
            it should be close to 2.0--say, between 1.4 and 2.6 for a sample size of 50.

    How to fix: Minor cases of positive serial correlation (say, lag-1 residual autocorrelation 
            in the range 0.2 to 0.4, or a Durbin-Watson statistic between 1.2 and 1.6) indicate 
            that there is some room for fine-tuning in the model. Consider adding lags of the 
            DV and/or lags of some of the IVs. Or, find an ARIMA+regressor procedure to add an
            AR(1) or MA(1) term to the regression model.  An AR(1) term adds a lag of the DV to
            the forecasting equation, whereas an MA(1) term adds a lag of the forecast error. 
            IF there is significant correlation at lag 2, then a 2nd-order lag may be appropriate.

            If there is significant negative correlation in the residuals (lag-1 autocorrelation
            more negative than -0.3 or DW stat greater than 2.6), watch out for the possibility 
            that you may have overdifferenced some of your variables. Differencing tends to 
            drive autocorrelations in the negative direction, and too much differencing may lead
            to artificial patterns of negative correlation that lagged variables cannot correct 
            for.

            If there is significant correlation at the seasonal period (e.g. at lag 4 for quarterly 
            data or lag 12 for monthly data), this indicates that seasonality has not been properly 
            accounted for in the model. Seasonality can be handled in a regression model in one of 
            the following ways: (i) seasonally adjust the variables (if they are not already 
            seasonally adjusted), or (ii) use seasonal lags and/or seasonally differenced variables 
            (caution: be careful not to overdifference!), or (iii) add seasonal dummy variables to 
            the model (i.e., indicator variables for different seasons of the year, such as MONTH=1 
            or QUARTER=2, etc.) The dummy-variable approach enables additive seasonal adjustment to 
            be performed as part of the regression model: a different additive constant can be 
            estimated for each season of the year. If the dependent variable has been logged, the 
            seasonal adjustment is multiplicative. (Something else to watch out for: it is possible 
            that although your dependent variable is already seasonally adjusted, some of your IVs 
            may not be, causing their seasonal patterns to leak into the forecasts.)

            Major cases of serial correlation (a Durbin-Watson statistic well below 1.0, 
            autocorrelations well above 0.5) usually indicate a fundamental structural problem in 
            the model. You may wish to reconsider the transformations (if any) that have been 
            applied to the dependent and independent variables. It may help to stationarize all 
            variables through appropriate combinations of differencing, logging, and/or deflating.

    """
    from statsmodels.stats.stattools import durbin_watson
    print('\n=======================================================================================')
    print('Assumption 4: No Autocorrelation')
    print('\nPerforming Durbin-Watson Test')
    print('Values of 1.5 < d < 2.5 generally show that there is no autocorrelation in the data')
    print('0 to 2< is positive autocorrelation')
    print('>2 to 4 is negative autocorrelation')
    print('-------------------------------------')
    durbinWatson = durbin_watson(df_results['Residuals'])
    print('Durbin-Watson:', durbinWatson)
    if durbinWatson < 1.5:
        print('Signs of positive autocorrelation', '\n')
        print('Assumption not satisfied', '\n')
        print('Consider adding lag variables')
    elif durbinWatson > 2.5:
        print('Signs of negative autocorrelation', '\n')
        print('Assumption not satisfied', '\n')
        print('Consider adding lag variables')
    else:
        print('Little to no autocorrelation', '\n')
        print('Assumption satisfied')

        
    ######################################
    #def homoscedasticity_assumption():
    """
    Homoscedasticity: Assumes that the errors exhibit constant variance

    How to diagnose: (same plots as linearity)
        Look at a plot of residuals versus predicted values and, in the case of time 
        series data, a plot of residuals versus time.  Be alert for evidence of residuals that grow 
        larger either as a function of time or as a function of the predicted value. To be really thorough, 
        you should also generate plots of residuals versus independent variables to look for consistency 
        there as well.  Because of imprecision in the coefficient estimates, the errors may tend to be 
        slightly larger for forecasts associated with predictions or values of independent variables that 
        are extreme in both directions, although the effect should not be too dramatic.  What you hope not 
        to see are errors that systematically get larger in one direction by a significant amount.

    How to fix:  
        1. Transform the dependent variable. One way to fix heteroscedasticity is to transform the
        dependent variable in some way. One common transformation is to simply take the log of the 
        dependent variable.

        2. Redefine the dependent variable. Another way to fix heteroscedasticity is to redefine the 
        dependent variable. One common way to do so is to use a rate for the dependent variable, 
        rather than the raw value.

        3. Use weighted regression. Another way to fix heteroscedasticity is to use weighted regression.
        This type of regression assigns a weight to each data point based on the variance of its 
        fitted value. When the proper weights are used, this can eliminate the problem of 
        heteroscedasticity.

        If the dependent variable is strictly positive and if the residual-versus-predicted plot
        shows that the size of the errors is proportional to the size of the predictions (i.e., if the 
        errors seem consistent in percentage rather than absolute terms), a log transformation applied to 
        the dependent variable may be appropriate.  In time series models, heteroscedasticity often arises 
        due to the effects of inflation and/or real compound growth. Some combination of logging and/or 
        deflating will often stabilize the variance in this case. Stock market data may show periods of 
        increased or decreased volatility over time. This is normal and is often modeled with so-called 
        ARCH (auto-regressive conditional heteroscedasticity) models in which the error variance is fitted 
        by an autoregressive model. Such models are beyond the scope of this discussion, but a simple fix 
        would be to work with shorter intervals of data in which volatility is more nearly constant. 
        Heteroscedasticity can also be a byproduct of a significant violation of the linearity and/or 
        independence assumptions, in which case it may also be fixed as a byproduct of fixing those problem. 

        Seasonal patterns in the data are a common source of heteroscedasticity in the errors:  
        unexplained variations in the dependent variable throughout the course of a season may be 
        consistent in percentage rather than absolute terms, in which case larger errors will be made 
        in seasons where activity is greater, which will show up as a seasonal pattern of changing variance 
        on the residual-vs-time plot.  A log transformation is often used to address this problem.  For 
        example, if the seasonal pattern is being modeled through the use of dummy variables for months 
        or quarters of the year, a log transformation applied to the dependent variable will convert the 
        coefficients of the dummy variables to multiplicative adjustment factors rather than additive 
        adjustment factors, and the errors in predicting the logged variable will be (roughly) 
        interpretable as percentage errors in predicting the original variable.  Seasonal adjustment of 
        all the data prior to fitting the regression model might be another option. 

        If a log transformation has already been applied to a variable, then (as noted above) additive 
        rather than multiplicative seasonal adjustment should be used, if it is an option that your 
        software offers.  Additive seasonal adjustment is similar in principle to including dummy 
        variables for seasons of the year.  Whether-or-not you should perform the adjustment outside the 
        model rather than with dummies depends on whether you want to be able to study the seasonally 
        adjusted data all by itself and on whether there are unadjusted seasonal patterns in some of the
        independent variables.  (The dummy-variable approach would address the latter problem.)  

    """
    print('\n=======================================================================================')
    print('Assumption 5: Homoscedasticity of Error Terms')
    print('Residuals should have relative constant variance')
    
    # Plotting the residuals
    plt.subplots(figsize=(12, 6))
    ax = plt.subplot(111)  # To remove spines
    plt.scatter(x=df_results.index, y=df_results.Residuals, alpha=0.5)
    plt.plot(np.repeat(0, df_results.index.max()), color='darkorange', linestyle='--')
    ax.spines['right'].set_visible(False)  # Removing the right spine
    ax.spines['top'].set_visible(False)  # Removing the top spine
    plt.title('Residuals')
    plt.show() 
    print('If heteroscedasticity is apparent, confidence intervals and predictions will be affected')
    
    
    bp_test = pd.DataFrame(sms.het_breuschpagan(df_results.Residuals, X), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(df_results.Residuals, X)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)

    ###############################
    # check the absolute value of the standardized residuals for influencial outliers
    ###############################

    '''ATTEMPT #1:
    #create instance of influence
    influence = model.get_influence()

    #obtain standardized residuals
    standardized_residuals = influence.resid_studentized_internal

    #display standardized residuals

    for x in standardized_residuals:
        if x >= 3:
            print (x)

    ATTEMPT #2:

    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
    print(beta_hat)

    y_hat = model.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
    for p_ in range(p):
        standard_error = var_beta_hat[p_, p_] ** 0.5
        print(f"SE(beta_hat[{p_}]): {standard_error}")
    '''
    print('Attempt to show standard errors of predictors')
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    ols = sm.OLS(Y.values, X_with_intercept)
    ols_result = ols.fit()
    print(ols_result.summary())

'''     
linear_assumption()
normal_errors_assumption()
multicollinearity_assumption()
autocorrelation_assumption()
homoscedasticity_assumption()
'''

##Univariate descriptive statistics -->univariate_metrics(df)

In [ ]:
def univariate_metrics (df):
    #create blank lists for univariate statistics
    my_col_dtype = []
    my_col_num_distinct = []
    my_col_perc_missing = []
    my_col_cnt_null = []
    my_col_min = []
    my_col_max = []
    my_col_range = []
    my_col_mean = []
    my_col_std = []
    my_col_median = []
    my_col_Q25 = []
    my_col_Q75 = []
    my_col_CV = []
    my_col_skew = []
    my_col_kurtosis = []

    my_col_mode = []
    my_col_unique_vals = []

    df_loop=[]
    for mycol in df.columns:  #add values to the newly created lists (use a dict to save col_name for each val)
    
        df_loop.append({'Col_Name' : mycol, 'desc':'DType', 'value':df.dtypes[mycol]})
        df_loop.append({'Col_Name' : mycol, 'desc':'NumUnique', 'value':df[mycol].nunique()})
        df_loop.append({'Col_Name' : mycol, 'desc':'PercNull', 'value':df[mycol].isnull().mean()})
        df_loop.append({'Col_Name' : mycol, 'desc':'CntNull', 'value':df[mycol].isnull().sum()})
        
        #if df[mycol].dtype.kind in 'iuf': # these metrics return a single value per feature
        if is_numeric_dtype(df[mycol]): # these metrics return a single value per feature
            df_loop.append({'Col_Name' : mycol, 'desc':'Min', 'value':df[mycol].min()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Max', 'value':df[mycol].max()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Range', 'value': df[mycol].max() - df[mycol].min()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Mean', 'value': df[mycol].mean()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Std', 'value': df[mycol].std()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Median', 'value': df[mycol].median()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Q25', 'value': df[mycol].quantile(q=.25)})
            df_loop.append({'Col_Name' : mycol, 'desc':'Q75', 'value': df[mycol].quantile(q=.75)}) 
            df_loop.append({'Col_Name' : mycol, 'desc':'CoefVariance', 'value': df[mycol].std() / df[mycol].mean()})
            df_loop.append({'Col_Name' : mycol, 'desc':'Skew', 'value': df[mycol].skew()})            
            df_loop.append({'Col_Name' : mycol, 'desc':'Kurtosis', 'value': df[mycol].kurtosis()})           

    df1 = pd.DataFrame.from_dict(df_loop)
    return df1


#Print available variables

In [ ]:
print('df_merged:\n',df_merged.columns.to_list(),'\nhead():\n\n',df_merged.head(100),'\n\ndesc values:\n',df_merged['desc'].unique())
#print('df_iset_subset:\n',df_iset_subset.columns.to_list(),df_iset_subset.head(200))


In [ ]:
print(df_merged.info(verbose=True,show_counts=True),'\n\n')

#ANOVA: MUST Update to pingouin & to test assumptions

In [ ]:
#dataframe "a" is the merged df of df_merged  


#Unconstrained oneway, uncontrained ANOVA
fvalue, pvalue = stats.f_oneway(df_merged['CIABudAll'], df_merged['CIABudHindAll'], df_merged['CIACathAll'], 
                                df_merged['CIAChrAll'],df_merged['CIAHindAll'], df_merged['CIAJewAll'], 
                                df_merged['CIAMusAll'], df_merged['CIANoRelAll'], df_merged['CIAOrthAll'],
                                df_merged['CIAOthNoJew'], df_merged['CIAOthWithJew'], df_merged['CIAProtAll'], df_merged['CIABelieverAll'] )
print('statsmodels unconstrained ANOVA: H0: no differences when comparing religions\n',fvalue, pvalue)

alpha=.05
if (pvalue < alpha):
    print('Reject H0, religions have different values')

# get ANOVA table as R like output

b=df_merged.loc[(df_merged['desc']=='Financial Development Index') & df_merged['overall']!= np.nan]
b.rename(columns={'overall':'value'},inplace=True)

# Ordinary Least Squares (OLS) model
model=ols('value ~ country_relig', data=b).fit() #added 'overall' column
#model = ols('value ~ C(treatments)', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print('\nstatsmodels ANOVA: H0=equal means\n',anova_table)


# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=b, res_var='value', anova_model='value ~ country_relig')
print('\nbioinfokit ANOVA: H0=equal means\n',res.anova_summary)

# note: if the data is balanced (equal sample size for each group), Type 1, 2, and 3 sums of squares
# (typ parameter) will produce similar results.

#Print TABLES

##Rename columns/rows for sorting purposes

In [ ]:
try:
    df_merged.drop(columns=['level_0'],inplace=True)
except:
    pass
df_merged=df_merged.reset_index(drop=False)
try:
    df_merged.drop(columns=['level_0'],inplace=True)
except:
    pass

fd_list=pd.Categorical(['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index'], ordered=True)

fd_list=['FD','FI','FID','FIA','FM','FMD','FMA']

df_merged['country_relig'].replace(
    {'CIAProtAll':'aCIAProtAll', 
    'CIACathAll':'bCIACathAll', 
    'CIAOrthAll':'cCIAOrthAll', 
    'CIAChrAll':'dCIAChrAll',
    'CIAMusAll':'eCIAMusAll', 
    'CIABudAll':'fCIABudAll', 
    'CIAHindAll':'gCIAHindAll', 
    'CIABudHindAll':'hCIABudHindAll',
    'CIAJewAll':'iCIAJewAll',
    'CIAOthNoJew':'jCIAOthNoJew', 
    'CIAOthWithJew':'kCIAOthWithJew',
    'CIABelieverAll':'lCIABelieverAll',
    'CIANoRelAll':'mCIANoRelAll'},inplace=True)

df_merged.rename(columns=
    {'CIAProtAll':'aCIAProtAll', 
    'CIACathAll':'bCIACathAll', 
    'CIAOrthAll':'cCIAOrthAll', 
    'CIAChrAll':'dCIAChrAll',
    'CIAMusAll':'eCIAMusAll', 
    'CIABudAll':'fCIABudAll', 
    'CIAHindAll':'gCIAHindAll', 
    'CIABudHindAll':'hCIABudHindAll',
    'CIAJewAll':'iCIAJewAll',
    'CIAOthNoJew':'jCIAOthNoJew', 
    'CIAOthWithJew':'kCIAOthWithJew',
    'CIABelieverAll':'lCIABelieverAll',
    'CIANoRelAll':'mCIANoRelAll',    
    'IsikFrench':'za_IsikFrench',
    'BeckSettlerMortality':'zb_BeckSettlerMortality',
    'wgi_avg':'zc_wgi_avg',
    'sfi':'zd_sfi',
    'Population (millions)':'ze_Population (millions)'},inplace=True)

''' Run this code right before report runs/saves:
#Rename columns for tables:
piv.rename(columns={'aCIAProtAll':'Prot', 
                    'bCIACathAll':'Cath', 
                    'cCIAOrthAll':'Orth', 
                    'dCIAChrAll':'Chr',
                    'eCIAMusAll':'Mus', 
                    'fCIABudAll':'Bud', 
                    'gCIAHindAll':'Hind', 
                    'hCIABudHindAll':'BudHind',
                    'iCIAJewAll':'Jew', 
                    'jCIAOthNoJew':'Oth-NoJew', 
                    'kCIAOthWithJew':'Oth-Jew', 
                    'lCIABelieverAll':'Believers',
                    'mCIANoRelAll':'No-Rel'},inplace=True)
'''
relig_list=['aCIAProtAll', 'bCIACathAll', 'cCIAOrthAll', 'dCIAChrAll', 
            'eCIAMusAll', 'fCIABudAll', 'gCIAHindAll', 'hCIABudHindAll',
            'iCIAJewAll', 'jCIAOthNoJew', 'kCIAOthWithJew', 'lCIABelieverAll', 'mCIANoRelAll' ] 


##Table 1: Economic, political, legal, and physical health across religions

In [ ]:
##########
# lists of columns for this report
##########
my_col_list=['FIPS','country_relig','desc','overall','1980_89','1990_99','2000_09','2010_20']

my_decades_list=['overall','1980_89','1990_99','2000_09','2010_20']

row_order =['HDI', 
            'GNI per capita', 
            'Gini 2010-18', 
            'ze_Population (millions)', 
            'Inequality in income %', 
            'Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)',
            'Poverty gap at $3.20 a day (2011 PPP) (%)',
            'Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population)',
            'Multidimensional Poverty Index',
            'Life expectancy at birth', 
            'Expected years of schooling', 
            'Prison population (per 100K) 2013-2018', 
            'zd_sfi',
            'actotal', 
            'autoc']

desc_lst = ['Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)',
            'Poverty gap at $3.20 a day (2011 PPP) (%)',
            'Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population)',
            'zd_sfi']

# Read data
df_tmp=df_merged.copy(deep=True)

# Save values for table into a new table
temp=[]
for curr_relig in relig_list:
    df_a=df_tmp.loc[(df_tmp['country_relig']==curr_relig)]
    cnt=len(df_a)
    for curr_col in row_order:
        if curr_col in (desc_lst):
            df_a=df_a.loc[df_a['desc']==curr_col]
            df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
            df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'value':df_a['overall'].mean()})
            #reset df_a for all columns in curr_relig
            df_a=df_tmp.loc[(df_tmp['country_relig']==curr_relig)]
        else:
            
            df_a[curr_col]=pd.to_numeric(df_a[curr_col], errors ='coerce')
            df_a[curr_col] = df_a[curr_col].astype(float, errors = 'raise')
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'value':df_a[curr_col].mean()})
            
for curr_relig in relig_list:
    t=df_tmp.loc[df_tmp['country_relig']==curr_relig]

    print(curr_relig,t['FIPS'].nunique())

#calc world averages
for curr_col in row_order:
    if curr_col in (desc_lst):
        df_a=df_tmp.loc[df_tmp['desc']==curr_col]
        df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
        df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
        temp.append({'country_relig':'world',
                    'desc':curr_col,
                    'value':df_a['overall'].mean()})
    else:
        df_a[curr_col]=pd.to_numeric(df_tmp[curr_col], errors ='coerce')
        df_a[curr_col] = df_a[curr_col].astype(float, errors = 'raise')
        temp.append({'country_relig':'World',
                    'desc':curr_col,
                    'value':df_a[curr_col].mean()})

dfloop = pd.DataFrame.from_dict(temp) 

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dfloop.pivot_table(
    index=['desc'], columns=['country_relig'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)


#$$$piv = piv.reindex(row_order, axis=0)
#Sort Col order
#print(piv.columns.to_list)

piv.sort_index(axis='columns', level='country_relig')
#$$$Rename columns for tables:
piv.rename(columns={'aCIAProtAll':'Prot', 
                    'bCIACathAll':'Cath', 
                    'cCIAOrthAll':'Orth', 
                    'dCIAChrAll':'Chr',
                    'eCIAMusAll':'Mus', 
                    'fCIABudAll':'Bud', 
                    'gCIAHindAll':'Hind', 
                    'hCIABudHindAll':'BudHind',
                    'iCIAJewAll':'Jew', 
                    'jCIAOthNoJew':'Oth-NoJew', 
                    'kCIAOthWithJew':'Oth-Jew', 
                    'lCIABelieverAll':'Believers',
                    'mCIANoRelAll':'No-Rel'},inplace=True)

print ('piv:\n\n',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_1_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#################   Multiple Comparison of means   ##################

# must define the following lists:
### 'row_order' = columns to be compared
### desc_lst = values in desc field to be compared
# must pass these values:
### 'passed_df' 
### 'passed_col'
### 'passed_col_name'

for curr_dv in row_order:
    # compare desc/overall value rather than column value
    if curr_dv in desc_lst:
        df_a=df_tmp.copy(deep=True)
        df_a=df_a.loc[df_a['desc']==curr_dv]
        df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
        df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
        if len(df_a)>0:
            #compare_means_long(df_a, 'overall')
            res=pg.pairwise_gameshowell(dv='overall', between='country_relig', data=df_a).round(3)
            print('\nGames Howell:',curr_dv,'\n')
            signif=res.loc[res['pval']<.05]
            insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
            print ('Signif Different Religions (Games Howell):', curr_dv,'\n', signif[['A','B','pval']])
            print ('Insignif Religions (Games Howell):', curr_dv,'\n', insignif[['A','B','pval']])

            try:
                print('\nttest of means --pairwise')
                #gbtmp=df_a.groupby(['country_relig']).filter(lambda x: len(x)>1).reset_index()
                #changed df to df_a
                res=pg.pairwise_tests(dv='overall', between='country_relig', data=df_a).round(3)
                signif=res.loc[res['p-unc']<.05]
                insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
                print ('Signif Different Religions (ttest of means --pairwise):', curr_dv,'\n', signif[['A','B','p-unc']])
                print ('Insignif Religions (ttest of means --pairwise)):', curr_dv,'\n', insignif[['A','B','p-unc']])

            except: 
                print(curr_dv ,'has a NULL value.  Cannot perform all pairwise ttests.')
                pass

            ''' 
            base=df_a.loc[b['country_relig']=='CIANoRelAll'].copy(deep=True)
            for curr_relig in relig_list:
                if curr_relig != 'CIANoRelAll':
                    try:
                        print("\n\nttest of means (baseline is CIANoRelAll --assumes equal variance and sample size): \n")
                        c=df_a.loc[(b['country_relig']==curr_relig)&(b['desc']==curr_dv)].copy(deep=True)
                        t_stat, p_val = stats.ttest_ind(c['overall'], base['overall'])
                        if p_val < .05:
                            print ('Signif Different Religions (baseline is CIANoRelAll). p-value=:',p_val, curr_dv, curr_relig,'\n')
                        else:
                            print ('Insignif Religions (baseline is CIANoRelAll). p-value=:',p_val, curr_dv,curr_relig,'\n')
                    except: 
                        print(curr_dv ,'has a NULL value. Cannot perform all pairwise ttests.')
                        pass
            '''
    else:  #compare column (rather than desc/overall value)
        df_tmp[curr_dv]=pd.to_numeric(df_tmp[curr_dv], errors ='coerce')
        df_tmp[curr_dv] = df_tmp[curr_dv].astype(float, errors = 'raise')    

        res=pg.pairwise_gameshowell(dv=curr_dv, between='country_relig', data=df_tmp).round(3)
        print('\nGames Howell:',curr_dv,'\n')
        signif=res.loc[res['pval']<.05]
        insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
        print ('Signif Different Religions (Games Howell):', curr_dv,'\n', signif[['A','B','pval']])
        print ('Insignif Religions (Games Howell):', curr_dv,'\n', insignif[['A','B','pval']])
    
        print('\nttest of means --pairwise; uses Welsh for unequal variance or unequal sample size')
        #gbtmp=df_tmp.groupby(['country_relig']).filter(lambda x: len(x)>1).reset_index()
        try:
            res=pg.pairwise_tests(dv=curr_dv, between='country_relig', data=df_tmp, nan_policy='listwise').round(3)
            signif=res.loc[res['p-unc']<.05]
            insignif = pd.concat([res,signif]).drop_duplicates(keep=False)
            print ('Signif Different Religions (ttest of means --pairwise):', curr_dv,'\n', signif[['A','B','p-unc']])
            print ('Insignif Religions (ttest of means --pairwise)):', curr_dv,'\n', insignif[['A','B','p-unc']])
    
        except: 
            print(curr_dv ,'has a NULL value. Cannot perform all pairwise ttests.')
            pass

        '''
        base=df_tmp.loc[df_tmp['country_relig']=='CIANoRelAll'].copy(deep=True)
        for curr_relig in relig_list:
            if curr_relig != 'CIANoRelAll':
                try:
                    print("\n\nttest of means (baseline is CIANoRelAll -assumes equal variance and sample size): \n")
                    c=df_tmp.loc[df_tmp['country_relig']==curr_relig].copy(deep=True)
                    t_stat, p_val = stats.ttest_ind(c[curr_dv], base[curr_dv])
                    if p_val < .05:
                        print ('Signif Different Religions (baseline is CIANoRelAll). p-value=:',p_val, curr_dv, curr_relig,'\n')
                    else:
                        print ('Insignif Religions (baseline is CIANoRelAll). p-value=:',p_val, curr_dv,curr_relig,'\n')
                except: 
                    print(curr_dv ,'has a NULL value. Cannot perform all pairwise ttests.')
                    pass
        '''


In [ ]:
a=df_tmp.copy(deep=True)
a.reset_index(drop=False, inplace=True)
print(a.index,a.columns.to_list)

cols=['index', 'country_relig', 'desc', 'my_level', 'value']

'''
    fvalue, pvalue = stats.f_oneway(b[['country_relig']])

            a[('value',     'CIAChrAll')],
            a[('value',     'CIAMusAll')],
            a[('value',     'CIABudAll')],
            a[('value',    'CIAHindAll')],
            a[('value', 'CIABudHindAll')],
            a[('value',   'CIANoRelAll')],
            a[('value',     'CIAJewAll')],
            a[('value',   'CIAOthNoJew')],
            a[('value', 'CIAOthWithJew')])

print('statsmodels unconstrained ANOVA: H0: no differences when comparing religions\n',fvalue, pvalue)

alpha=.05
if (pvalue < alpha):
    print('Reject H0, religions have different values')
'''
'''    # get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols
############later I should include actotal & autoc##############
for curr_dv in desc_col_lst: 
    b=a.loc[a['desc']==curr_dv]
    #b['value']=np.abs(b['value'])
    print(a.head(50))

    # Ordinary Least Squares (OLS) model
    model=ols('value ~ country_relig', data=b).fit() #added 'overall' column
    #model = ols('value ~ C(treatments)', data=df_melt).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print('\n ANOVA: H0=equal means:\n',curr_col, '\n', anova_table)

'''

##Table 2 Longitudinal religions

In [ ]:
'''
#load hist_country_relig data (by FIPS)
mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/data'
myfile = 'df_all_religions.xlsx'
joined_path = os.path.join(mypath, myfile)
df_all_religions=pd.read_excel(joined_path, sheet_name='Sheet1')
print('df_all_religions data:\n',df_all_religions.info(verbose=True),df_all_religions.head())

#load Population data (by FIPS, by year)
myfile = 'df_population.xlsx'
joined_path = os.path.join(mypath, myfile)
df_population=pd.read_excel(joined_path, sheet_name='Sheet1')

#df_populations FIPS ='world' for each year

#drop unnamed columns
df_population = df_population[df_population.columns.drop(list(df_population.filter(regex='Unnamed')))]
df_population = df_population[['FIPS', 1900, 1945, 1960, 1980, 2000, 2021]]
#df_population.set_index('FIPS')
df_all_religions=df_all_religions[['CIAProt%',
                                    'CIACath%',
                                    'CIAOrth%',
                                    'CIAChr%',
                                    'CIAMus%',
                                    'CIABud%',
                                    'CIAHindu%',
                                    'CIABudHind%',
                                    'CIANoRel%',
                                    'CIAJew%',
                                    'CIAOthNoJew%',
                                    'CIAOthWithJew%',
                                    'FIPS',
                                    'Prot2000%',
                                    'Cath2000%',
                                    'Orth2000%',
                                    'NonTradChr2000%',
                                    'Chr2000%',
                                    'Mus2000%',
                                    'Bud2000%',
                                    'Hindu2000%',
                                    'BudHind2000%',
                                    'NoRel2000%',
                                    'Jew2000%',
                                    'OthNoJew2000%',
                                    'OthWithJew2000%',
                                    'Prot1945%',
                                    'Cath1945%',
                                    'Orth1945%',
                                    'NonTradChr1945%',
                                    'Chr1945%',
                                    'Mus1945%',
                                    'Bud1945%',
                                    'Hindu1945%',
                                    'BudHind1945%',
                                    'NoRel1945%',
                                    'Jew1945%',
                                    'OthNoJew1945%',
                                    'OthWithJew1945%',
                                    'Prot1960%',
                                    'Cath1960%',
                                    'Orth1960%',
                                    'NonTradChr1960%',
                                    'Chr1960%',
                                    'Mus1960%',
                                    'Bud1960%',
                                    'Hindu1960%',
                                    'BudHind1960%',
                                    'NoRel1960%',
                                    'Jew1960%',
                                    'OthNoJew1960%',
                                    'OthWithJew1960%',
                                    'Prot1980%',
                                    'Cath1980%',
                                    'Orth1980%',
                                    'NonTradChr1980%',
                                    'Chr1980%',
                                    'Mus1980%',
                                    'Bud1980%',
                                    'Hindu1980%',
                                    'BudHind1980%',
                                    'NoRel1980%',
                                    'Jew1980%',
                                    'OthNoJew1980%',
                                    'OthWithJew1980%',
                                    'Prot1900%',
                                    'Cath1900%',
                                    'Orth1900%',
                                    'NonTradChr1900%',
                                    'Chr1900%',
                                    'Mus1900%',
                                    'Bud1900%',
                                    'Hindu1900%',
                                    'BudHind1900%',
                                    'NoRel1900%',
                                    'Jew1900%',
                                    'OthNoJew1900%',
                                    'OthWithJew1900%'
                                    ]]

#df_all_religions.set_index('FIPS')
df=pd.DataFrame(columns=['FIPS','Prot2021'])
df[2021] =df_population['FIPS'].map(df_population.set_index('FIPS')[2021])
df['Prot2021%'] = df['FIPS'].map(df_all_religions.set_index('FIPS')['CIAProt%'])
df=df_population.merge(df_all_religions, how='right',on='FIPS')
df['prot_2021']=(df['CIAProt%']*df[2021]).sum()

print('sum-product?:\n',df.head(75))

#valid_columns = [col for col in df.columns if col.isdigit()]
#df['sumproduct'] = (df[valid_columns] * [int(x) for x in valid_columns]).sum(axis=1)

#print('df_population data:\n',df_population.info(verbose=True),df_population.head())

#manually load data into df
#df = pd.DataFrame(columns=['FIPS','hist_country_relig%', 1900, 1945, 1960, 1980, 2000, 2021 ])

try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

keep_col_lst=   ['FIPS',
                 'country',
                 'country_relig',
                 'CIAProt%', 'CIACath%', 'CIAOrth%', 'CIAChr%', 'CIAMus%', 'CIABud%', 'CIAHindu%', 'CIABudHind%', 'CIANoRel%', 'CIAJew%', 'CIAOthNoJew%', 'CIAOthWithJew%', 
                 'Prot2000', 'Cath2000', 'Orth2000', 'Chr2000', 'Mus2000', 'Bud2000', 'Hind2000', 'BudHind2000', 'NoRel2000', 'Jew2000', 'OthNoJew2000', 'OthWithJew2000', 'Believer2000', 
                 'Prot2000%', 'Cath2000%', 'Orth2000%', 'NonTradChr2000%', 'Chr2000%', 'Mus2000%', 'Bud2000%', 'Hindu2000%', 'BudHind2000%', 'NoRel2000%', 'Jew2000%', 'OthNoJew2000%', 'OthWithJew2000%', 
                 'Prot1945', 'Cath1945', 'Orth1945', 'Chr1945', 'Mus1945', 'Bud1945', 'Hind1945', 'BudHind1945', 'NoRel1945', 'Jew1945', 'OthNoJew1945', 'OthWithJew1945', 'Believer1945', 
                 'Prot1945%', 'Cath1945%', 'Orth1945%', 'NonTradChr1945%', 'Chr1945%', 'Mus1945%', 'Bud1945%', 'Hindu1945%', 'BudHind1945%', 'NoRel1945%', 'Jew1945%', 'OthNoJew1945%', 'OthWithJew1945%', 
                 'Prot1960', 'Cath1960', 'Orth1960', 'Chr1960', 'Mus1960', 'Bud1960', 'Hind1960', 'BudHind1960', 'NoRel1960', 'Jew1960', 'OthNoJew1960', 'OthWithJew1960', 'Believer1960', 
                 'Prot1960%', 'Cath1960%', 'Orth1960%', 'NonTradChr1960%', 'Chr1960%', 'Mus1960%', 'Bud1960%', 'Hindu1960%', 'BudHind1960%', 'NoRel1960%', 'Jew1960%', 'OthNoJew1960%', 'OthWithJew1960%', 
                 'Prot1980', 'Cath1980', 'Orth1980', 'Chr1980', 'Mus1980', 'Bud1980', 'Hind1980', 'BudHind1980', 'NoRel1980', 'Jew1980', 'OthNoJew1980', 'OthWithJew1980', 'Believer1980', 
                 'Prot1980%', 'Cath1980%', 'Orth1980%', 'NonTradChr1980%', 'Chr1980%', 'Mus1980%', 'Bud1980%', 'Hindu1980%', 'BudHind1980%', 'NoRel1980%', 'Jew1980%', 'OthNoJew1980%', 'OthWithJew1980%', 
                 'Prot1900', 'Cath1900', 'Orth1900', 'Chr1900', 'Mus1900', 'Bud1900', 'Hind1900', 'BudHind1900', 'NoRel1900', 'Jew1900', 'OthNoJew1900', 'OthWithJew1900', 'Believer1900', 
                 'Prot1900%', 'Cath1900%', 'Orth1900%', 'NonTradChr1900%', 'Chr1900%', 'Mus1900%', 'Bud1900%', 'Hindu1900%', 'BudHind1900%', 'NoRel1900%', 'Jew1900%', 'OthNoJew1900%', 'OthWithJew1900%', 
                 'desc', 
                 2020, 2021, 
                 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
                 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 
                 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 
                 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 
                 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 
                 1945, 1946, 1947, 1948, 1949, 
                 1900
                ]

'''
'''
df_tmp=df_tmp[[keep_col_lst]]
df_tmp.drop_duplicates(inplace=True, ignore_index=True)
'''
'''
# add additional columns


# by relig by year
temp=[]
for curr_level in select_values:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_r1.copy(deep=True)
            df_col1=df_col.loc[(df_col['my_level']==curr_level)&(df_col['desc']==curr_col)&(df_col['country_relig']==curr_relig)]
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'my_level':curr_level,
                        'value':df_col1['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 


#sort=False keeps the order of the columns=['country_relig'] in pivot_table
print ('dfloop:::::::',dfloop.head(15))
piv=np.round(dfloop.pivot_table(
    index=['my_level','country_relig'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)
row_order =['HDI Score', 
         'UN Income Index',
         'Gini index (World Bank estimate)', 
         'Multidimensional Poverty Index',
         'UN Life Expectancy Index', 
         'Life expectancy at birth, total (years)',
         'UIS: Mean years of schooling (ISCED 1 or higher), population 25+ years, both sexes',
         'Prison per 100K', 
         'sfi',
         'actotal', 
         'autoc']

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        
   
col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)

print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_colonizer_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_colonizer_counts.xlsx'
joined_path = os.path.join(mypath, myfile)
df_level.to_excel(joined_path)

time_frame=['overall']

#compare_means(df_tmp, other)
'''

##Table 3: relig by decade

In [ ]:
# Clean load of data
a=df_merged.copy(deep=True)

#lists of columns for this report
fd_list=pd.Categorical(['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index'], ordered=True)

my_col_list=['FIPS','country_relig','desc','overall','1980_89','1990_99','2000_09','2010_20']

my_decades_list=['overall','1980_89','1990_99','2000_09','2010_20']

#Sort Row order
#print(piv.index.to_list) # to see row index

row_order =[    ('1980_89',             'Financial Development Index'),
            ('1980_89',            'Financial Institutions Index'),
            ('1980_89',      'Financial Institutions Depth Index'),
            ('1980_89',     'Financial Institutions Access Index'),
            ('1980_89', 'Financial Institutions Efficiency Index'),
            ('1980_89',                 'Financial Markets Index'),
            ('1980_89',           'Financial Markets Depth Index'),
            ('1980_89',          'Financial Markets Access Index'),
            ('1980_89',      'Financial Markets Efficiency Index'),
            ('1990_99',             'Financial Development Index'),
            ('1990_99',            'Financial Institutions Index'),
            ('1990_99',      'Financial Institutions Depth Index'),
            ('1990_99',     'Financial Institutions Access Index'),
            ('1990_99', 'Financial Institutions Efficiency Index'),
            ('1990_99',                 'Financial Markets Index'),
            ('1990_99',           'Financial Markets Depth Index'),
            ('1990_99',          'Financial Markets Access Index'),
            ('1990_99',      'Financial Markets Efficiency Index'),
            ('2000_09',             'Financial Development Index'),
            ('2000_09',            'Financial Institutions Index'),
            ('2000_09',      'Financial Institutions Depth Index'),
            ('2000_09',     'Financial Institutions Access Index'),
            ('2000_09', 'Financial Institutions Efficiency Index'),
            ('2000_09',                 'Financial Markets Index'),
            ('2000_09',           'Financial Markets Depth Index'),
            ('2000_09',          'Financial Markets Access Index'),
            ('2000_09',      'Financial Markets Efficiency Index'),
            ('2010_20',             'Financial Development Index'),
            ('2010_20',            'Financial Institutions Index'),
            ('2010_20',      'Financial Institutions Depth Index'),
            ('2010_20',     'Financial Institutions Access Index'),
            ('2010_20', 'Financial Institutions Efficiency Index'),
            ('2010_20',                 'Financial Markets Index'),
            ('2010_20',           'Financial Markets Depth Index'),
            ('2010_20',          'Financial Markets Access Index'),
            ('2010_20',      'Financial Markets Efficiency Index'),
            ('overall',             'Financial Development Index'),
            ('overall',            'Financial Institutions Index'),
            ('overall',      'Financial Institutions Depth Index'),
            ('overall',     'Financial Institutions Access Index'),
            ('overall', 'Financial Institutions Efficiency Index'),
            ('overall',                 'Financial Markets Index'),
            ('overall',           'Financial Markets Depth Index'),
            ('overall',          'Financial Markets Access Index'),
            ('overall',      'Financial Markets Efficiency Index')]

#print(df_b_piv.columns.to_list) # to see column index

col_order = [('mean',   'CIAProtAll'),
            ('mean',    'CIACathAll'),
            ('mean',    'CIAOrthAll'),
            ('mean',     'CIAChrAll'),
            ('mean',     'CIAMusAll'),
            ('mean',     'CIABudAll'),
            ('mean',    'CIAHindAll'),
            ('mean', 'CIABudHindAll'),
            ('mean',     'CIAJewAll'),
            ('mean',   'CIAOthNoJew'),
            ('mean', 'CIAOthWithJew'),
            ('mean',  'CIABelieverAll'),
            ('mean',   'CIANoRelAll')
            ]  

row_cnt_order=[('1980_89', 'CIAProtAll'),
            ('1980_89',    'CIACathAll'),
            ('1980_89',    'CIAOrthAll'),
            ('1980_89',     'CIAChrAll'),
            ('1980_89',     'CIAMusAll'),
            ('1980_89',     'CIABudAll'),
            ('1980_89',    'CIAHindAll'),
            ('1980_89', 'CIABudHindAll'),
            ('1980_89',     'CIAJewAll'),
            ('1980_89',   'CIAOthNoJew'),
            ('1980_89', 'CIAOthWithJew'),
            ('1980_89',  'CIABelieverAll'),
            ('1980_89',   'CIANoRelAll'),
            ('1990_99',    'CIAProtAll'),
            ('1990_99',    'CIACathAll'),
            ('1990_99',    'CIAOrthAll'),
            ('1990_99',     'CIAChrAll'),
            ('1990_99',     'CIAMusAll'),
            ('1990_99',     'CIABudAll'),
            ('1990_99',    'CIAHindAll'),
            ('1990_99', 'CIABudHindAll'),
            ('1990_99',     'CIAJewAll'),
            ('1990_99',   'CIAOthNoJew'),
            ('1990_99', 'CIAOthWithJew'),
            ('1990_99',  'CIABelieverAll'),
            ('1990_99',   'CIANoRelAll'),

            ('2000_09',    'CIAProtAll'),
            ('2000_09',    'CIACathAll'),
            ('2000_09',    'CIAOrthAll'),
            ('2000_09',     'CIAChrAll'),
            ('2000_09',     'CIAMusAll'),
            ('2000_09',     'CIABudAll'),
            ('2000_09',    'CIAHindAll'),
            ('2000_09', 'CIABudHindAll'),
            ('2000_09',     'CIAJewAll'),
            ('2000_09',   'CIAOthNoJew'),
            ('2000_09', 'CIAOthWithJew'),
            ('2000_09',  'CIABelieverAll'),
            ('2000_09',   'CIANoRelAll'),
            
            ('2010_20',    'CIAProtAll'),
            ('2010_20',    'CIACathAll'),
            ('2010_20',    'CIAOrthAll'),
            ('2010_20',     'CIAChrAll'),
            ('2010_20',     'CIAMusAll'),
            ('2010_20',     'CIABudAll'),
            ('2010_20',    'CIAHindAll'),
            ('2010_20', 'CIABudHindAll'),
            ('2010_20',     'CIAJewAll'),
            ('2010_20',   'CIAOthNoJew'),
            ('2010_20', 'CIAOthWithJew'),
            ('2010_20',  'CIABelieverAll'),
            ('2010_20',   'CIANoRelAll'),

            ('overall',    'CIAProtAll'),
            ('overall',    'CIACathAll'),
            ('overall',    'CIAOrthAll'),
            ('overall',     'CIAChrAll'),
            ('overall',     'CIAMusAll'),
            ('overall',     'CIABudAll'),
            ('overall',    'CIAHindAll'),
            ('overall', 'CIABudHindAll'),
            ('overall',     'CIAJewAll'),
            ('overall',   'CIAOthNoJew'),
            ('overall', 'CIAOthWithJew'),
            ('overall',  'CIABelieverAll'),
            ('overall',   'CIANoRelAll')
            ] 

#reduce columns of dataframe
df_a=a.loc[a['desc'].isin(fd_list)]
df_a.drop(df_a.columns.difference(my_col_list),axis=1, inplace=True )

###########
# PRINTS the world avg by decade
############
w=df_a.loc[df_a['FIPS']=='wld']
print('FD order is NOT correct!!!!\n',w)

#move decade values from columns to rows (make long)
b=pd.melt(df_a,id_vars=['FIPS','country_relig','desc'])
b.rename(columns={'variable':'year'},inplace=True)
b=b.loc[(b['year']!=0)]
wcnt=b.loc[b['FIPS']=='wld']
df_wcnt=b.groupby(['year'])['FIPS'].nunique()
print('Unique FIPS per decade for world:\n',df_wcnt)

#generate pivot_table with mean values
df_b_piv = np.round(b.pivot_table(index=['year','desc'], columns='country_relig', values='value', aggfunc=['mean'],margins=True),3)

#update row, then column order
piv = df_b_piv.reindex(row_order, axis=0)
piv = piv.reindex(col_order, axis=1)

print (df_b_piv)

#Calc counts
relig_cnt = b.groupby(['year','country_relig'])['FIPS'].nunique()

#print(relig_cnt.index.to_list) # to see row index
print(relig_cnt)
piv_cnt = relig_cnt.reindex(row_cnt_order, axis=0)
print ('piv_cnt:',piv_cnt)

piv.rename(columns={'CIAProtAll':'Prot', 'CIACathAll':'Cath', 'CIAOrthAll':'Orth', 'CIAChrAll':'Chr', 
            'CIAMusAll':'Mus', 'CIABudAll':'Bud', 'CIAHindAll':'Hind', 'CIABudHindAll':'BudHind',
            'CIAJewAll':'Jew', 'CIAOthNoJew':'Oth-NoJew', 'CIAOthWithJew':'Oth-Jew', 
            'CIABelieverAll':'Believers','CIANoRelAll':'No-Rel'},inplace=True)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_3_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_3_cnt.xlsx'
joined_path = os.path.join(mypath, myfile)
piv_cnt.to_excel(joined_path)

myfile = 'nTable_3_World.xlsx'
joined_path = os.path.join(mypath, myfile)
w.to_excel(joined_path)

time_frame=['overall','1980_89','1990_99','2000_09','2010_20']

#####################  
# compare_means does not work  
################################
#compare_means(df_a, 'decades')

'''
#######OLD method
temp=[]
for decade in my_decades_list: 
    for curr_relig in relig_list:
        for curr_col in fd_list:
            df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==curr_relig)]
            temp.append({'country_relig':curr_relig,'year':decade,'desc':curr_col,'value':df_a[decade].mean()})
dftemp = pd.DataFrame.from_dict(temp) 

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dftemp.pivot_table(
    index=['year','desc'], columns=['country_relig'], 
    values=['value'], aggfunc=['mean','count'], sort=False),3)
#world values df
dftemp = pd.DataFrame.from_dict(temp)
dftemp["desc"] = dftemp["desc"].astype('category')

wrld=np.round(dftemp.groupby(['desc','year'])['value'].mean(),3)

#REFRESH the data for the counts

a=pd.melt(df_iset_subset,id_vars=['country','FIPS','desc'])
a.rename(columns={'variable':'year'},inplace=True)

#Merge with religion data
a=a.merge(df_all_religions,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)
a=a.loc[~a['value'].isnull()]
a=a.loc[a['year'].isin(my_decades_list)]
relig_cnt=a.groupby(['year','country_relig'])['FIPS'].nunique()
wrld_cnt=a.groupby(['year'])['FIPS'].nunique()
print ('WO|RLD:',wrld_cnt,'BY RELIG:', relig_cnt)
'''





###NOT USED For Table 3... Unequal variance/sample size: Welsh's ANOVA & Games Howell multiple comparison of means

In [ ]:

'''
#perform Welch's ANOVA (for unequal variances)

print("Welch's ANOVA for unequal variances\n",)
print(pg.welch_anova(dv='value', between='country_relig', data=a))

my_decades_list=['overall','1980_89','1990_99','2000_09','2010_20']
b=copy.deepcopy(b.loc[b['year']=='overall'])
a=b
print("\n\nGames Howell multiple comparison of means for unequal variances and/or unequal sample sizes\n")

for curr_decade in my_decades_list:
    for curr_dv in b['desc'].unique():
        c=a.loc[(a['year']==curr_decade) & (a['desc']==curr_dv)]
        try:
            res=pg.pairwise_gameshowell(dv='value', between='country_relig', data=c)
            print('\n',curr_decade, curr_dv,':\n')
            signif=res.loc[res['pval']<.05]
            print (signif[['A','B','pval']])
        except:
            pass

a=a.loc[a['year'].isin(my_decades_list)]

for curr_relig in relig_list:
    for curr_dv in b['desc'].unique():
        c=a.loc[(a['country_relig']==curr_relig) & (a['desc']==curr_dv)]
        try:
            res=pg.pairwise_gameshowell(dv='value', between='year', data=c)
            print('\n',curr_relig, curr_dv,':\n')
            signif=res.loc[res['pval']<.05]
            print (signif[['A','B','pval']])
        except:
            pass
'''

###NOT USED: Original Table 3 code

In [ ]:
#Print mean values for table 3
'''
my_decades_list=['overall','1980_89','1990_99','2000_09','2010_20']

temp=[]
for decade in my_decades_list: 
    for curr_relig in relig_list:
        collst=a['desc'].unique()
        for curr_col in collst:
            df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==curr_relig) & (a['year']==decade)].copy(deep=True)
            temp.append({'country_relig':curr_relig,'year':decade,'desc':curr_col,'value':df_a['value'].mean()})
dftemp = pd.DataFrame.from_dict(temp) 

fd_list=pd.Categorical(['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index'], ordered=True)

for decade in my_decades_list:
    dftemp = pd.DataFrame.from_dict(temp)
    dftemp = dftemp.loc[dftemp['year']==decade].copy(deep=True) 
    dftemp["country_relig"] = dftemp["country_relig"].astype('category')
    dftemp["desc"] = dftemp["desc"].astype('category')

    #sort=False keeps the order of the columns=['country_relig'] in pivot_table
    piv=np.round(dftemp.pivot_table(index='desc', columns=['country_relig'], values='value', sort=False),3)

    #sort rows in pivot_table
    piv=piv.reindex(fd_list,axis=0)
    print ('piv for:',decade,'\n',piv)

    mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
    myfile = 'nTable_3_'+decade+'.xlsx'
    joined_path = os.path.join(mypath, myfile)
    piv.to_excel(joined_path)

dftemp = pd.DataFrame.from_dict(temp)
dftemp["desc"] = dftemp["desc"].astype('category')

wrld=np.round(dftemp.groupby(['desc','year'])['value'].mean(),3)

#REFRESH the data for the counts
a=pd.melt(df_iset_subset,id_vars=['country','FIPS','desc'])
a.rename(columns={'variable':'year'},inplace=True)

#Merge with religion data
a=a.merge(df_all_religions,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)
a=a.loc[~a['value'].isnull()]
a=a.loc[a['year'].isin(my_decades_list)]
r=a.groupby(['year','country_relig'])['FIPS'].nunique()
w=a.groupby(['year'])['FIPS'].nunique()
#print(a.groupby(['year','country_relig'])['FIPS'].nunique())
print ('WORLD:',w,'BY RELIG:', r)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_3_wrld_values.xlsx'
joined_path = os.path.join(mypath, myfile)
wrld.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_3_wrld_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
w.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_3_relig_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
r.to_excel(joined_path)

'''

##Table x. Religion and settler mortality


In [ ]:
#REFRESH the data for the counts
'''
c=pd.melt(df_iset_subset,id_vars=['country','FIPS','desc'])
c.rename(columns={'variable':'year'},inplace=True)

#Merge with religion data
b=c.merge(df_all_religions,on='FIPS',how='outer')
a=b.merge(df_country,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)
'''
a=df_merged.copy(deep=True)

#filter dataframe
#dftemp=a.loc[a['year']=='overall']
dftemp=a.loc[~a['overall'].isnull()]
dftemp=dftemp.loc[~dftemp['zb_BeckSettlerMortality'].isnull()]


#make bins
labels=['lo_25','lomed_25', 'himed_25', 'hi_25']
make_bins(df2split=dftemp,
          split_into=4,
          val_col=['zb_BeckSettlerMortality'],
          col_name='zb_BeckSettlerMortality',
          labels=labels)


'''bins=['bin_zb_BeckSettlerMortality_dist', 'bin_zb_BeckSettlerMortality_jenkspy','bin_zb_BeckSettlerMortality_freq']
for curr_bin in bins:
    df_sort = dftemp.reindex(dftemp['zb_BeckSettlerMortality'].sort_values(ascending=True).index)
    fig = plt.figure(figsize=(20,5))
    ax = sns.barplot(x='FIPS', y="zb_BeckSettlerMortality", hue=curr_bin, data=df_sort)
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
    plt.show()
'''
use_bin = 'bin_zb_BeckSettlerMortality_freq'

#calculate counts 
temp=[]
for level_desc in labels:
    for curr_relig in relig_list:
        df_a=dftemp.loc[(dftemp[use_bin]==level_desc) & (dftemp['country_relig']==curr_relig)].copy(deep=True)
        #df_a=df_a['FIPS'].unique()
        if len(df_a)>0:
            temp.append({use_bin:level_desc, 'country_relig':curr_relig, 'cnt':df_a['FIPS'].nunique()})

dftemp2 = pd.DataFrame.from_dict(temp) 

fd_list=pd.Categorical(['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index'], ordered=True)

dftemp.reset_index(drop=True)

dftemp=dftemp.merge(dftemp2,on=[use_bin,'country_relig'],how='outer')
dftemp=dftemp.loc[dftemp['desc'].isin(fd_list)]
dftemp.rename(columns={'overall':'value'},inplace=True)

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(
             pd.pivot_table(data=dftemp, index=[use_bin,'country_relig','cnt'], 
                            columns=['desc'], values=['value'], margins=True, sort=False
                            ), 3
             )

print('here are the ALL values:\n',piv)
#piv.plot(kind='bar')
#plt.show()

#sort columns in pivot_table
#     source: https://towardsdatascience.com/how-to-use-multiindex-in-pandas-to-level-up-your-analysis-aeac7f451fce
#print('indexes\n',piv.index.to_list)
#print('cols\n',piv.columns)


order = [('value', 'Financial Development Index'), 
         ('value', 'Financial Institutions Index'), 
         ('value', 'Financial Institutions Depth Index'), 
         ('value', 'Financial Institutions Access Index'), 
         ('value', 'Financial Institutions Efficiency Index'),
         ('value', 'Financial Markets Index'),
         ('value', 'Financial Markets Depth Index'), 
         ('value', 'Financial Markets Access Index'), 
         ('value', 'Financial Markets Efficiency Index')
        ]


piv = piv.reindex(order, axis=1)


print ('\nTable_x_relig_mortality:\n')
print(piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_x_relig_mortality.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#compare_means(dftemp, 'other')
 

##Table wgi_avg (average 1980-2020)


In [ ]:
#calculate the average for 6 wgi measures

##########
# lists of columns for this report
##########
avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

df_tmp=df_merged.copy(deep=True)

#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'wgi_avg'},inplace=True)

'''
#print(df_t2['wgi_avg'].head(20))
try:
    df_merged.drop(columns={'wgi_avg'},inplace=True)
except: 
    pass
df_merged=df_merged.join(df_t2['wgi_avg'],on='FIPS', how='outer')

#print ('@@@@@\n',df_merged['religiosity'].describe(),c['religiosity'].describe())
'''
#make bins
labels=['a_lo_25','b_lomed_25', 'c_himed_25', 'd_hi_25']
make_bins(df2split=df_t2,
          split_into=4,
          val_col=['wgi_avg'],
          col_name='wgi_avg',
          labels=labels)

bins=['bin_wgi_avg_jenkspy','bin_wgi_avg_freq', 'bin_wgi_avg_dist']
use_bin='bin_wgi_avg_freq'

df_relig_wgi=df_t2.groupby([use_bin,'country_relig']).agg({'wgi_avg':'mean', 'FIPS':pd.Series.nunique},margins=True)

'''
print('\nSeparation into WGI_avg groups: (df_relig_wgi).',
      '\nShowing wgi_avg (which is not reported). ',
      '\nMight lose some count when FD values are N/A\n', df_relig_wgi)
'''

# add additional columns, limit the desc values to only the FD list
df_r0 = df_tmp.loc[df_tmp['desc'].isin(fd_list)].copy(deep=True)
df_r1=df_t2.merge(df_r0,on='FIPS',how='outer')

#rename desc values (to allow pivot_table to correctly sort). Will search/replace "a_", "b_" with nothing in Excel
df_r1.rename(columns={'country_relig_x':'country_relig'},inplace=True)
relig_replace_lst={'CIAProtAll':'a_Prot', 'CIACathAll':'b_Cath', 'CIAOrthAll':'c_Orth', 
                      'CIAChrAll':'d_Chr', 'CIAMusAll':'e_Mus', 'CIABudAll':'f_Bud', 
                      'CIAHindAll':'g_Hind', 'CIABudHindAll':'h_BudHind',
                      'CIAJewAll':'i_Jew', 'CIAOthNoJew':'j_Oth-NoJew', 
                      'CIAOthWithJew':'k_Oth-Jew', 'CIABelieverAll':'l_Believers',
                      'CIANoRelAll':'m_No-Rel'}
df_r1.replace({'country_relig':relig_replace_lst},inplace=True)

fd_replace_lst= {'Financial Development Index':'a_FD',
               'Financial Institutions Index':'b_FI',
               'Financial Institutions Depth Index':'c_FID',
               'Financial Institutions Access Index':'d_FIA',
               'Financial Institutions Efficiency Index':'e_FIE',
               'Financial Markets Index':'f_FM',
               'Financial Markets Depth Index':'g_FMD',
               'Financial Markets Access Index':'h_FMA',
               'Financial Markets Efficiency Index':'i_FME'}
df_r1.replace({'desc':fd_replace_lst},inplace=True)

print('\ndf_r1.head\n',df_r1.head(3))
#df_t2 has level, relig, fips
#df_tmp has fips, fd

df_r1.rename(columns={'overall':'value'},inplace=True)

#create one pivot table for 'overall' for each FD; another for 'FIPS' nunique() for bin:country_relig index
wgi_cnt=pivot_table_w_subtotals(df_r1, indices=['bin_wgi_avg_freq','country_relig'], columns=['desc'], values=['FIPS'], aggfunc=[pd.Series.nunique], fill_value='')
print('\nwgi_cnt\n',wgi_cnt)

wgi_avg=pivot_table_w_subtotals(df_r1, indices=['bin_wgi_avg_freq','country_relig'], columns=['desc'], values=['value'], aggfunc=[np.mean], fill_value='')

print('\nwgi_avg: FD avg\n',wgi_avg)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_wgi_avg_main.xlsx'
joined_path = os.path.join(mypath, myfile)
wgi_avg.to_excel(joined_path)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_wgi_avg_counts.xlsx'
joined_path = os.path.join(mypath, myfile)
wgi_cnt.to_excel(joined_path)

#compare_means(df_r1, 'other')



##Religiosity as NoRel%

In [ ]:
#'CIANoRel%'
df_tmp=df_merged.copy(deep=True)

col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of latitude

df_t1=df_tmp.copy(deep=True)
df_t2=df_t1[['FIPS','country_relig', 'desc','overall','CIANoRel%']]
df_t2=df_t2.loc[df_t2['desc'].isin(fd_list)]
df_t2.drop_duplicates(inplace=True, ignore_index=True)


df_t2.rename(columns={'CIANoRel%':'CIANoRel_per'},inplace=True)
df_t2['CIANoRel_per'].fillna(0)
df_t2['CIANoRel_per']=pd.to_numeric(df_t2['CIANoRel_per'], errors='coerce')
#make bins
labels=['lo_25','lomed_25', 'himed_25', 'hi_25']

make_bins(df2split=df_t2,
          split_into=4,
          val_col=['CIANoRel_per'],
          col_name='CIANoRel_per',
          labels=labels)

bins=['bin_CIANoRel_per_dist','bin_CIANoRel_per_jenkspy','bin_CIANoRel_per_freq']

df_NoRel_avg=df_t2.copy(deep=True)
#counts
use_bin='bin_CIANoRel_per_freq'

print(len(df_t2.loc[df_t2['bin_CIANoRel_per_freq']=='lo_25']),len(df_t2.loc[df_t2['bin_CIANoRel_per_freq']=='hi_25']))

#create one pivot table for 'overall' for each FD; another for 'FIPS' nunique() for bin:country_relig index
CIANoRel_per_cnt=pivot_table_w_subtotals(df_t2, indices=['bin_CIANoRel_per_freq','country_relig'], columns=['desc'], values=['FIPS'], aggfunc=[pd.Series.nunique], fill_value='')
print('\nCIANoRel%_cnt\n',CIANoRel_per_cnt)

CIANoRel_per_avg=pivot_table_w_subtotals(df_t2, indices=['bin_CIANoRel_per_freq','country_relig'], columns=['desc'], values=['overall'], aggfunc=[np.mean], fill_value='')

print('\nbin_CIANoRel%_freqavg: FD avg\n',CIANoRel_per_avg)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_CIANoRel_percent_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#compare_means(df_tmp, 'other')





##GDP groups

In [ ]:
## not currently working

#calculate the average for NoRel% over time

df_tmp=df_merged.copy(deep=True)

#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc']=='GDP')].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'GDP_avg'},inplace=True)

#make bins
labels=['a_lo_25','b_lomed_25', 'c_himed_25', 'd_hi_25']
make_bins(df2split=df_t2,
          split_into=4,
          val_col=['GDP_avg'],
          col_name='GDP_avg',
          labels=labels)

bins=['bin_GDP_avg_jenkspy','bin_GDP_avg_freq', 'bin_GDP_avg_dist']
use_bin='bin_GDP_avg_freq'

df_relig_gdp=df_t2.groupby([use_bin,'country_relig']).agg({'GDP_avg':'mean', 'FIPS':pd.Series.nunique},margins=True)
print('df_t2\n',df_t2.head(),'\ndf_relig_gdp\n',df_relig_gdp)

# add additional columns, limit the desc values to only the FD list
df_r0 = df_tmp.loc[df_tmp['desc'].isin(fd_list)].copy(deep=True)
df_r1=df_t2.append(df_r0)

#df_r1=df_t2.merge(df_r0[['FIPS','desc','overall']],on='FIPS',how='outer')

#rename desc values (to allow pivot_table to correctly sort). Will search/replace "a_", "b_" with nothing in Excel
df_r1.rename(columns={'country_relig_x':'country_relig'},inplace=True)
relig_replace_lst={'CIAProtAll':'a_Prot', 'CIACathAll':'b_Cath', 'CIAOrthAll':'c_Orth', 
                      'CIAChrAll':'d_Chr', 'CIAMusAll':'e_Mus', 'CIABudAll':'f_Bud', 
                      'CIAHindAll':'g_Hind', 'CIABudHindAll':'h_BudHind',
                      'CIAJewAll':'i_Jew', 'CIAOthNoJew':'j_Oth-NoJew', 
                      'CIAOthWithJew':'k_Oth-Jew', 'CIABelieverAll':'l_Believers',
                      'CIANoRelAll':'m_No-Rel'}
df_r1.replace({'country_relig':relig_replace_lst},inplace=True)

fd_replace_lst= {'Financial Development Index':'a_FD',
               'Financial Institutions Index':'b_FI',
               'Financial Institutions Depth Index':'c_FID',
               'Financial Institutions Access Index':'d_FIA',
               'Financial Institutions Efficiency Index':'e_FIE',
               'Financial Markets Index':'f_FM',
               'Financial Markets Depth Index':'g_FMD',
               'Financial Markets Access Index':'h_FMA',
               'Financial Markets Efficiency Index':'i_FME'}
df_r1.replace({'desc':fd_replace_lst},inplace=True)

print('\ndf_r1.head\n',df_r1.head(3),'overall:\n',df_r1['overall'].unique())
#df_t2 has level, relig, fips
#df_tmp has fips, fd

df_r1.rename(columns={'overall':'value'},inplace=True)




#create one pivot table for 'overall' for each FD; another for 'FIPS' nunique() for bin:country_relig index
GDP_cnt=pivot_table_w_subtotals(df_r1, indices=['bin_GDP_avg_freq','country_relig'], columns=['desc'], values=['FIPS'], aggfunc=[pd.Series.nunique], fill_value='')
print('\nwgi_cnt\n',wgi_cnt)

GDP_avg=pivot_table_w_subtotals(df_r1, indices=['bin_GDP_avg_freq','country_relig'], columns=['desc'], values=['value'], aggfunc=[np.mean], fill_value='')

print('\nGDP_avg: FD avg\n',GDP_avg)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GDP_avg_main.xlsx'
joined_path = os.path.join(mypath, myfile)
GDP_avg.to_excel(joined_path)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GDP_avg_counts.xlsx'
joined_path = os.path.join(mypath, myfile)
GDP_cnt.to_excel(joined_path)

#compare_means(df_r1, 'other')



In [ ]:
#print(df_merged.loc[df_merged['wgi_avg']>1.6],'\n\n',df_t2.loc[df_t2['wgi_avg']>1.6])

###OLD -not used

In [ ]:
'''df_tmp=df_merged.copy(deep=True)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']
#---------------------------
# clean-up column names --remove spaces and special characters
#---------------------------
avg_col_lst = [curr_col.strip() for curr_col in avg_col_lst] #remove leading & trailing spaces

new_strings = []
for string in avg_col_lst:
    new_string = string.replace(" ", "_") 
    new_string = new_string.replace(":", "") 
    new_string = new_string.replace("/", "_and_") 
    new_strings.append(new_string) 
new_avg_col_lst = new_strings

#Convert to set to eliminate redundant pairs

combo=new_avg_col_lst+avg_col_lst
perm_set=list( permutations( combo, 2 ) )
perm_set=set(perm_set)
df_tmp['desc']

rename_lst = {'Voice and Accountability: Estimate':'Voice_and_Accountability_Estimate',
            'Government Effectiveness: Estimate': 'Government_Effectiveness_Estimate', 
            'Regulatory Quality: Estimate': 'Regulatory_Quality_Estimate', 
            'Control of Corruption: Estimate': 'Control_of_Corruption_Estimate', 
            'Rule of Law: Estimate': 'Rule_of_Law_Estimate', 
            'Political Stability and Absence of Violence/Terrorism: Estimate': 'Political_Stability_and_Absence_of_Violence_and_Terrorism_Estimate'}
 
for k, v in rename_lst.items():
    df_tmp['desc']=df_tmp['desc'].replace(k,v)

print(df_tmp['desc'].unique())

#slice df to only keep desired metrics

print (avg_col_lst, df_tmp.info(verbose=True))

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

temp=[]
for curr_relig in relig_list:
    df_a=df_tmp.loc[(df_tmp['country_relig']==curr_relig)]
    for curr_col in desc_col_lst:
            df_col=df_a.loc[(df_a['desc']==curr_col)]
            
            #calculate the average for 6 wgi measures
            wgi_avg=df_col[new_avg_col_lst].mean(axis=1)
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'value':df_col['overall'].mean(),
                        'wgi_avg':wgi_avg})
dfloop = pd.DataFrame.from_dict(temp) 

print(dfloop.head(10))
#sort=False keeps the order of the columns=['country_relig'] in pivot_table

piv=np.round(dfloop.pivot_table(
    index=['country_relig','desc'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)
row_order =['HDI Score', 
         'UN Income Index',
         'Gini index (World Bank estimate)', 
         'Multidimensional Poverty Index',
         'UN Life Expectancy Index', 
         'Life expectancy at birth, total (years)',
         'UIS: Mean years of schooling (ISCED 1 or higher), population 25+ years, both sexes',
         'Prison per 100K', 
         'sfi',
         'actotal', 
         'autoc']

piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',   'CIANoRelAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew')
        ]        
#piv = piv.reindex(order, axis=1)

#print ('piv',piv)
'''

##Table Religiosity 

In [ ]:
df_tmp=df_merged.copy(deep=True)

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

col_avg=['Bel_Afterlife_1_yes', 
         'Bel_God_1_yes',
         'Bel_heaven_1_yes',
         'Bel_hell_1_yes',
         'God_importance_High',
         'Relig_attend_lo',
         'Relig_person_lo']

keep_cols=[('overall',    'Bel_Afterlife_1_yes'), 
         ('overall',    'Bel_God_1_yes'),
         ('overall',    'Bel_heaven_1_yes'),
         ('overall',    'Bel_hell_1_yes'),
         ('overall',    'God_importance_High'),
         ('overall',    'Relig_attend_lo'),
         ('overall',    'Relig_person_lo')]

#Get the religiosity value alone, normalize & transform scales, then calc mean
df_tmp0=np.round(df_tmp.pivot_table(index=['FIPS', 'country', 'country_relig'], columns=['desc'],values=['overall']),3)

#Drop multilevel index and reset index to see FIPS
df_tmp0.columns = df_tmp0.columns.droplevel(0) 
df_tmp0.reset_index(inplace=True, drop=False)

df_tmp0.drop(df_tmp0.columns.difference(['FIPS', 'country', 'country_relig',
                                        'Bel_Afterlife_1_yes', 
                                        'Bel_God_1_yes',
                                        'Bel_heaven_1_yes',
                                        'Bel_hell_1_yes',
                                        'God_importance_High',
                                        'Relig_attend_lo',
                                        'Relig_person_lo']), axis=1,inplace=True)

#reverse scale for low-scored values
df_tmp0['Relig_attend_lo']=df_tmp0['Relig_attend_lo'].apply(lambda x: 1/x if x>0 else 1/1000)
df_tmp0['Relig_person_lo']=df_tmp0['Relig_person_lo'].apply(lambda x: 1/x if x>0 else 1/1000)

#print('after reverse scale\n',df_tmp0.columns.to_list())
#normalize col_avg values
for curr_col in col_avg:
    df_tmp0[curr_col] = (df_tmp0[curr_col] - df_tmp0[curr_col].min()) / (df_tmp0[curr_col].max() - df_tmp0[curr_col].min())    


print ('\ndf_tmp0.columns.to_list():\n',df_tmp0.columns.to_list())
df_tmp0['religiosity']=df_tmp0.mean(numeric_only=True, axis=1)
df_tmp0['religiosity'].replace(0, np.nan, inplace=True)
#df_merged['religiosity']=df_tmp0['religiosity']

print(df_tmp0.loc[df_tmp0['religiosity']>.6])

#make long
c=pd.melt(df_tmp0,id_vars=['FIPS','country', 'country_relig','religiosity'])
c.drop(columns='desc',inplace=True)                             
c.drop_duplicates(inplace=True, ignore_index=True)
#print(c.head(100))

#save 'religiosity' column to df_merged 
c.set_index('FIPS', inplace=True)
try:
    df_merged.drop(columns={'religiosity'},inplace=True)
except:
    pass
df_merged=df_merged.join(c['religiosity'],how='outer')
print ('@@@@@\n',df_merged['religiosity'].describe(),c['religiosity'].describe())

#make bins
labels=['lo_25','lomed_25', 'himed_25', 'hi_25']

make_bins(df2split=c,
          split_into=4,
          val_col=['religiosity'],
          col_name='religiosity',
          labels=labels)

bins=['bin_religiosity_jenkspy','bin_religiosity_freq', 'bin_religiosity_dist']
'''
for curr_bin in bins:
    c=c.loc[c['religiosity']>0]
    df_sort1 = c.reindex(c['religiosity'].sort_values(ascending=True).index)
    fig1 = plt.figure(figsize=(20,8))
    ax1 = sns.barplot(x='FIPS', y="religiosity", hue=curr_bin, data=df_sort1)
    #ax1.set_xticklabels(ax1.get_xticklabels(),rotation = 90)
    plt.show()
'''
df_t2=df_tmp.merge(c,on=['FIPS','country','country_relig'],how='outer')

df_religiosity=df_t2.copy(deep=True)


#counts
use_bin='bin_religiosity_freq'

# by "LEVEL", by religion, by FD (column)
temp=[]
for curr_level in labels:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_t2.loc[(df_t2['bin_religiosity_freq']==curr_level)&(df_t2['country_relig']==curr_relig)].copy(deep=True)
            df_col1=df_col.loc[(df_col['desc']==curr_col)]
            temp.append({'country_relig':curr_relig,
                        'cnt':df_col['FIPS'].nunique(),
                        'desc':curr_col,
                        'bin_religiosity_freq':curr_level,
                        'value':df_col1['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dfloop.pivot_table(
    index=['bin_religiosity_freq','country_relig','cnt'], columns=['desc'], 
    values=['value'], sort=False),3)


#Sort Row order
#print(piv.index.to_list)

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

     
col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)

piv.sort_index(axis='rows', level='country_relig')
print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_religiosity_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#compare_means(df_tmp, 'other')





In [ ]:
#df_tmp0.set_index('FIPS')
#df_merged.set_index('FIPS')

#Sort Row order
print(df_tmp0.index.to_list)

#Sort Col order
print(df_tmp0.columns.to_list)

##Table latitude

In [ ]:
df_tmp=df_merged.copy(deep=True)

col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of latitude

df_t1=df_tmp.copy(deep=True)
df_t2=df_t1[['FIPS','country_relig', 'desc','overall','AbsLatitude']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)

#make bins
labels=['lo_25','lomed_25', 'himed_25', 'hi_25']

make_bins(df2split=df_t2,
          split_into=4,
          val_col=['AbsLatitude'],
          col_name='AbsLatitude',
          labels=labels)

bins=['bin_AbsLatitude_jenkspy','bin_AbsLatitude_freq', 'bin_AbsLatitude_dist']
'''
for curr_bin in bins:
    df_t2=df_t2.loc[df_t2['AbsLatitude']>0]
    df_sort = df_t2.reindex(df_t2['AbsLatitude'].sort_values(ascending=True).index)
    fig = plt.figure(figsize=(40,8))
    ax = sns.barplot(x='FIPS', y="AbsLatitude", hue=curr_bin, data=df_sort)
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
    plt.show()
'''
#counts
use_bin='bin_AbsLatitude_freq'

# by lat_level, by religion, by FD (column)
temp=[]
for curr_level in labels:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_t2.loc[(df_t2['bin_AbsLatitude_freq']==curr_level)&(df_t2['country_relig']==curr_relig)].copy(deep=True)
            df_col1=df_col.loc[(df_col['desc']==curr_col)]
            temp.append({'country_relig':curr_relig,
                        'cnt':df_col['FIPS'].nunique(),
                        'desc':curr_col,
                        'bin_AbsLatitude_freq':curr_level,
                        'value':df_col1['overall'].mean()})
            
dfloop = pd.DataFrame.from_dict(temp) 

dan=pivot_table_w_subtotals(dfloop, values=['value'], indices=['bin_AbsLatitude_freq','country_relig','cnt'], columns=['desc'], aggfunc=[np.mean], fill_value='')
print('\ndan\n',dan)
#sort=False keeps the order of the columns=['country_relig'] in pivot_table

piv=np.round(dfloop.pivot_table(
    index=['bin_AbsLatitude_freq','country_relig','cnt'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)


#piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

piv = piv.reindex(col_order, axis=1)
print('piv:\n',piv.head(50))

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_latitude_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#compare_means(df_tmp, 'other')





##Table Population size

In [ ]:
df_tmp=df_merged.copy(deep=True)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of latitude

df_t1=df_tmp.copy(deep=True)
df_t2=df_t1[['FIPS','country_relig', 'desc','overall','ze_Population (millions)']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)
df_t2.rename(columns={'ze_Population (millions)':'Population_millions'},inplace=True)

#make bins
labels=['lo_25','lomed_25', 'himed_25', 'hi_25']

make_bins(df2split=df_t2,
          split_into=4,
          val_col=['Population_millions'],
          col_name='Population_millions',
          labels=labels)

bins=['bin_Population_millions_jenkspy','bin_Population_millions_freq', 'bin_Population_millions_dist']

# Calc FD metric averages for each grp
temp=[]
for curr_grp in labels: 
    df_a=df_t2.loc[df_t2['bin_Population_millions_freq']==curr_grp]
    level_cnt=df_a['FIPS'].nunique()
    for curr_col in desc_col_lst:
        df_a=df_t2.loc[df_t2['desc']==curr_col]
        df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
        df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
        temp.append({'bin_Population_millions_freq':curr_grp,'desc':curr_col,'avg':df_a['overall'].mean(),'cnt':df_a['FIPS'].nunique(), 'lev_cnt':level_cnt})
dfgroups = pd.DataFrame.from_dict(temp) 
print('Averages for each inc_grp:\n', dfgroups)

#counts
use_bin='bin_Population_millions_freq'

# by lat_level, by religion, by FD (column)
temp=[]
for curr_level in labels:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_t2.loc[(df_t2['bin_Population_millions_freq']==curr_level)&(df_t2['country_relig']==curr_relig)].copy(deep=True)
            df_col1=df_col.loc[(df_col['desc']==curr_col)]
            temp.append({'country_relig':curr_relig,
                        'cnt':df_col['FIPS'].nunique(),
                        'desc':curr_col,
                        'bin_Population_millions_freq':curr_level,
                        'value':df_col1['overall'].mean()})
            
dfloop = pd.DataFrame.from_dict(temp) 

#sort=False keeps the order of the columns=['country_relig'] in pivot_table

piv=np.round(dfloop.pivot_table(
    index=['bin_Population_millions_freq','country_relig','cnt'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)

row_order =['HDI', 
            'GNI per capita', 
            'Gini 2010-18', 
            'Population (millions)', 
            'Inequality in income %', 
            'Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)',
            'Poverty gap at $3.20 a day (2011 PPP) (%)',
            'Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population)',
            'Multidimensional Poverty Index',
            'Life expectancy at birth', 
            'Expected years of schooling', 
            'Prison population (per 100K) 2013-2018', 
            'sfi',
            'actotal', 
            'autoc']

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        

col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)
print('piv:\n',piv.head(50))

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_Populations_size_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

myfile = 'nTable_Populations_group_means.xlsx'
joined_path = os.path.join(mypath, myfile)
dfgroups.to_excel(joined_path)


#compare_means(df_tmp, 'other')





##Table Continents

In [ ]:
df_tmp=df_merged.copy(deep=True)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

df_t2=df_tmp[['FIPS','country','country_relig','desc','overall','continent']].copy(deep=True)
df_t2.drop_duplicates(inplace=True, ignore_index=True)

temp=[]
for curr_continent in df_t2['continent'].unique():
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_t2.loc[(df_t2['continent']==curr_continent)&(df_t2['country_relig']==curr_relig)].copy(deep=True)
            df_col1=df_col.loc[(df_col['desc']==curr_col)]
            temp.append({'country_relig':curr_relig,
                         'cnt':df_col['FIPS'].nunique(),
                         'desc':curr_col,
                         'continent':curr_continent,
                         'value':df_col1['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 


#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dfloop.pivot_table(
    index=['continent','country_relig','cnt'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)
row_order =['HDI Score', 
         'UN Income Index',
         'Gini index (World Bank estimate)', 
         'Multidimensional Poverty Index',
         'UN Life Expectancy Index', 
         'Life expectancy at birth, total (years)',
         'UIS: Mean years of schooling (ISCED 1 or higher), population 25+ years, both sexes',
         'Prison per 100K', 
         'sfi',
         'actotal', 
         'autoc']

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        

col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)

print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_continent_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

#compare_means(df_tmp, 'other')



##Table Regions

In [ ]:
df_tmp=df_merged.copy(deep=True)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of: latitude

df_t1=df_tmp.copy(deep=True)
df_t2=df_tmp[['FIPS','country','country_relig','desc','overall','region']].copy(deep=True)
df_t2.drop_duplicates(inplace=True, ignore_index=True)

temp=[]
for curr_region in df_t2['region'].unique():
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_t2.loc[(df_t2['region']==curr_region)&(df_t2['country_relig']==curr_relig)].copy(deep=True)
            df_col1=df_col.loc[(df_col['desc']==curr_col)]
            temp.append({'country_relig':curr_relig,
                         'cnt':df_col['FIPS'].nunique(),
                         'desc':curr_col,
                         'region':curr_region,
                         'value':df_col1['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 


#sort=False keeps the order of the columns=['country_relig'] in pivot_table

piv=np.round(dfloop.pivot_table(
    index=['region','country_relig','cnt'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)
row_order =['HDI Score', 
         'UN Income Index',
         'Gini index (World Bank estimate)', 
         'Multidimensional Poverty Index',
         'UN Life Expectancy Index', 
         'Life expectancy at birth, total (years)',
         'UIS: Mean years of schooling (ISCED 1 or higher), population 25+ years, both sexes',
         'Prison per 100K', 
         'sfi',
         'actotal', 
         'autoc']

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
#print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        

col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)

print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_region_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)


#compare_means(df_tmp, 'other')



##Table Colonized, Colonizer, NeitherCol

In [ ]:
'''df_tmp=df_merged.copy(deep=True)

desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of: Colonized, Colonizer, NeitherCol

df_t1=df_tmp.copy(deep=True)
df_t2=df_t1[['FIPS', 'country', 'desc','country_relig','Colonized', 'Colonizer', 'NeitherCol']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)

select_conditions = [
    (df_t2['Colonized'] ==1),
    (df_t2['Colonizer'] ==1),
    (df_t2['NeitherCol'] ==1)
    ]
select_values = ['Colonized', 'Colonizer', 'NeitherCol']

df_t2['my_level'] = np.select(select_conditions, select_values)

df_level=df_t2.groupby(['my_level','country_relig'],as_index=False).agg({'FIPS':'count'})

print('Separation into my_level groups:', df_level)

# add additional columns
df_r0 = df_tmp.loc[df_tmp['desc'].isin(desc_col_lst)].copy(deep=True)
df_r1=df_t2.merge(df_r0,on='FIPS',how='outer')

df_r1.rename(columns={'country_relig_x':'country_relig','desc_x':'desc', 'overall_x':'overall','Colonized_x':'Colonized', 'Colonizer_x':'Colonizer', 'NeitherCol_x':'NeitherCol'},inplace=True)
print('df_r1.head',df_r1.head(10))
#df_t2 has level, relig, fips
#df_tmp has fips, fd


# by lat_level, by religion, by FD (column)
temp=[]
for curr_level in select_values:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_r1.copy(deep=True)
            df_col1=df_col.loc[(df_col['my_level']==curr_level)&(df_col['desc']==curr_col)&(df_col['country_relig']==curr_relig)]
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'my_level':curr_level,
                        'value':df_col1['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 


#sort=False keeps the order of the columns=['country_relig'] in pivot_table
print ('dfloop:::::::',dfloop.head(15))
piv=np.round(dfloop.pivot_table(
    index=['my_level','country_relig'], columns=['desc'], 
    values=['value'], sort=False),3)

#Sort Row order
#print(piv.index.to_list)

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        
   
col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)

print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_colonizer_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_colonizer_counts.xlsx'
joined_path = os.path.join(mypath, myfile)
df_level.to_excel(joined_path)

time_frame=['overall']

#compare_means(df_tmp, other)
'''

##Table who_colonized

'COLONY_AUSTRALIA', 'COLONY_AUSTRO_HUNG', 'COLONY_BELGIUM', 'COLONY_BRITAIN', 'COLONY_CHINA', 'COLONY_DENMARK', 'COLONY_FRANCE', 'COLONY_GERMANY', 'COLONY_ITALY', 'COLONY_JAPAN', 'COLONY_NETH', 'COLONY_NEW_ZEALAND', 'COLONY_NORWAY', 'COLONY_PORTUGAL', 'COLONY_RUSSIA', 'COLONY_SPAIN', 'COLONY_TURKEY', 'COLONY_USA'

In [ ]:
'''
df_tmp=df_merged.copy(deep=True)
print(df_merged.info(verbose=True))

try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass
df_tmp=df_tmp.reset_index(drop=False)
try:
    df_tmp.drop(columns=['level_0'],inplace=True)
except:
    pass

df_tmp['colony_oth']= df_tmp['COLONY_AUSTRALIA'] + df_tmp['COLONY_AUSTRO_HUNG'] + df_tmp['COLONY_BELGIUM'] + df_tmp['COLONY_CHINA'] + df_tmp['COLONY_DENMARK'] + df_tmp['COLONY_GERMANY'] + df_tmp['COLONY_ITALY'] + df_tmp['COLONY_JAPAN'] + df_tmp['COLONY_NETH'] + df_tmp['COLONY_NEW_ZEALAND'] + df_tmp['COLONY_NORWAY'] + df_tmp['COLONY_PORTUGAL'] + df_tmp['COLONY_RUSSIA'] + df_tmp['COLONY_USA']
colonizers=['COLONY_AUSTRALIA', 'COLONY_AUSTRO_HUNG', 'COLONY_BELGIUM', 'COLONY_BRITAIN', 'COLONY_CHINA', 'COLONY_DENMARK', 'COLONY_FRANCE', 'COLONY_GERMANY', 'COLONY_ITALY', 'COLONY_JAPAN', 'COLONY_NETH', 'COLONY_NEW_ZEALAND', 'COLONY_NORWAY', 'COLONY_PORTUGAL', 'COLONY_RUSSIA', 'COLONY_SPAIN', 'COLONY_TURKEY', 'COLONY_USA', 'colony_oth']


desc_col_lst =['Financial Development Index',
               'Financial Institutions Index',
               'Financial Institutions Depth Index',
               'Financial Institutions Access Index',
               'Financial Institutions Efficiency Index',
               'Financial Markets Index',
               'Financial Markets Depth Index',
               'Financial Markets Access Index',
               'Financial Markets Efficiency Index']

keep_col_lst=   ['FIPS',
                'country',
                'country_relig'
                ]

#Assign the level of: Colonized, Colonizer, NeitherCol

'''
'''
#df_t1=df_tmp.copy(deep=True)
#df_t2=df_t1[['FIPS','country_relig', 'desc','COLONY_AUSTRALIA', 'COLONY_AUSTRO_HUNG', 'COLONY_BELGIUM', 'COLONY_BRITAIN', 'COLONY_CHINA', 'COLONY_DENMARK', 'COLONY_FRANCE', 'COLONY_GERMANY', 'COLONY_ITALY', 'COLONY_JAPAN', 'COLONY_NETH', 'COLONY_NEW_ZEALAND', 'COLONY_NORWAY', 'COLONY_PORTUGAL', 'COLONY_RUSSIA', 'COLONY_SPAIN', 'COLONY_TURKEY', 'COLONY_USA','colony_oth']]
#df_t2.drop_duplicates(inplace=True, ignore_index=True)

#save colonizer name into ['my_level'] column
for curr_colonizer in colonizers:
    df_t2.loc[df_t2[curr_colonizer]==1,'my_level']=curr_colonizer

df_level=df_t2.groupby(['my_level','country_relig'],as_index=False).agg({'FIPS':'count'})

print('Separation into my_level groups:', df_level)

df_level=df_tmp.groupby(['country_relig'])
# add additional columns
df_r0 = df_tmp.loc[df_tmp['desc'].isin(desc_col_lst)].copy(deep=True)
df_r1=df_t2.merge(df_r0,on='FIPS',how='outer')

df_r1.rename(columns={'country_relig_x':'country_relig','desc_x':'desc', 
                      'overall_x':'overall',
                      'COLONY_AUSTRALIA_x':'COLONY_AUSTRALIA', 
                      'COLONY_AUSTRO_HUNG_x':'COLONY_AUSTRO_HUNG', 
                      'COLONY_BELGIUM_x':'COLONY_BELGIUM', 
                      'COLONY_BRITAIN_x':'COLONY_BRITAIN', 
                      'COLONY_CHINA_x':'COLONY_CHINA', 
                      'COLONY_DENMARK_x':'COLONY_DENMARK', 
                      'COLONY_FRANCE_x':'COLONY_FRANCE', 
                      'COLONY_GERMANY_x':'COLONY_GERMANY', 
                      'COLONY_ITALY_x':'COLONY_ITALY', 
                      'COLONY_JAPAN_x':'COLONY_JAPAN', 
                      'COLONY_NETH_x':'COLONY_NETH', 
                      'COLONY_NEW_ZEALAND_x':'COLONY_NEW_ZEALAND', 
                      'COLONY_NORWAY_x':'COLONY_NORWAY', 
                      'COLONY_PORTUGAL_x':'COLONY_PORTUGAL', 
                      'COLONY_RUSSIA_x':'COLONY_RUSSIA', 
                      'COLONY_SPAIN_x':'COLONY_SPAIN', 
                      'COLONY_TURKEY_x':'COLONY_TURKEY', 
                      'COLONY_USA_x':'COLONY_USA'},inplace=True)
print('df_r1.head',df_r1.head(10))
#df_t2 has level, relig, fips
#df_tmp has fips, fd
'''
'''
df_tmp=df_tmp[['FIPS','country_relig', 'desc','COLONY_AUSTRALIA', 'COLONY_AUSTRO_HUNG', 'COLONY_BELGIUM', 'COLONY_BRITAIN', 'COLONY_CHINA', 'COLONY_DENMARK', 'COLONY_FRANCE', 'COLONY_GERMANY', 'COLONY_ITALY', 'COLONY_JAPAN', 'COLONY_NETH', 'COLONY_NEW_ZEALAND', 'COLONY_NORWAY', 'COLONY_PORTUGAL', 'COLONY_RUSSIA', 'COLONY_SPAIN', 'COLONY_TURKEY', 'COLONY_USA','colony_oth']]

df_tmp= df_tmp.loc[df_tmp['desc'].isin(desc_col_lst)].copy(deep=True)

print(df_tmp.head(200))

# by lat_level, by religion, by FD (column)
temp=[]
for curr_level in colonizers:
    for curr_relig in relig_list:
        for curr_col in desc_col_lst:
            df_col=df_tmp.copy(deep=True)
            df_col=df_col.loc[(df_col['my_level']==curr_level)&(df_col['desc']==curr_col)&(df_col['country_relig']==curr_relig)]
            temp.append({'country_relig':curr_relig,
                        'desc':curr_col,
                        'my_level':curr_level,
                        'value':df_col['overall'].mean()})
dfloop = pd.DataFrame.from_dict(temp) 


#sort=False keeps the order of the columns=['country_relig'] in pivot_table
print ('dfloop:::::::',dfloop.head(15))
piv=np.round(dfloop.pivot_table(
    index=['my_level','country_relig'], columns=['desc'], 
    values=['value'], sort=False),3)
'''
'''
piv.reset_index(inplace=True)
try:
    piv.drop(columns=['level_0'],inplace=True)
except:
    pass
'''
'''
#Sort Row order
#print(piv.index.to_list)
row_order =['HDI Score', 
         'UN Income Index',
         'Gini index (World Bank estimate)', 
         'Multidimensional Poverty Index',
         'UN Life Expectancy Index', 
         'Life expectancy at birth, total (years)',
         'UIS: Mean years of schooling (ISCED 1 or higher), population 25+ years, both sexes',
         'Prison per 100K', 
         'sfi',
         'actotal', 
         'autoc']

#piv = piv.reindex(row_order, axis=0)

#Sort Col order
print(piv.columns.to_list)

order = [   ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]        
col_order=[('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')
          ]
piv = piv.reindex(col_order, axis=1)
try:
    piv.drop(columns=['level_0'],inplace=True)
except:
    pass

print ('piv',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_who_colonized_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)


mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_who_colonized_counts.xlsx'
joined_path = os.path.join(mypath, myfile)
df_level.to_excel(joined_path)

#compare_means(df_tmp, 'other')


'''

##Table INC_GRP: Religion, income, finance

In [ ]:
col_order = ['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index']

order =    [(        'Low income',             'Financial Development Index'),
            (        'Low income',            'Financial Institutions Index'),
            (        'Low income',      'Financial Institutions Depth Index'),
            (        'Low income',     'Financial Institutions Access Index'),
            (        'Low income', 'Financial Institutions Efficiency Index'),
            (        'Low income',                 'Financial Markets Index'),
            (        'Low income',           'Financial Markets Depth Index'),
            (        'Low income',          'Financial Markets Access Index'),
            (        'Low income',      'Financial Markets Efficiency Index'),
            (        'Lower middle income',             'Financial Development Index'),
            (        'Lower middle income',            'Financial Institutions Index'),
            (        'Lower middle income',      'Financial Institutions Depth Index'),
            (        'Lower middle income',     'Financial Institutions Access Index'),
            (        'Lower middle income', 'Financial Institutions Efficiency Index'),
            (        'Lower middle income',                 'Financial Markets Index'),
            (        'Lower middle income',           'Financial Markets Depth Index'),
            (        'Lower middle income',          'Financial Markets Access Index'),
            (        'Lower middle income',      'Financial Markets Efficiency Index'),
            (        'Upper middle income',             'Financial Development Index'),
            (        'Upper middle income',            'Financial Institutions Index'),
            (        'Upper middle income',      'Financial Institutions Depth Index'),
            (        'Upper middle income',     'Financial Institutions Access Index'),
            (        'Upper middle income', 'Financial Institutions Efficiency Index'),
            (        'Upper middle income',                 'Financial Markets Index'),
            (        'Upper middle income',           'Financial Markets Depth Index'),
            (        'Upper middle income',          'Financial Markets Access Index'),
            (        'Upper middle income',      'Financial Markets Efficiency Index'),
            (        'High income',             'Financial Development Index'),
            (        'High income',            'Financial Institutions Index'),
            (        'High income',      'Financial Institutions Depth Index'),
            (        'High income',     'Financial Institutions Access Index'),
            (        'High income', 'Financial Institutions Efficiency Index'),
            (        'High income',                 'Financial Markets Index'),
            (        'High income',           'Financial Markets Depth Index'),
            (        'High income',          'Financial Markets Access Index'),
            (        'High income',      'Financial Markets Efficiency Index')]


'''
c=pd.melt(df_iset_subset,id_vars=['country','FIPS','desc'])
c.rename(columns={'variable':'year'},inplace=True)

#Merge with religion data
b=c.merge(df_all_religions,on='FIPS',how='outer')
a=b.merge(df_country,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)

a=a.loc[~a['value'].isnull()]
a=a.loc[a['year']=='overall']
#print (a.head(100))
'''

my_col_list=['FIPS','country_relig','desc', 'inc_grp','overall']

a=df_merged.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
#a.rename(columns={'overall':'value'},inplace=True)

my_inc_grp = a['inc_grp'].unique()

# Calc FD metric averages for each inc_grp
temp=[]
for curr_inc_grp in my_inc_grp: 
    for curr_col in col_order:
        df_a=df_tmp.loc[(df_tmp['inc_grp']==curr_inc_grp)&(df_tmp['desc']==curr_col)]
        df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
        df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
        temp.append({'inc_grp':curr_inc_grp,'desc':curr_col,'avg':df_a['overall'].mean()})
dfgroups = pd.DataFrame.from_dict(temp) 
print('Averages for each inc_grp:\n', dfgroups)

#Calculate country_relig averages within each inc_grp
temp=[]
for curr_inc_grp in my_inc_grp: 
    grp_cnt=a.loc[a['inc_grp']==curr_inc_grp]['FIPS'].nunique()
    for curr_relig in relig_list:
        relig_cnt=a.loc[(a['inc_grp']==curr_inc_grp)&(a['country_relig']==curr_relig)]['FIPS'].nunique()
        collst=a['desc'].unique()
        for curr_col in collst:
            df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==curr_relig) & (a['inc_grp']==curr_inc_grp)].copy(deep=True)
            temp.append({'grp_cnt':grp_cnt,'relig_cnt':relig_cnt,'country_relig':curr_relig,'inc_grp':curr_inc_grp,'desc':curr_col,'overall':df_a['overall'].mean()})
dftemp = pd.DataFrame.from_dict(temp) 
dftemp["country_relig"] = dftemp["country_relig"].astype('category')
dftemp["desc"] = dftemp["desc"].astype('category')

# Main df

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dftemp.pivot_table(index=['inc_grp','desc'], 
                                    columns=['country_relig'], 
                                    values=['overall'], sort=False),3)
#print(piv.index.to_list)

# Group & Relig count dfs
piv_relig_cnt=np.round(dftemp.pivot_table(index=['inc_grp'],columns=['country_relig'], values=['relig_cnt'],sort=False),3)
piv_grp_cnt=np.round(dftemp.pivot_table(index=['inc_grp'], values=['grp_cnt'],sort=False))

#sort rows in pivot_table
piv = piv.reindex(order, axis=0)
#piv_relig_cnt = piv_relig_cnt.reindex(order, axis=0)
#piv_grp_cnt = piv_grp_cnt.reindex(order, axis=0)


print ('\n',curr_inc_grp,'\n',piv)


print('piv_relig_cnt',piv_relig_cnt)
print('piv_grp_cnt',piv_grp_cnt)

# Group values df
for curr_inc_grp in my_inc_grp:
    dftemp = pd.DataFrame.from_dict(temp)
    dftemp = dftemp.loc[dftemp['inc_grp']==curr_inc_grp].copy(deep=True) 

dftemp = pd.DataFrame.from_dict(temp)
dftemp["desc"] = dftemp["desc"].astype('category')

inc_grp_vals=np.round(dftemp.groupby(['inc_grp','desc'])['overall'].mean(),3)
inc_grp_vals = inc_grp_vals.reindex(order, axis=0)

print('inc_grp_val:\n',inc_grp_vals)

piv.rename(columns={'CIAProtAll':'Prot', 'CIACathAll':'Cath', 'CIAOrthAll':'Orth', 'CIAChrAll':'Chr', 
            'CIAMusAll':'Mus', 'CIABudAll':'Bud', 'CIAHindAll':'Hind', 'CIABudHindAll':'BudHind',
            'CIAJewAll':'Jew', 'CIAOthNoJew':'Oth-NoJew', 'CIAOthWithJew':'Oth-Jew', 
            'CIABelieverAll':'Believers','CIANoRelAll':'No-Rel'},inplace=True)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_INC_GRP_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_INC_GRP_relig_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
piv_relig_cnt.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_INC_GRP_group_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
piv_grp_cnt.to_excel(joined_path)

print(inc_grp_vals)
mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_INC_GRP_group_vals.xlsx'
joined_path = os.path.join(mypath, myfile)
inc_grp_vals.to_excel(joined_path)

#compare_means(dftemp, 'other')

 

##Table INC_GRP using GNI per capita bins

In [ ]:
#########################
###   Create bins using ['GNI per capita'] column
#########################

my_col_list=['FIPS','country_relig','desc','overall','GNI per capita']

col_order = ['Financial Development Index', 
         'Financial Institutions Index', 'Financial Institutions Depth Index',
         'Financial Institutions Access Index', 'Financial Institutions Efficiency Index',
         'Financial Markets Index','Financial Markets Depth Index',
         'Financial Markets Access Index','Financial Markets Efficiency Index']

order =    [(        'lo_25',             'Financial Development Index'),
            (        'lo_25',            'Financial Institutions Index'),
            (        'lo_25',      'Financial Institutions Depth Index'),
            (        'lo_25',     'Financial Institutions Access Index'),
            (        'lo_25', 'Financial Institutions Efficiency Index'),
            (        'lo_25',                 'Financial Markets Index'),
            (        'lo_25',           'Financial Markets Depth Index'),
            (        'lo_25',          'Financial Markets Access Index'),
            (        'lo_25',      'Financial Markets Efficiency Index'),
            (        'lomed_25',             'Financial Development Index'),
            (        'lomed_25',            'Financial Institutions Index'),
            (        'lomed_25',      'Financial Institutions Depth Index'),
            (        'lomed_25',     'Financial Institutions Access Index'),
            (        'lomed_25', 'Financial Institutions Efficiency Index'),
            (        'lomed_25',                 'Financial Markets Index'),
            (        'lomed_25',           'Financial Markets Depth Index'),
            (        'lomed_25',          'Financial Markets Access Index'),
            (        'lomed_25',      'Financial Markets Efficiency Index'),
            (        'himed_25',             'Financial Development Index'),
            (        'himed_25',            'Financial Institutions Index'),
            (        'himed_25',      'Financial Institutions Depth Index'),
            (        'himed_25',     'Financial Institutions Access Index'),
            (        'himed_25', 'Financial Institutions Efficiency Index'),
            (        'himed_25',                 'Financial Markets Index'),
            (        'himed_25',           'Financial Markets Depth Index'),
            (        'himed_25',          'Financial Markets Access Index'),
            (        'himed_25',      'Financial Markets Efficiency Index'),
            (        'hi_25',             'Financial Development Index'),
            (        'hi_25',            'Financial Institutions Index'),
            (        'hi_25',      'Financial Institutions Depth Index'),
            (        'hi_25',     'Financial Institutions Access Index'),
            (        'hi_25', 'Financial Institutions Efficiency Index'),
            (        'hi_25',                 'Financial Markets Index'),
            (        'hi_25',           'Financial Markets Depth Index'),
            (        'hi_25',          'Financial Markets Access Index'),
            (        'hi_25',      'Financial Markets Efficiency Index')]

a=df_merged.loc[df_merged['desc'].isin(fd_list)]
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )

a.rename(columns={'GNI per capita':'GNIperCapita'},inplace=True)

#make bins

labels=['lo_25','lomed_25', 'himed_25', 'hi_25']

make_bins(df2split=a,
          split_into=4,
          val_col=['GNIperCapita'],
          col_name='GNIperCapita',
          labels=labels)

bins=['bin_GNIperCapita_jenkspy','bin_GNIperCapita_freq', 'bin_GNIperCapita_dist']

#counts
use_bin='bin_GNIperCapita_freq'

# Calc FD metric averages for each inc_grp
temp=[]
for curr_inc_grp in labels: 
    for curr_col in col_order:
        df_a=a.loc[(a['bin_GNIperCapita_freq']==curr_inc_grp)&(a['desc']==curr_col)]
        df_a['overall']=pd.to_numeric(df_a['overall'], errors ='coerce')
        df_a['overall'] = df_a['overall'].astype(float, errors = 'raise')
        temp.append({'bin_GNIperCapita_freq':curr_inc_grp,'desc':curr_col,'avg':df_a['overall'].mean()})
dfgroups = pd.DataFrame.from_dict(temp) 
print('Averages for each inc_grp:\n', dfgroups)

temp=[]
for curr_inc_grp in labels: 
    grp_cnt=a.loc[a['bin_GNIperCapita_freq']==curr_inc_grp]['FIPS'].nunique()
    for curr_relig in relig_list:
        relig_cnt=a.loc[(a['bin_GNIperCapita_freq']==curr_inc_grp)&(a['country_relig']==curr_relig)]['FIPS'].nunique()
        collst=a['desc'].unique()
        for curr_col in collst:
            df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==curr_relig) & (a['bin_GNIperCapita_freq']==curr_inc_grp)].copy(deep=True)
            temp.append({'grp_cnt':grp_cnt,'relig_cnt':relig_cnt,'country_relig':curr_relig,'GNI_inc_grp':curr_inc_grp,'desc':curr_col,'overall':df_a['overall'].mean()})
dftemp = pd.DataFrame.from_dict(temp) 
dftemp["country_relig"] = dftemp["country_relig"].astype('category')
dftemp["desc"] = dftemp["desc"].astype('category')

# Main df

#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dftemp.pivot_table(index=['GNI_inc_grp','desc'], 
                                    columns=['country_relig'], 
                                    values=['overall'], sort=False),3)


#sort rows in pivot_table
piv = piv.reindex(order, axis=0)
print ('\n',piv)

# Group & Relig count dfs
piv_relig_cnt=np.round(dftemp.pivot_table(index=['GNI_inc_grp'],columns=['country_relig'], values=['relig_cnt'],sort=False),3)
piv_grp_cnt=np.round(dftemp.pivot_table(index=['GNI_inc_grp'], values=['grp_cnt'],sort=False))

print('piv_relig_cnt',piv_relig_cnt)
print('piv_grp_cnt',piv_grp_cnt)

# Group values df
for curr_inc_grp in labels:
    dftemp = pd.DataFrame.from_dict(temp)
    dftemp = dftemp.loc[dftemp['GNI_inc_grp']==curr_inc_grp].copy(deep=True) 

dftemp = pd.DataFrame.from_dict(temp)
dftemp["desc"] = dftemp["desc"].astype('category')

inc_grp_vals=np.round(dftemp.groupby(['GNI_inc_grp','desc'])['overall'].mean(),3)

inc_grp_vals = inc_grp_vals.reindex(order, axis=0)

print('inc_grp_val:\n',inc_grp_vals)

piv.rename(columns={'CIAProtAll':'Prot', 'CIACathAll':'Cath', 'CIAOrthAll':'Orth', 'CIAChrAll':'Chr', 
            'CIAMusAll':'Mus', 'CIABudAll':'Bud', 'CIAHindAll':'Hind', 'CIABudHindAll':'BudHind',
            'CIAJewAll':'Jew', 'CIAOthNoJew':'Oth-NoJew', 'CIAOthWithJew':'Oth-Jew', 
            'CIABelieverAll':'Believers','CIANoRelAll':'No-Rel'},inplace=True)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GNI_INC_GRP_main.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GNI_INC_GRP_relig_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
piv_relig_cnt.to_excel(joined_path)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GNI_INC_GRP_group_cnts.xlsx'
joined_path = os.path.join(mypath, myfile)
piv_grp_cnt.to_excel(joined_path)

print(inc_grp_vals)
mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables/' 
myfile = 'nTable_GNI_INC_GRP_group_vals.xlsx'
joined_path = os.path.join(mypath, myfile)
inc_grp_vals.to_excel(joined_path)


##Table LAW_means: Religion, law origin, finance

In [ ]:
#########
#  Ihsan needs to figure-out what to do with multiple-assigned Rule of law
#######
#adjust column order
order =    [('value',             'Financial Development Index'),
            ('value',            'Financial Institutions Index'),
            ('value',      'Financial Institutions Depth Index'),
            ('value',     'Financial Institutions Access Index'),
            ('value', 'Financial Institutions Efficiency Index'),
            ('value',                 'Financial Markets Index'),
            ('value',           'Financial Markets Depth Index'),
            ('value',          'Financial Markets Access Index'),
            ('value',      'Financial Markets Efficiency Index')]

order_col= [('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll'),
            ('value',    'CIAProtAll'),
            ('value',    'CIACathAll'),
            ('value',    'CIAOrthAll'),
            ('value',     'CIAChrAll'),
            ('value',     'CIAMusAll'),
            ('value',     'CIABudAll'),
            ('value',    'CIAHindAll'),
            ('value', 'CIABudHindAll'),
            ('value',     'CIAJewAll'),
            ('value',   'CIAOthNoJew'),
            ('value', 'CIAOthWithJew'),
            ('value',   'CIABeliever'),  
            ('value',   'CIANoRelAll')
        ]       

a=df_merged.copy(deep=True)

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=df_merged.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
a.rename(columns={'overall':'value'},inplace=True)

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

#calc group counts/vals
temp=[]
for curr_rule_law in my_rule_law_lst:
    tt=a.loc[a[curr_rule_law]==1]
    for curr_col in fd_list:
        ttt=tt.loc[tt['desc']==curr_col]
        grp_val=ttt['value'].mean()
        temp.append({'grp':curr_rule_law, 'col':curr_col, 'value':grp_val})
dfgrptemp = pd.DataFrame.from_dict(temp) 
grppiv=np.round(dfgrptemp.pivot_table(index=['grp'], columns=['col'], values=['value'], sort=False),3)
grppiv = grppiv.reindex(order, axis=1)

rel_lst = a['country_relig'].unique()
temp=[]
for curr_rule_law in my_rule_law_lst:
    tt=a.loc[a[curr_rule_law]==1]
    for curr_col in rel_lst:
        ttt=tt.loc[tt['country_relig']==curr_col]
        grp_cnt=ttt['FIPS'].nunique()
        temp.append({'grp':curr_rule_law, 'col':curr_col, 'value':grp_cnt})
dfgrpcnt = pd.DataFrame.from_dict(temp) 
grp_cnt_piv=np.round(dfgrpcnt.pivot_table(index=['grp'], columns=['col'], values=['value'], sort=False),3)
grp_cnt_piv = grp_cnt_piv.reindex(order_col, axis=1)
print ('MY COUNTS:\n',grp_cnt_piv)

#calc cnt/val by relig
temp=[]
for curr_rule_law in my_rule_law_lst:
    for curr_relig in rel_lst:
        tt=a.loc[(a[curr_rule_law]==1)&(a['country_relig']==curr_relig)]
        relig_cnt=tt['FIPS'].nunique() 
        for curr_col in fd_list:
            df_a=tt.loc[tt['desc']==curr_col].copy(deep=True)
            temp.append({'relig_cnt':relig_cnt,'country_relig':curr_relig,'rule_of_law':curr_rule_law,'desc':curr_col,'value':df_a['value'].mean()})
dftemp = pd.DataFrame.from_dict(temp) 

# Main df
#sort=False keeps the order of the columns=['country_relig'] in pivot_table
piv=np.round(dftemp.pivot_table(index=['rule_of_law','country_relig'], columns=['desc'], values=['value'], sort=False),3)

piv2=np.round(dftemp.pivot_table(index=['rule_of_law','country_relig'], values=['relig_cnt'], sort=False),3)

print(piv2)

#view existing row order (index)
#print(piv.index.to_list)

#view existing col order
#print(piv.columns.to_list)

#sort cols in pivot_table
piv = piv.reindex(order, axis=1)
grppiv = grppiv.reindex(order, axis=1)

print ('\n main table:',piv)
print ('\n grp_vals',grppiv)



piv.rename(columns={'CIAProtAll':'Prot', 'CIACathAll':'Cath', 'CIAOrthAll':'Orth', 'CIAChrAll':'Chr', 
            'CIAMusAll':'Mus', 'CIABudAll':'Bud', 'CIAHindAll':'Hind', 'CIABudHindAll':'BudHind',
            'CIAJewAll':'Jew', 'CIAOthNoJew':'Oth-NoJew', 'CIAOthWithJew':'Oth-Jew', 
            'CIABelieverAll':'Believers','CIANoRelAll':'No-Rel'},inplace=True)

for curr_rule_law in my_rule_law_lst:
    dftemp = pd.DataFrame.from_dict(temp)
    dftemp = dftemp.loc[dftemp['rule_of_law']==curr_rule_law].copy(deep=True) 
    dftemp["country_relig"] = dftemp["country_relig"].astype('category')

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_LAW_main_means.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

myfile = 'nTable_LAW_grp_means.xlsx'
joined_path = os.path.join(mypath, myfile)
grppiv.to_excel(joined_path)

myfile = 'nTable_LAW_grp_count.xlsx'
joined_path = os.path.join(mypath, myfile)
grp_cnt_piv.to_excel(joined_path)


#compare_means(dftemp, 'other')

 

#LAW regression options:

In [ ]:
'''
########################################
#
#  https://stackoverflow.com/questions/50733014/linear-regression-with-dummy-categorical-variables
#
#  MUST CONSIDER categorical variables and multicollinearity
#
########################################
'''

In [ ]:
a=df_merged.copy(deep=True)

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=df_merged.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
a.rename(columns={'overall':'value'},inplace=True)

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']
collst=a['desc'].unique()
fips_list=a['FIPS'].unique()
temp=[]
'''for curr_rule_law in my_rule_law_lst:
    for curr_relig in relig_list:
        for curr_col in collst:
            for curr_fips in fips_list:
                df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==curr_relig) & (a[curr_rule_law]==1) & (a['FIPS']==curr_fips)].copy(deep=True)
                if df_a['FIPS'].isnull:
                    pass
                else:
                    temp.append({'FIPS':curr_fips,'country_relig':curr_relig,'rule_of_law':curr_rule_law,'desc':curr_col,'value':df_a['value']})
dftemp = pd.DataFrame.from_dict(temp) 
'''
#a_melt=pd.melt(a,id_vars=['FIPS','country_relig','desc','value'], var_name=['rule_law'], value_vars=['IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil'],)
a_piv=np.round(a.pivot_table(index=['FIPS','IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil'],columns=['desc'],values=['value']),3)
#a_piv.reset_index(inplace=True)
print (a_piv.head(20))

# Main df
#sort=False keeps the order of the columns=['country_relig'] in pivot_table
#piv=np.round(dftemp.pivot_table(index=['FIPS','rule_of_law','country_relig'], columns=['desc'], values=['value'], sort=False),3)


##LAW assumption testing: 

In [ ]:
# Build linear regression model 
# To test assumptions, we must run the rgression, then inspect the predicted vs. actual results 
# Split data into predictors X and output Y
# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

a=df_merged.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
a.rename(columns={'overall':'value'},inplace=True)

temp=[]
for curr_rule_law in my_rule_law_lst:
    df_a=a.copy(deep=True)
    df_a=df_a.fillna(0)
    df_a.reset_index(drop=False,inplace=True)
    Y = df_a[curr_rule_law]
    X = df_a['value']
    X = sm.add_constant(X)  

    assumption_check = linear_regression_assumptions(X, Y)



##LAW: sklean LinearRegression

In [ ]:
# Build linear regression model 

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

a=df_merged.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
a.rename(columns={'overall':'value'},inplace=True)

temp=[]
for curr_rule_law in my_rule_law_lst:
    df_a=a.copy(deep=True)
    df_a=df_a.fillna(0)
    df_a.reset_index(drop=False,inplace=True)
    Y = df_a[curr_rule_law]
    X = df_a['value']
    X = sm.add_constant(X)  

    lm = LinearRegression()
    result = lm.fit(X, Y)

    print(f'\n{curr_rule_law}')
    print(f'intercept = {result.intercept_}\nBeta vals:')
    cnt=1
    for pred in predictors:
        try:
            print(f'{pred}: {result.coef_[cnt]} ')    
        except:
            print('no coef for cnt = ',cnt, pred)
        cnt+=1

    #temp.append({'country_relig':curr_relig,'rule_of_law':curr_rule_law,'desc':curr_col,'value':df_a['value'].mean()})

#dftemp = pd.DataFrame.from_dict(temp) 


##LAW: Tobit regression (without providing artificial censorship values)

In [ ]:


# Build linear regression model 
# To test assumptions, we must run the rgression, then inspect the predicted vs. actual results 
# Split data into predictors X and output Y

predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

a=df_merged.copy(deep=True)

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'za_IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=a.loc[df_merged['desc'].isin(fd_list)]

#reduce columns of dataframe
a.drop(a.columns.difference(my_col_list),axis=1, inplace=True )
a.rename(columns={'overall':'value'},inplace=True)

ns=df_a.size
cens = pd.Series(np.zeros((ns,))) 

df_a=a.copy(deep=True)
df_a=df_a.fillna(0)
df_a.reset_index(drop=False,inplace=True)

temp=[]
for curr_rule_law in my_rule_law_lst:
    Y = df_a[curr_rule_law]
    X = df_a['value']
    X = sm.add_constant(X)  

    tr = TobitModel()
    result = tr.fit(X, Y,cens, verbose=False)
    print(f'\n{curr_rule_law}')
    print(f'sigma is the std dist b/t obs and the regression line ~~ 95% obs will [-2*sigma, +2*sigma]:\nsigma = {result.sigma_}\nintercept = {result.intercept_}\nBeta vals:')
    cnt=1
    for pred in predictors:
        try:
            print(f'{pred}: {result.coef_[cnt]} ')    
        except:
            print('no coef for:',cnt, pred)
        cnt+=1
    score = tr.score(X, Y, scoring_function=explained_variance_score) #r2_score is providing a huge value????
    print ('Model R2:', score)
'''    lm = LinearRegression()
    model = lm.fit(X, Y)
    print(f'\n{curr_rule_law}: alpha = {model.intercept_}')
    print(f'betas = {model.coef_}')
    for pred in predictors:
      print(pred)
''' 
    #temp.append({'country_relig':curr_relig,'rule_of_law':curr_rule_law,'desc':curr_col,'value':df_a['value'].mean()})




#Corr Matrix & Descriptive Statistics

In [ ]:
df_tmp=df_merged.copy(deep=True)
df_tmp.reset_index(inplace=True)
df_tmp.rename(columns={'overall':'value'}, inplace=True)

# Calculate Institutions composite (zc_wgi_avg)
avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

#limit the values in the desc column to calc wgi mean-values 
df_t1=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t1=df_t1.groupby(['FIPS','country_relig'],as_index=False)['value'].mean()
df_t1.rename(columns={'overall':'zc_wgi_avg'},inplace=True)
df_t1.reset_index(inplace=True)
#print(df_t1.head())

#df_t2=[] = df_tmp.copy(deep=True)
df_t2=df_tmp.loc[df_tmp['desc']=='openness'].copy(deep=True)
df_t2=df_t2[['FIPS','desc','value']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)
df_t2=df_t2.pivot(index='FIPS', columns='desc',values='value')
#df_t2.rename(columns={'openness':'zc_openness'},inplace=True)
df_t2.reset_index(inplace=True)


print(df_t2.head())

'''

df_tmp['FD']=df_tmp.loc[df_tmp['desc']=='Financial Development Index']['overall']
df_tmp['FI']=df_tmp.loc[df_tmp['desc']=='Financial Institutions Index']['overall']
df_tmp['FM']=df_tmp.loc[df_tmp['desc']=='Financial Markets Index']['overall']
df_tmp['FID']=df_tmp.loc[df_tmp['desc']=='Financial Institutions Depth Index']['overall']
df_tmp['FIA']=df_tmp.loc[df_tmp['desc']=='Financial Institutions Access Index']['overall']
df_tmp['FIE']=df_tmp.loc[df_tmp['desc']=='Financial Institutions Efficiency Index']['overall']
df_tmp['FMD']=df_tmp.loc[df_tmp['desc']=='Financial Markets Depth Index']['overall']
df_tmp['FMA']=df_tmp.loc[df_tmp['desc']=='Financial Markets Access Index']['overall']
df_tmp['FME']=df_tmp.loc[df_tmp['desc']=='Financial Markets Efficiency Index']['overall']


df_dan=df_tmp.loc[df_tmp['desc']=='Financial Development Index']

df_dan['diff'] = np.where((df_dan['FD'] == df_dan['overall']),1,np.nan)
print('\nmy unique\n',df_dan['diff'].unique())
'''
df_tmp.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_tmp.loc[df_tmp['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

#print(df_sfi.head())

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

df_a=df_tmp
#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
#print('\ndf_a_1\n',df_a.head())
df_a=df_a.merge(df_t2, on='FIPS', how='outer')
#print('\ndf_a_2\n',df_a.head())
df_a=df_a.merge(df_t1, on='FIPS', how='outer')
#print('\ndf_a_3\n',df_a.head())

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
#df_a['openness']=np.log(df_a['openness'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)
df_test=df_a.copy(deep=True)

df_a = pd.get_dummies(df_a, columns=['continent'])
df_a.rename(columns={'continent_Africa':'zc_Africa',
             'continent_Asia':'zc_Asia', 
             'continent_North America':'zc_N_Amer', 
             'continent_Oceania':'zc_Oceania', 
             'continent_South America':'zc_S_Amer'},inplace=True)
df_a = pd.get_dummies(df_a, columns=['region'])
#print(df_a.columns.to_list())

df_a.rename(columns={
    'region_Northern Europe':'zc_N_Europe',  
    'region_Southern Asia':'zc_S_Asia', 
    'region_Southern Europe':'zc_S_Europe', 
    'region_Western Africa':'zc_W_Africa', 
    'region_Western Asia':'zc_W_Asia', 
    'region_Australia and New Zealand':'zc_Australia_NZ', 
    'region_Caribbean':'zc_Caribbean', 
    'region_Central America':'zc_C_Amer', 
    'region_Central Asia':'zc_C_Asia', 
    'region_Eastern Africa':'zc_E_Africa', 
    'region_Eastern Asia':'zc_E_Asia', 
    'region_Eastern Europe':'zc_E_Europe', 
    'region_Melanesia':'zc_Melanesia', 
    'region_Micronesia':'zc_Micronesia', 
    'region_Middle Africa':'zc_M_Africa', 
    'region_Northern Africa':'zc_N_Africa', 
    'region_Northern America':'zc_N_Amer', 
    'region_Polynesia':'zc_Polynesia', 
    'region_South America':'zc_S_Amer', 
    'region_South-eastern Asia':'zc_SE_Asia', 
    'region_Southern Africa':'zc_S_Africa'},inplace=True)

####################### 
# Corr Matrix
#
df_test=df_a[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
              'za_IsikFrench','zb_BeckSettlerMortality','zd_sfi','ze_Population (millions)',
              'CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 'CIAOthWithJew%',
              'openness', 'AbsLatitude',
              'FD','FI','FIA','FID','FIE','FM','FMA','FMD',
              'zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ',
              'zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 
              'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa',
              'zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 
              'zc_S_Africa', 'zc_Africa','zc_Asia', 'zc_Oceania']]
#print(df_test.columns.to_list())
#df_test=df_a[['FD','FI','FIA','FID','FIE','FM','FMA','FMD']]
corr_matrix=df_test.rcorr()
print(df_test.corr())

#corr_matrix=pg.pairwise_corr(df_test)
print (corr_matrix)
print(df_test.corr())

myfile = 'Corr_Matrix.xlsx'
joined_path = os.path.join(mypath, myfile)
corr_matrix.to_excel(joined_path)

####################### 
# Descriptive Statistics
#
df_test=df_a[['za_IsikFrench','zb_BeckSettlerMortality','zd_sfi','ze_Population (millions)',
              'CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 'CIAOthWithJew%',
              'AbsLatitude',
              'FD','FI','FIA','FID','FIE','FM','FMA','FMD']]

my_res=univariate_metrics(df_test)
my_descript=my_res.pivot(index=['desc'],columns=['Col_Name'],values=['value'])

print(my_res)

myfile = 'Descriptive_stats.xlsx'
joined_path = os.path.join(mypath, myfile)
my_descript.to_excel(joined_path)

####################### 
#TEST ASSUMPTIONS

'''for curr_dv in predictors:
    df_loop=df_a.loc[df_a['desc']==curr_dv].copy(deep=True)
    Y = df_loop['value']
    X = df_a[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew']]
    X = sm.add_constant(X)  
    LinearRegression.fit(X=X,y=Y)
    assumption_check = linear_regression_assumptions(X, Y)
'''




#Regression Tables

##OLD_NOT_USED  Regression Table 2.1

In [ ]:
'''df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Markets Index']


# Calculate Institutions composite (wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'zc_wgi_avg'},inplace=True)


##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's

iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)'],
         ['zc_wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_wgi_avg','zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality','zc_wgi_avg','zd_sfi','ze_Population (millions)'],
        ]

temp=[]
for curr_fd in predictors:
    # Only look at current FD value
    df_loop=df_a.loc[df_a['desc']==curr_fd].copy(deep=True)
    # define DV
    Y = df_loop['value']  
    print ('\nnbr of countries in DV:',df_loop['FIPS'].nunique())

    for curr_iv in iv_list:
        X = df_loop[curr_iv]
        X = sm.add_constant(X)  
        pglm = pg.linear_regression(X, Y)
        pglm=pglm[['names', 'coef','pval','r2']]
        pglm=pglm.loc[pglm['names']!='Intercept']
        
        #assign pval designator
        select_conditions = [
            (pglm['pval'] <=.01),
            ((pglm['pval'] <=.05)&(pglm['pval'] >.01)),
            ((pglm['pval'] <=.1)&(pglm['pval'] >.05)),
            (pglm['pval'] >.1)
            ]
        select_values = ['a', 'b', 'c','']
        pglm['my_level'] = np.select(select_conditions, select_values)

        pglm.drop(columns=['pval'],inplace=True)

        #transpose data for output table
        pglm=pglm.transpose()
        temp.append(pglm.round(3))

df_table_2_1 = pd.concat(temp)
print('df_table_2_1',df_table_2_1)
print(X.rcorr())

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_1.xlsx'
joined_path = os.path.join(mypath, myfile)
df_table_2_1.to_excel(joined_path)

'''

'''
for curr_fd in fd_list:
    df_loop=df_a.loc[df_a['desc']==curr_fd].copy(deep=True)
    Y = df_loop['value']
    X = df_loop['IsikCivil']
    X = sm.add_constant(X)  

    pglm = pg.linear_regression(X, Y)
    print(f'\n{curr_fd}')
    print(pglm.round(2))
'''
######################################
##### response variable = Financial index; --reset multi-valued index
######################################
'''df_a=a.copy(deep=True)
df_a=df_a.fillna(0)
df_a.reset_index(drop=False,inplace=True)
df_a.rename(columns={'overall':'value'},inplace=True)
df_a_piv=np.round(df_a.pivot_table(index=['FIPS','country_relig'],columns=['desc'],values=['value']),3)
df_a_piv.columns = df_a_piv.columns.droplevel(0)
df_a=df_a_piv.merge(a,on='FIPS',how='outer')
df_a = pd.get_dummies(df_a, columns=['country_relig'])
print(df_a.columns.to_list())
'''



##Table 2.1 Relig dummies

In [ ]:
df_tmp=df_merged.copy(deep=True)
df_tmp.reset_index(inplace=True)
df_tmp.rename(columns={'overall':'value'},inplace=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']


# Calculate Institutions composite (zc_wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

'''
#limit the values in the desc column to calc wgi mean-values 
df_t1=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)
df_t1=df_t1.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t1.rename(columns={'overall':'zc_wgi_avg'},inplace=True)
df_t1.reset_index(inplace=True)
#print(df_t1.head())
df_a=df_a.merge(df_t1, on='FIPS', how='outer')

'''
##########new comment remove fd from filter
##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')|(df_tmp['desc']=='openness')]


# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_tmp[['FIPS','desc','value']]
df_sfi=df_sfi.loc[df_sfi['desc']=='sfi']
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')
df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)
df_a=df_tmp.merge(df_sfi, on='FIPS', how='outer')
#print('\ndf_a_3\n',df_a.head())

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

df_a = pd.get_dummies(df_a, columns=['continent'])
df_a.rename(columns={'continent_Africa':'zc_Africa_cont',
             'continent_Asia':'zc_Asia_cont', 
             'continent_North America':'zc_N_Amer_cont', 
             'continent_Oceania':'zc_Oceania_cont', 
             'continent_South America':'zc_S_Amer_cont'},inplace=True)
df_a = pd.get_dummies(df_a, columns=['region'])
print(df_a.head(50))

df_a.rename(columns={
    'region_Northern Europe':'zc_N_Europe',  
    'region_Southern Asia':'zc_S_Asia', 
    'region_Southern Europe':'zc_S_Europe', 
    'region_Western Africa':'zc_W_Africa', 
    'region_Western Asia':'zc_W_Asia', 
    'region_Australia and New Zealand':'zc_Australia_NZ', 
    'region_Caribbean':'zc_Caribbean', 
    'region_Central America':'zc_C_Amer', 
    'region_Central Asia':'zc_C_Asia', 
    'region_Eastern Africa':'zc_E_Africa', 
    'region_Eastern Asia':'zc_E_Asia', 
    'region_Eastern Europe':'zc_E_Europe', 
    'region_Melanesia':'zc_Melanesia', 
    'region_Micronesia':'zc_Micronesia', 
    'region_Middle Africa':'zc_M_Africa', 
    'region_Northern Africa':'zc_N_Africa', 
    'region_Northern America':'zc_N_Amer', 
    'region_Polynesia':'zc_Polynesia', 
    'region_South America':'zc_S_Amer', 
    'region_South-eastern Asia':'zc_SE_Asia', 
    'region_Southern Africa':'zc_S_Africa'},inplace=True)

#create list of lists for regression IV's

iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)']
        ]
print('DF_A\n',df_a.head(),'\nPREDICTORS\n',predictors,'\nIV_LIST\n',iv_list)
piv = regress_table(df_a, predictors, iv_list)

print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_1.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)


####################### 
# Descriptive Statistics
#
df_test=df_a[['FD','FI','FM','aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
              'za_IsikFrench','zb_BeckSettlerMortality','zd_sfi','ze_Population (millions)',
              ]]
#print(df_test.columns.to_list())

corr_matrix=df_test.rcorr()
print (corr_matrix)

myfile = 'nCorr_Matrix.xlsx'
joined_path = os.path.join(mypath, myfile)
corr_matrix.to_excel(joined_path)

my_res=univariate_metrics(df_test)
my_descript=my_res.pivot(index=['desc'],columns=['Col_Name'],values=['value'])

print(my_descript)

myfile = 'nTable_2_1_descriptive_stats.xlsx'
joined_path = os.path.join(mypath, myfile)
my_descript.to_excel(joined_path)

####################### 
#TEST ASSUMPTIONS

'''for curr_dv in predictors:
    df_loop=df_a.loc[df_a['desc']==curr_dv].copy(deep=True)
    Y = df_loop['value']
    X = df_a[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew']]
    X = sm.add_constant(X)  
    LinearRegression.fit(X=X,y=Y)
    assumption_check = linear_regression_assumptions(X, Y)
'''




##Regression Table 2.2: relig %

In [ ]:
df_tmp=df_merged.copy(deep=True)
df_tmp.reset_index(inplace=True)

print(df_tmp.head())
# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']


# Calculate Institutions composite (zc_wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

#limit the values in the desc column to calc wgi mean-values 
df_t1=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t1=df_t1.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t1.rename(columns={'overall':'zc_wgi_avg'},inplace=True)
df_t1.reset_index(inplace=True)
#print(df_t1.head())

##########new comment remove fd from filter
##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')|(df_tmp['desc']=='openness')]

df_sfi=df_tmp.loc[df_tmp['desc']=='sfi']
df_sfi.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

#print(df_sfi.head())

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_tmp.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t1, on='FIPS', how='outer')
#print('\ndf_a_3\n',df_a.head())

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)
df_test=df_a.copy(deep=True)

df_a = pd.get_dummies(df_a, columns=['continent'])
df_a.rename(columns={'continent_Africa':'zc_Africa',
             'continent_Asia':'zc_Asia', 
             'continent_North America':'zc_N_Amer', 
             'continent_Oceania':'zc_Oceania', 
             'continent_South America':'zc_S_Amer'},inplace=True)
df_a = pd.get_dummies(df_a, columns=['region'])
#print(df_a.columns.to_list())

df_a.rename(columns={
    'region_Northern Europe':'zc_N_Europe',  
    'region_Southern Asia':'zc_S_Asia', 
    'region_Southern Europe':'zc_S_Europe', 
    'region_Western Africa':'zc_W_Africa', 
    'region_Western Asia':'zc_W_Asia', 
    'region_Australia and New Zealand':'zc_Australia_NZ', 
    'region_Caribbean':'zc_Caribbean', 
    'region_Central America':'zc_C_Amer', 
    'region_Central Asia':'zc_C_Asia', 
    'region_Eastern Africa':'zc_E_Africa', 
    'region_Eastern Asia':'zc_E_Asia', 
    'region_Eastern Europe':'zc_E_Europe', 
    'region_Melanesia':'zc_Melanesia', 
    'region_Micronesia':'zc_Micronesia', 
    'region_Middle Africa':'zc_M_Africa', 
    'region_Northern Africa':'zc_N_Africa', 
    'region_Northern America':'zc_N_Amer', 
    'region_Polynesia':'zc_Polynesia', 
    'region_South America':'zc_S_Amer', 
    'region_South-eastern Asia':'zc_SE_Asia', 
    'region_Southern Africa':'zc_S_Africa'},inplace=True)

#create list of lists for regression IV's
iv_list=[['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)'],
         ['zc_wgi_avg'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'zc_wgi_avg'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'zc_wgi_avg','zd_sfi','ze_Population (millions)'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality','zc_wgi_avg',
          'zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_2_relig_percent.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

####################### 
# Descriptive Statistics
#
df_test=df_a[['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality','zc_wgi_avg',
          'zd_sfi','ze_Population (millions)']]
#print(df_test.columns.to_list())
my_res=univariate_metrics(df_test)
my_descript=my_res.pivot(index=['desc'],columns=['Col_Name'],values=['value'])

print(my_descript)

myfile = 'nTable_2_2_descriptive_stats.xlsx'
joined_path = os.path.join(mypath, myfile)
my_descript.to_excel(joined_path)




##Table 2.2 (religiosity interaction term) --NOT USED

In [ ]:
'''# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Markets Index']
'''

'''
df_tmp=df_religiosity.copy(deep=True)

df_relig_intens=df_tmp.groupby(['country_relig'],as_index=False)['religiosity'].mean()
df_relig_intens=df_relig_intens.transpose()

print (df_relig_intens)
'''
'''df_tmp=df_merged.copy(deep=True)
'''
''' wgi_avg...
# Generate data for dfs
avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'wgi_avg'},inplace=True)
print('\n\ndf_t2-wgi_avg',df_t2['wgi_avg'].mean())
'''
'''##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

#print (df_a.head())

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
ddf_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi & wgi_avg columns with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
'''
'''
print('\n######',df_a.loc[df_a['desc']=='sfi']['value'].mean())
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

print('######',df_a['wgi_avg'].mean())
'''
#natural log transformations
'''df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)
'''
'''#interaction terms (religiosity * religious dummy)
df_a['ProtIntens']=df_a['aCIAProtAll']*df_a['religiosity']
df_a['OrthIntens']=df_a['cCIAOrthAll']*df_a['religiosity']
df_a['MusIntens']=df_a['eCIAMusAll']*df_a['religiosity']
df_a['BudHindIntens']=df_a['hCIABudHindAll']*df_a['religiosity']
df_a['NoRelIntens']=df_a['mCIANoRelAll']*df_a['religiosity']



#create list of lists for regression IV's
iv_list=[['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%','ProtIntens','OrthIntens','MusIntens','BudHindIntens','NoRelIntens'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%', 'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality'],
         ['CIAProt%', 'CIAOrth%', 'CIAMus%', 'CIABudHind%',  'CIANoRel%', 
          'CIAOthWithJew%', 'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)']
        ]

temp=[]
for curr_fd in predictors:
    # Only look at current FD value
    df_loop=df_a.loc[df_a['desc']==curr_fd].copy(deep=True)
    # define DV
    Y = df_loop['value']  
    print ('\nnbr of countries in DV:',df_loop['FIPS'].nunique())

    for curr_iv in iv_list:
        X = df_loop[curr_iv]
        X = sm.add_constant(X)  
        pglm = pg.linear_regression(X, Y)
        pglm=pglm[['names', 'coef','pval','r2']]
        pglm=pglm.loc[pglm['names']!='Intercept']
        
        #assign pval designator
        select_conditions = [
            (pglm['pval'] <=.01),
            ((pglm['pval'] <=.05)&(pglm['pval'] >.01)),
            ((pglm['pval'] <=.1)&(pglm['pval'] >.05)),
            (pglm['pval'] >.1)
            ]
        select_values = ['a', 'b', 'c','']
        pglm['my_level'] = np.select(select_conditions, select_values)

        pglm.drop(columns=['pval'],inplace=True)

        #transpose data for output table
        pglm=pglm.transpose()
        temp.append(pglm.round(3))

df_table_2_2_interaction = pd.concat(temp)
print('df_table_2_2_interaction',df_table_2_2_interaction)
print(X.rcorr())

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_2_interaction.xlsx'
joined_path = os.path.join(mypath, myfile)
df_table_2_2_interaction.to_excel(joined_path)
'''


######################################
##### response variable = Financial index; --reset multi-valued index
######################################
'''df_a=a.copy(deep=True)
df_a=df_a.fillna(0)
df_a.reset_index(drop=False,inplace=True)
df_a.rename(columns={'overall':'value'},inplace=True)
df_a_piv=np.round(df_a.pivot_table(index=['FIPS','country_relig'],columns=['desc'],values=['value']),3)
df_a_piv.columns = df_a_piv.columns.droplevel(0)
df_a=df_a_piv.merge(a,on='FIPS',how='outer')
df_a = pd.get_dummies(df_a, columns=['country_relig'])
print(df_a.columns.to_list())
'''



##Regression Table 2.3 --data from 1900

In [ ]:
# 'Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900', 'Prot1900%', 'Orth1900%', 'Mus1900%', 'BudHind1900%', 'NoRel1900%', 'OthWithJew1900%', 

df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']

# Generate data for dfs
avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']

#'Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900', 'Prot1900%', 'Orth1900%', 'Mus1900%', 'BudHind1900%', 'NoRel1900%', 'OthWithJew1900%', 

#assign 1900_country_relig designator
select_conditions = [
(df_tmp['Prot1900'] == 1),
(df_tmp['Orth1900'] == 1),
(df_tmp['Mus1900'] == 1),
(df_tmp['BudHind1900'] == 1),
(df_tmp['NoRel1900'] == 1),
(df_tmp['OthWithJew1900'] == 1)
]
select_values = ['Prot', 'Ortho', 'Mus','BudHind', 'NoRel', 'Oth+Jew']
df_tmp['1900_country_relig'] = np.select(select_conditions, select_values)

#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

################
# recalculate wgi_avg based on the religion of 1900? 

df_t2=df_t2.groupby(['FIPS','1900_country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'wgi_avg'},inplace=True)

###df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi & wgi_avg columns with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's

iv_list=[['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'zb_BeckSettlerMortality'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)'],
         ['wgi_avg'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'wgi_avg'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'wgi_avg','zd_sfi','ze_Population (millions)'],
         ['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench','zb_BeckSettlerMortality','wgi_avg','zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)
mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_3_1900.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

####################### 
# Descriptive Statistics
#
df_test=df_a[['Prot1900', 'Orth1900', 'Mus1900', 'BudHind1900', 'NoRel1900', 'OthWithJew1900',
          'za_IsikFrench','zb_BeckSettlerMortality','wgi_avg','zd_sfi','ze_Population (millions)']]
#print(df_test.columns.to_list())
my_res=univariate_metrics(df_test)
my_descript=my_res.pivot(index=['desc'],columns=['Col_Name'],values=['value'])

print(my_descript)

myfile = 'nTable_2_3_descriptive_stats.xlsx'
joined_path = os.path.join(mypath, myfile)
my_descript.to_excel(joined_path)




##Regress Table 2.4 High religiosity & 2.5, Low religiosity using NoRel% median

In [ ]:
#df_NoRel_avg
df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']

##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')
df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')

#natural log transformations
try:
    df_a.rename(columns={'Population (millions)':'ze_Population (millions)'},inplace=True)
except: 
    pass
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's
iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)']
        ]
#to create table for each quartile... 
#my_vals = ['lo_25', 'himed_25', 'hi_25', 'lomed_25']
#for curr_quartile in my_vals:
#    my_FIPS=df_NoRel_avg.loc[df_NoRel_avg['bin_CIANoRel_per_freq']==curr_quartile]['FIPS'].unique()

##############
#table 2.4 is high religiosity which is low NoRel%

#Only consider NoRel% greater than zero
#df_NoRel_avg=df_NoRel_avg.loc[df_NoRel_avg['CIANoRel_per']>0]
#
my_FIPS=df_NoRel_avg.loc[(df_NoRel_avg['CIANoRel_per']<df_NoRel_avg['CIANoRel_per'].median())]['FIPS']
df_tmp0=df_a.loc[df_a['FIPS'].isin(my_FIPS)].copy(deep=True)
print('\nmy_FIPS\n',df_tmp0['FIPS'].unique())
piv = regress_table(df_tmp0, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_4_High_Religiosity.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)

###################
#Table 2.5 Low religiosity 
#
my_FIPS=df_NoRel_avg.loc[df_NoRel_avg['CIANoRel_per']>=df_NoRel_avg['CIANoRel_per'].median()]['FIPS']
df_tmp0=df_a.loc[df_a['FIPS'].isin(my_FIPS)].copy(deep=True)
print('\nmy_FIPS\n',df_tmp0['FIPS'].unique())
piv = regress_table(df_tmp0, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_5_Low_Religiosity.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)




##Regression Table 2.4 --high_religiosity only (EVS/WVS)

In [ ]:
df_tmp=df_merged.copy(deep=True)

#only include high religiosity

my_FIPS=df_religiosity.loc[df_religiosity['religiosity']>=df_religiosity['religiosity'].median()]['FIPS'].unique()

df_tmp=df_tmp.loc[df_tmp['FIPS'].isin(my_FIPS)]

print(df_tmp['FIPS'].unique())

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']


# Calculate Institutions composite (wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'wgi_avg'},inplace=True)


##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's

iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)'],
         ['wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'wgi_avg','zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality','wgi_avg','zd_sfi','ze_Population (millions)'],
        ]
piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_4_hi_wvs.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)



##Regression Table 2.5 --low religiosity data (EVS/WVS)

In [ ]:
df_tmp=df_merged.copy(deep=True)

#only include low religiosity

my_FIPS=df_religiosity.loc[df_religiosity['religiosity']<df_religiosity['religiosity'].median()]['FIPS'].unique()

df_tmp=df_tmp.loc[df_tmp['FIPS'].isin(my_FIPS)]

print(df_tmp['FIPS'].unique())

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']


# Calculate Institutions composite (wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'wgi_avg'},inplace=True)


##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's

iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zd_sfi', 'ze_Population (millions)'],
         ['za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zd_sfi','ze_Population (millions)'],
         ['zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zd_sfi','ze_Population (millions)'],
         ['wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'wgi_avg'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'wgi_avg','zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality','wgi_avg','zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_5_lo_wvs.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)



##Regression Table 2.6 --LN(openness)

In [ ]:

df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']

# Calculate Institutions composite (zc_wgi_avg)
#df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')|(df_tmp['desc']=='openness')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')
df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

# Convert 'openness' from 'desc' value to a column value, then merge with df_loop
df_t2=df_a.loc[df_a['desc']=='openness'].copy(deep=True)
df_t2=df_t2[['FIPS','desc','value']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)
df_t2=df_t2.pivot(index='FIPS', columns='desc',values='value')
df_t2.rename(columns={'openness':'zc_openness'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a['zc_openness']=np.log(df_a['zc_openness'])

df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's
iv_list=[['zc_openness'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_openness'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_openness','zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality','zc_openness','zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_6_openness.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)




##Regression Table 2.7 --Openness-religion interaction term

In [ ]:
df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']

# Calculate Institutions composite (zc_wgi_avg)
#df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')|(df_tmp['desc']=='openness')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')
df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

# Convert 'openness' from 'desc' value to a column value, then merge with df_loop
df_t2=df_a.loc[df_a['desc']=='openness'].copy(deep=True)
df_t2=df_t2[['FIPS','desc','value']]
df_t2.drop_duplicates(inplace=True, ignore_index=True)
df_t2=df_t2.pivot(index='FIPS', columns='desc',values='value')
df_t2.rename(columns={'openness':'zc_openness'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a['zc_openness']=np.log(df_a['zc_openness'])

#add interaction terms
df_a['xa_open_Prot']=df_a['aCIAProtAll']*df_a['zc_openness']
df_a['xb_open_Cath']=df_a['bCIACathAll']*df_a['zc_openness']
df_a['xc_open_Ortho']=df_a['cCIAOrthAll']*df_a['zc_openness']
df_a['xd_open_Mus']=df_a['eCIAMusAll']*df_a['zc_openness']
df_a['xe_open_BudHind']=df_a['hCIABudHindAll']*df_a['zc_openness']
df_a['xf_open_NoRel']=df_a['mCIANoRelAll']*df_a['zc_openness']


df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

#create list of lists for regression IV's
iv_list=[['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'xa_open_Prot','xb_open_Cath','xc_open_Ortho','xd_open_Mus','xe_open_BudHind','xf_open_NoRel',
          'zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv.transpose())

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_7_openness_interaction.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.transpose().to_excel(joined_path)




##Regression Table 2.8 --Latitude

In [ ]:
df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']

# Calculate Institutions composite (zc_wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'zc_wgi_avg'},inplace=True)


##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

df_a.rename(columns={'AbsLatitude':'zc_AbsLatitude'},inplace=True)
#create list of lists for regression IV's
#latitude
iv_list=[['zc_AbsLatitude'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_AbsLatitude'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_AbsLatitude','zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality','zc_AbsLatitude','zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_8_latitude.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)



##Regression Table 2.9 --continent

In [ ]:
# 'continents: 'Asia' 'Africa' 'South America' 'North America' 'Europe' 'Oceania' 0
#'continent_Africa', 'continent_Asia', 'continent_North America', 'continent_Oceania', 'continent_South America'

df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']


# Calculate Institutions composite (zc_wgi_avg)

avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'zc_wgi_avg'},inplace=True)


##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')

df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)


#my_cont=df_a['continent'].unique()
#print(my_cont)
df_a = pd.get_dummies(df_a, columns=['continent'])

df_a.rename(columns={'continent_Africa':'zc_Africa',
             'continent_Asia':'zc_Asia', 
             'continent_North America':'zc_N_Amer', 
             'continent_Oceania':'zc_Oceania', 
             'continent_South America':'zc_S_Amer'},inplace=True)

#'zc_Africa','zc_Asia', 'zc_N_Amer','zc_Oceania','zc_S_Amer'

#create list of lists for regression IV's

iv_list=[['zc_Africa','zc_Asia', 'zc_N_Amer','zc_Oceania','zc_S_Amer'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_Africa','zc_Asia', 'zc_N_Amer','zc_Oceania','zc_S_Amer'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_Africa','zc_Asia', 'zc_N_Amer','zc_Oceania','zc_S_Amer',
          'zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zc_Africa','zc_Asia', 'zc_N_Amer','zc_Oceania','zc_S_Amer',
          'zd_sfi','ze_Population (millions)'],
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_9_Continent.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)




##Regression Table 2.10 --region



In [ ]:

df_tmp=df_merged.copy(deep=True)

# Build linear regression model 
# Split data into predictors X and output Y
predictors = ['FD','FI','FM']
# Calculate Institutions composite (zc_wgi_avg)
avg_col_lst = ['Voice and Accountability: Estimate',
               'Political Stability and Absence of Violence/Terrorism: Estimate',
               'Government Effectiveness: Estimate',
               'Regulatory Quality: Estimate',
               'Rule of Law: Estimate',
               'Control of Corruption: Estimate']


#limit the values in the desc column to calc wgi mean-values 
df_t2=df_tmp.loc[(df_tmp['desc'].isin(avg_col_lst))].copy(deep=True)

df_t2=df_t2.groupby(['FIPS','country_relig'],as_index=False)['overall'].mean()
df_t2.rename(columns={'overall':'zc_wgi_avg'},inplace=True)

##df_a=df_tmp.loc[(df_tmp['desc'].isin(predictors))|(df_tmp['desc']=='sfi')]
df_a.rename(columns={'overall':'value'},inplace=True)

# Convert 'zd_sfi' from 'desc' value to a column value, then merge with df_loop
df_sfi=df_a.loc[df_a['desc']=='sfi'].copy(deep=True)
df_sfi=df_sfi[['FIPS','desc','value']]
df_sfi.drop_duplicates(inplace=True, ignore_index=True)
df_sfi=df_sfi.pivot(index='FIPS', columns='desc',values='value')
df_sfi.rename(columns={'sfi':'zd_sfi'},inplace=True)

#merge sfi column with FD values df 
df_a=df_a.merge(df_sfi, on='FIPS', how='outer')
df_a=df_a.merge(df_t2, on='FIPS', how='outer')

#natural log transformations
df_a['ze_Population (millions)']=np.log(df_a['ze_Population (millions)'])
df_a['zb_BeckSettlerMortality']=np.log(df_a['zb_BeckSettlerMortality'])
df_a.replace([np.inf, -np.inf], 0, inplace=True)
df_a=df_a.fillna(0)

df_a = pd.get_dummies(df_a, columns=['region'])
#print(df_a.columns.to_list())

df_a.rename(columns={
    'region_Northern Europe':'zc_N_Europe',  
    'region_Southern Asia':'zc_S_Asia', 
    'region_Southern Europe':'zc_S_Europe', 
    'region_Western Africa':'zc_W_Africa', 
    'region_Western Asia':'zc_W_Asia', 
    'region_Australia and New Zealand':'zc_Australia_NZ', 
    'region_Caribbean':'zc_Caribbean', 
    'region_Central America':'zc_C_Amer', 
    'region_Central Asia':'zc_C_Asia', 
    'region_Eastern Africa':'zc_E_Africa', 
    'region_Eastern Asia':'zc_E_Asia', 
    'region_Eastern Europe':'zc_E_Europe', 
    'region_Melanesia':'zc_Melanesia', 
    'region_Micronesia':'zc_Micronesia', 
    'region_Middle Africa':'zc_M_Africa', 
    'region_Northern Africa':'zc_N_Africa', 
    'region_Northern America':'zc_N_Amer', 
    'region_Polynesia':'zc_Polynesia', 
    'region_South America':'zc_S_Amer', 
    'region_South-eastern Asia':'zc_SE_Asia', 
    'region_Southern Africa':'zc_S_Africa'},inplace=True)

# 'zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ','zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa','zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 'zc_S_Africa',

#create list of lists for regression IV's

iv_list=[['zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ','zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa','zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 'zc_S_Africa'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ','zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa','zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 'zc_S_Africa'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ','zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa','zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 'zc_S_Africa',
          'zd_sfi','ze_Population (millions)'],
         ['aCIAProtAll', 'cCIAOrthAll', 'eCIAMusAll', 'hCIABudHindAll', 'mCIANoRelAll', 'kCIAOthWithJew', 
          'za_IsikFrench','zb_BeckSettlerMortality',
          'zc_N_Europe', 'zc_S_Asia', 'zc_S_Europe','zc_W_Africa', 'zc_W_Asia', 'zc_Australia_NZ','zc_Caribbean','zc_C_Amer', 'zc_C_Asia', 'zc_E_Africa', 'zc_E_Asia', 'zc_E_Europe', 'zc_Melanesia','zc_Micronesia','zc_M_Africa', 'zc_N_Africa','zc_N_Amer', 'zc_Polynesia','zc_S_Amer', 'zc_SE_Asia', 'zc_S_Africa',
          'zd_sfi','ze_Population (millions)']
        ]

piv = regress_table(df_a, predictors, iv_list)
print('\npiv\n:',piv)

mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/tables' 
myfile = 'nTable_2_10_UNregion.xlsx'
joined_path = os.path.join(mypath, myfile)
piv.to_excel(joined_path)




##Classification (pycaret)

In [ ]:
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']
                    

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=df_merged.loc[df_merged['desc'].isin(fd_list)].copy(deep=True)
a.rename(columns={'overall':'value'},inplace=True)

X=a[['country_relig','desc','IsikCommon','IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 
          ('value', 'Financial Development Index'), 
          ('value', 'Financial Institutions Index'), 
          ('value', 'Financial Institutions Depth Index'), 
          ('value', 'Financial Institutions Access Index'), 
          ('value', 'Financial Institutions Efficiency Index'),
          ('value', 'Financial Markets Index'),
          ('value', 'Financial Markets Depth Index'), 
          ('value', 'Financial Markets Access Index'), 
          ('value', 'Financial Markets Efficiency Index')]]

my_model = setup(data=X, # we are setting up the model using the training dataset
                target='desc', # this is the response variable (since this is a classification model, the target should be a categorical feature)
                numeric_imputation='mean', #this parameter says to replace NULL values with the mean of a column
                #ignore_features = ['class', 'who', 'adult_male', 'alive', 'embarked', 'alone', 'deck'], #ignore the duplicate features and features with excessive NULLs
                #categorical_features = ['sex', 'embark_town', 'pclass'],
                #numeric_features = ['age', 'fare', 'sibsp', 'parch'],
                silent=True 
                )
compare_models()
my_lr_mod=create_model('lr')
#evaluate_model(my_lr_mod)

In [ ]:
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=df_merged.loc[df_merged['desc'].isin(fd_list)].copy(deep=True)
a.rename(columns={'overall':'value'},inplace=True)
a_piv=np.round(a.pivot_table(index=['FIPS','country_relig'],columns=['desc'],values=['value']),3)
print ('before', a_piv.head())

#a_piv.columns = a_piv.columns.map(''.join)
a_piv.columns = a_piv.columns.droplevel(0)
print ('after', a_piv.head())

#a_piv.columns.set_levels('a','b','c',level=1,inplace=True)
a=a_piv.merge(a,on='FIPS',how='outer')
#a.rename(columuns=('value', 'Financial Institutions Index'),

print(a.head(10))


X=a[['country_relig','desc','IsikCommon','IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']]

my_model = setup(data=X, # we are setting up the model using the training dataset
                target='Financial Development Index', # this is the response variable (since this is a classification model, the target should be a categorical feature)
                numeric_imputation='mean', #this parameter says to replace NULL values with the mean of a column
                #ignore_features = ['class', 'who', 'adult_male', 'alive', 'embarked', 'alone', 'deck'], #ignore the duplicate features and features with excessive NULLs
                #categorical_features = ['sex', 'embark_town', 'pclass'],
                #numeric_features = ['age', 'fare', 'sibsp', 'parch'],
                silent=True 
                )
compare_models()
my_lr_mod=create_model('lr')
#evaluate_model(my_lr_mod)


In [ ]:
predictors = ['Financial Development Index',
              'Financial Institutions Index',
              'Financial Institutions Depth Index',
              'Financial Institutions Access Index', 
              'Financial Institutions Efficiency Index',
              'Financial Markets Index',
              'Financial Markets Depth Index', 
              'Financial Markets Access Index', 
              'Financial Markets Efficiency Index']

# Generate data for dfs
my_rule_law_lst=['IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed']

my_col_list=['FIPS','country_relig','desc','overall','IsikCommon', 'IsikCivil', 'IsikFrench', 'IsikGerman', 'IsikNordic', 'IsikSoc', 'IsikMixed', 'IsikCivil']

a=df_merged.loc[df_merged['desc'].isin(fd_list)].copy(deep=True)

a_piv=np.round(a.pivot_table(index=['FIPS','country_relig'],columns=['desc'],values=['overall']),3)
a=a_piv.merge(a,on='FIPS',how='outer')

X=a[['country_relig','desc','IsikCommon']]

my_model = setup(data=X, # we are setting up the model using the training dataset
                target='IsikCommon', # this is the response variable (since this is a classification model, the target should be a categorical feature)
                numeric_imputation='mean', #this parameter says to replace NULL values with the mean of a column
                #ignore_features = ['class', 'who', 'adult_male', 'alive', 'embarked', 'alone', 'deck'], #ignore the duplicate features and features with excessive NULLs
                #categorical_features = ['sex', 'embark_town', 'pclass'],
                #numeric_features = ['age', 'fare', 'sibsp', 'parch'],
                silent=True 
                )
compare_models()
my_lr_mod=create_model('lr')
#evaluate_model(my_lr_mod)

In [ ]:
evaluate_model(my_lr_mod)


In [ ]:
print(result.coef_,'\nitercept\n', result.intercept_,'\nsigma\n', result.sigma_)


#multiple t-tests

In [ ]:
print(stop program here)

In [ ]:
#REFRESH the data for the counts
c=pd.melt(df_iset_subset,id_vars=['country','FIPS','desc'])
c.rename(columns={'variable':'year'},inplace=True)

#Merge with religion data
b=c.merge(df_all_religions,on='FIPS',how='outer')
a=b.merge(df_country,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)
a=a.loc[~a['value'].isnull()]
a=a.loc[a['year']=='overall']
#print (a.head(100))

print (a.head())

#Use relig_list from code-block that creates df_all_religions
#Create permutation of religions
perm_set=list( permutations( relig_list, 2 ) )

#Convert to set to eliminate redundant pairs
perm_set=set(perm_set)

#Convert back to a list for ease of use
for x in perm_set:
    relig_pair=list(x)
    #relig_pair[0] vs. relig_pair[1] for t-test

    # 2-sample t-test for each DV variable
    collst=predictors #a['desc'].unique()

    for curr_col in collst:
        
        # Dataframes to use for statistical tests
        df_a=a.loc[(a['desc']==curr_col) & (a['country_relig']==relig_pair[0]) & (a['year']=='overall')].copy(deep=True)
        df_b=a.loc[(a['desc']==curr_col) & (a['country_relig']==relig_pair[1]) & (a['year']=='overall')].copy(deep=True)

        #perform Bartlett's test homogeneity of variance
        stat, p = stats.bartlett(df_a['value'], df_b['value'])
        print("H0 is equal variances; Barlett's p-value",p, curr_col)
        
        # Levene's Test in Python with Scipy:
        stat, p = stats.levene(df_a['value'], df_b['value'])
        print("H0 is equal variances; Levene's p-value (more robust to Normal distrib violations)",p, curr_col)

        ####### p-value ~since we are looking to see if OIC is less than the non-OIC, we only need one-tail. Since scipy give 2-tail, we can divide p by 2

        if p > .05: # p value from Levene homoscedasticity test
            t_stat, p_val = stats.ttest_ind(df_a['value'], df_b['value'], equal_var=True)
            #p_val=p_val/2 #one-tail test
            if p_val < .001:
                print('***',curr_col,'for','\n     ',relig_pair[0], 'compared to',relig_pair[1])
            elif p_val < .01:
                print('**',curr_col,'for','\n     ',relig_pair[0], 'compared to',relig_pair[1])
            elif p_val < .05:
                    print('*',curr_col,'for','\n     ',relig_pair[0], 'compared to',relig_pair[1])
            else:
                print('#',curr_col,'for','\n     ',relig_pair[0], 'compared to',relig_pair[1], 'is not significantly different')
        else:
            print('NEED to do ttest equiv for heteroscedasticity for', relig_pair)


#Multiple comparision of means Tukey (df_tukey_results)

In [ ]:

# Use MOD function to determine which 'desc' value is being evaluated
uniq_desc_num=df_iset['desc'].nunique()
uniq_desc_list=df_iset['desc'].unique()

#make long
a=pd.melt(df_iset,id_vars=['country','FIPS','desc'])
a.rename(columns={'variable':'year'},inplace=True)

a=a.merge(df_all_religions,on='FIPS',how='outer')
a.rename(columns={'country_x':'country'},inplace=True)
a=a.loc[~a['value'].isnull()]
print(a.head())
'''
a['dupe']=a.duplicated()
print ('DUPES:\n', a.loc[a['dupe']==True], '\nhead()',a.head())
'''

desc_key=[]
decade_key=[]
atables=[]
ttables=[]
df_tukey_results=[]

# iterate over each group
my_decades_list=['overall','1980_89','1990_99','2000_09','2010_20']
for decade in my_decades_list: 
    for curr_desc in uniq_desc_list:
        my_endog = a.loc[(a['desc']==curr_desc) & (a['year']==decade)]['value']
        my_endog=pd.to_numeric(my_endog, errors='coerce')
        my_endog=my_endog.dropna()
        my_curr_grp = a.loc[(a['desc']==curr_desc) & (a['year']==decade)]['country_relig']
        my_curr_grp=my_curr_grp.dropna()

        #model=ols('{} ~ country_relig'.format(curr_desc), data=my_endog).fit() #added 'overall' column
        #anova_table = sm.stats.anova_lm(model, typ=2)
        exclude_lst=['taoism','druze','Poverty headcount ratio at national poverty lines (% of population)']
        if ((my_endog.size == my_curr_grp.size) & (my_endog.size>2) & (curr_desc not in exclude_lst)):
            '''            
            if curr_desc in exclude_lst:
                continue
            else:            
            '''
            tukey_results = pairwise_tukeyhsd(endog=my_endog,
                                    groups=my_curr_grp,
                                    alpha=0.1)
            desc_key.append(curr_desc)
            decade_key.append(decade)
            #atables.append(anova_table)
            ttables.append(tukey_results)

            #print(anova_table)
            print (decade, 'for', currcol,'\nTukey HSD H0 = means are equal\n')
            #print(tukey_results) #this prints the results from the tukey class object

            #To save tukey results into a dataframe...
            #https://stackoverflow.com/questions/40516810/saving-statmodels-tukey-hsd-into-a-python-pandas-dataframe
            df_tukey_results.append(pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0]))
df_tukey_results=pd.DataFrame(df_tukey_results)
print(df_tukey_results.info(verbose=True)) #this prints the results from the pd.DataFrame()

    #Save df_iset, df_demograph, df_incgrp to disk

#df_ttables = pd.DataFrame(data=ttables.data[1:], columns=ttables.data[0])



#Create Tukey plots

In [ ]:
from pandas.api.types import is_string_dtype

print('\nTukey plots KEY:\n   Dot = mean for group; Bar=90% CI; Tukey HSD is based on CI; Blue=Reference Group; Red=Different than Reference Group\n\n')
i=0
while i < len(ttables):
    my_label = desc_key[i]+'_'+decade_key[i]
    '''    # create xlabel for each figure
    if uniq_desc_num > 0:
        curr_decade=my_decades_list[i//uniq_desc_num] 
        curr_desc=uniq_desc_list[np.mod(i,uniq_desc_num)]
    else:
        curr_decade=np.nan
        curr_desc=np.nan

    if(isinstance(curr_decade, type('str')) & isinstance(curr_desc, type('str'))):
        my_label=my_decades_list[i//uniq_desc_num]+'_'+uniq_desc_list[np.mod(i,uniq_desc_num)]
    else:
        my_label='None'
        print('no label for decade=',curr_decade, type(curr_decade), 'and desc=', curr_desc, type(curr_desc))
    '''    
    try:
        ttables[i].plot_simultaneous(comparison_name='CIAProtAll',xlabel=my_label)
    except:
        ttables[i].plot_simultaneous(xlabel=my_label)
    
    my_filename = ''.join(char for char in my_label if char.isalnum())
    mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/TukeyPlots' 
    myfile = my_filename + '.pdf'
    joined_path = os.path.join(mypath, myfile)
    plt.savefig(joined_path)                  
    i+=1


#Print country:religion assignments

In [ ]:
cols=df_all_religions.columns.to_list()
print (cols)
print('\nTotal countries:',df_all_religions['country'].nunique())
for curr_rel in relig_list:
    a=df_all_religions.loc[df_all_religions['country_relig']==curr_rel]
    #the following line of code removes duplicated columns in data
    a = a.loc[:,~a.columns.duplicated()]
    print(curr_rel,'Count:',a['country'].nunique() ,'\n',a['country'].unique())



#Pre-process data

##KNN to replace NaN
####naming convention:
*  tmp=no KNN, not transformed
*  i=inc_grp KNN
*  t=transformed
*  o=overall KNN

>e.g.,

>>
*    tmp_OIC = no KNN, not transformed
*    df_OIC_i = inc_grp KNN, not transformed
*    df_OIC_o = overall KNN, not *transformed
*    df_OIC_it = inc_grp KNN, transformed
*    df_OIC_ot = overall KNN, transformed

####  groups:
*    OIC
* not 
* all 
* OECD

In [ ]:
####
#
#
# Calculate the fillna(knn) with OIC & IncGrp & Region?
# Generate boxplots by region too
# Can I show plot by region & incGrp & OIC
#
#
######
#df2=df2.fillna({'Muslims':0})
#'region', 'inc_grp', 'OECD'

#create subsets of overall dataframe
OIC_demo=df_demograph.loc[df_demograph['OIC']==1]
notOIC_demo=df_demograph.loc[df_demograph['OIC']!=1]
OECD_demo=df_demograph.loc[df_demograph['OECD']==1]
#turn into dataframe
OIC_countries = pd.DataFrame(OIC_demo['country'].unique(), columns = ['country'])
notOIC_countries = pd.DataFrame(notOIC_demo['country'].unique(), columns = ['country'])
OECD_countries = pd.DataFrame(OECD_demo['country'].unique(), columns = ['country'])
all_countries = pd.DataFrame(df_demograph['country'].unique(), columns = ['country'])
class_codes = df_demograph[['country','FIPS','region','inc_grp', 'OIC', 'OECD']]
#merge with other data
tmp_OIC=tmp.merge(OIC_countries,on='country',how='inner')
tmp_OIC=tmp_OIC.merge(class_codes, on='country',how='inner')
tmp_notOIC=tmp.merge(notOIC_countries,on='country',how='inner')
tmp_notOIC=tmp_notOIC.merge(class_codes, on='country',how='inner')
tmp_OECD=tmp.merge(OECD_countries,on='country',how='inner')
tmp_OECD=tmp_OECD.merge(class_codes, on='country',how='inner')
tmp_all=tmp.merge(all_countries, on='country',how='inner')
tmp_all=tmp_all.merge(class_codes, on='country',how='inner')

'''
KNN = fillna with imputed KNN value

df naming convention:
  tmp=no KNN, not transformed
  i=inc_grp KNN
  t=transformed
  o=overall KNN

  e.g.,
    tmp_OIC = no KNN, not transformed
    df_OIC_i = inc_grp KNN, not transformed
    df_OIC_o = overall KNN, not transformed
    df_OIC_it = inc_grp KNN, transformed
    df_OIC_ot = overall KNN, transformed

  groups:
    OIC, not, all, OECD
'''


''' /////// overall & by inc_grp //////// '''
df_OECD_o = tmp_OECD.copy(deep=True)
df_all_o = tmp_all.copy(deep=True)
df_OIC_o = tmp_OIC.copy(deep=True)
df_not_o = tmp_notOIC.copy(deep=True)

df_OIC_l = tmp_OIC.loc[tmp_OIC['inc_grp']=='Low income']
df_OIC_lm = tmp_OIC.loc[tmp_OIC['inc_grp']=='Lower middle income']
df_OIC_um = tmp_OIC.loc[tmp_OIC['inc_grp']=='Upper middle income']
df_OIC_h = tmp_OIC.loc[tmp_OIC['inc_grp']=='High income']

df_not_l = tmp_notOIC.loc[tmp_notOIC['inc_grp']=='Low income']
df_not_lm = tmp_notOIC.loc[tmp_notOIC['inc_grp']=='Lower middle income']
df_not_um = tmp_notOIC.loc[tmp_notOIC['inc_grp']=='Upper middle income']
df_not_h = tmp_notOIC.loc[tmp_notOIC['inc_grp']=='High income']

#instantiate new lists
df_OIC_ot=df_not_ot=df_OECD_ot=df_all_ot=[]
df_OIC_it=df_not_it=df_all_it=[]
df_OIC_i=df_not_i=df_all_i=[]

dfs=[df_OIC_l,df_OIC_lm, df_OIC_um, df_OIC_h,
     df_not_l, df_not_lm, df_not_um, df_not_h,
     df_OIC_o, df_not_o, df_OECD_o, df_all_o, 
     df_OIC_ot, df_not_ot, df_OECD_ot, df_all_ot, 
     df_OIC_i, df_not_i, df_all_i, 
     df_OIC_it, df_not_it, df_all_it, 
     tmp_OIC, tmp_notOIC, tmp_OECD, tmp_all
    ]
#setup a dict and lists for the dataframes
dfs_dict={'df_OIC_l':df_OIC_l, 'df_OIC_lm':df_OIC_lm, 'df_OIC_um':df_OIC_um, 'df_OIC_h':df_OIC_h,
     'df_not_l':df_not_l, 'df_not_lm':df_not_lm, 'df_not_um':df_not_um, 'df_not_h':df_not_h,
     'df_OIC_o':df_OIC_o, 'df_not_o':df_not_o,'df_OECD_o':df_OECD_o, 'df_all_o':df_all_o, 
     'df_OIC_ot':df_OIC_ot, 'df_not_ot':df_not_ot,'df_OECD_ot':df_OECD_ot, 'df_all_ot':df_all_ot, 
     'df_OIC_i':df_OIC_i, 'df_not_i':df_not_i,'df_all_i':df_all_i, 
     'df_OIC_it':df_OIC_it, 'df_not_it':df_not_it,'df_all_it':df_all_it, 
     'tmp_OIC':tmp_OIC, 'tmp_notOIC':tmp_notOIC, 'tmp_OECD':tmp_OECD, 'tmp_all':tmp_all
     }

dflst=[df_OIC_l, df_OIC_lm, df_OIC_um, df_OIC_h,
       df_not_l, df_not_lm, df_not_um, df_not_h,
       df_OECD_o, df_all_o, df_OIC_o, df_not_o]

# Convert to wide before transformations
df_OIC_l=df_OIC_l.pivot_table(index='country', columns='desc', values='value')
df_OIC_lm=df_OIC_lm.pivot_table(index='country', columns='desc', values='value')
df_OIC_um=df_OIC_um.pivot_table(index='country', columns='desc', values='value')      
df_OIC_h=df_OIC_h.pivot_table(index='country', columns='desc', values='value')
df_not_l =df_not_l.pivot_table(index='country', columns='desc', values='value')
df_not_lm =df_not_lm.pivot_table(index='country', columns='desc', values='value')
df_not_um=df_not_um.pivot_table(index='country', columns='desc', values='value')
df_not_h=df_not_h.pivot_table(index='country', columns='desc', values='value')
df_all_o=df_all_o.pivot_table(index='country', columns='desc', values='value')
df_OIC_o =df_OIC_o.pivot_table(index='country', columns='desc', values='value')
df_not_o =df_not_o.pivot_table(index='country', columns='desc', values='value')
df_OECD_o=df_OECD_o.pivot_table(index='country', columns='desc', values='value')
tmp_all=tmp_all.pivot_table(index='country', columns='desc', values='value')
tmp_OIC =tmp_OIC.pivot_table(index='country', columns='desc', values='value')
tmp_notOIC =tmp_notOIC.pivot_table(index='country', columns='desc', values='value')
tmp_OECD=tmp_OECD.pivot_table(index='country', columns='desc', values='value')

df_univar_OIC = df_OIC_o.copy(deep=True)
df_univar_not = df_not_o.copy(deep=True)
df_univar_all = df_all_o.copy(deep=True)

#KNN imputation for missing values by OIC income group and by Not_OIC income group
imputer = KNNImputer(n_neighbors= 5) #use the 5 closest neighbors

dfs_dict={'df_OIC_l':df_OIC_l, 'df_OIC_lm':df_OIC_lm, 'df_OIC_um':df_OIC_um, 'df_OIC_h':df_OIC_h}
d=[] 
for curkey, curval in dfs_dict.items():
  t=imputer.fit_transform(curval)
  r=pd.DataFrame(t,columns=curval.columns,index=curval.index)
  d.append(r)
df_OIC_i = pd.concat(d) #answer to how to save dataframe inside loop, need to define d[] and to d.append during iteration
df_OIC_i=df_OIC_i.merge(class_codes, on='country',how='inner')

#same loop for not, 
dfs_dict={'df_not_l':df_not_l, 'df_not_lm':df_not_lm, 'df_not_um':df_not_um, 'df_not_h':df_not_h}
d=[] 
for curkey, curval in dfs_dict.items():
  t=imputer.fit_transform(curval)
  r=pd.DataFrame(t,columns=curval.columns,index=curval.index)
  d.append(r)
df_not_i = pd.concat(d) #answer to how to save dataframe inside loop, need to define d[] and to d.append during iteration
df_not_i=df_not_i.merge(class_codes, on='country',how='inner')

df_all_i =pd.concat([df_not_i, df_OIC_i])

dfs_dict={'df_OIC_o':df_OIC_o}
d=[] 
for curkey, curval in dfs_dict.items():
  t=imputer.fit_transform(curval)
  r=pd.DataFrame(t,columns=curval.columns,index=curval.index)
  d.append(r)
df_OIC_o = pd.concat(d) #answer to how to save dataframe inside loop, need to define d[] and to d.append during iteration
df_OIC_o=df_OIC_o.merge(class_codes, on='country',how='inner')

dfs_dict={'df_not_o':df_not_o}
d=[] 
for curkey, curval in dfs_dict.items():
  t=imputer.fit_transform(curval)
  r=pd.DataFrame(t,columns=curval.columns,index=curval.index)
  d.append(r)
df_not_o = pd.concat(d) #answer to how to save dataframe inside loop, need to define d[] and to d.append during iteration
df_not_o=df_not_o.merge(class_codes, on='country',how='inner')

dfs_dict={'df_OECD_o':df_OECD_o}
d=[] 
for curkey, curval in dfs_dict.items():
  t=imputer.fit_transform(curval)
  r=pd.DataFrame(t,columns=curval.columns,index=curval.index)
  d.append(r)
df_OECD_o = pd.concat(d) #answer to how to save dataframe inside loop, need to define d[] and to d.append during iteration
df_OECD_o=df_OECD_o.merge(class_codes, on='country',how='inner')

df_all_o=pd.concat([df_not_o, df_OIC_o])

#print (df_all_o.info(verbose=True),'\n',df_not_o.info(verbose=True))
#print (df_all_o.tail(),'\n',df_not_o.tail())
#
#
####///////////////////SUCCESS!!\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
###//////////////how to save df in a loop\\\\\\\\\\\\\\\\\\\\\\\\\\
# see loop above this comment (before loop -> d=[], in loop -> d.append(new_list), after loop-> d=pd.concat(d))
#

'''
# data transformation (could use pycaret to transform data)
df_all_it = df_all_i
df_OIC_it = df_OIC_i
df_not_it = df_not_i
df_all_ot = df_all_o
df_OIC_ot = df_OIC_o
df_not_ot = df_not_o
df_OECD_ot = df_OECD_o

dfs={'df_all_it':df_all_it, 'df_OIC_it':df_OIC_it, 'df_not_it':df_not_it, 
       'df_all_ot':df_all_ot, 'df_OIC_ot':df_OIC_ot, 'df_not_ot':df_not_ot, 'df_OECD_ot':df_not_ot}

for curkey, curval in dfs_dict.items():
  my_list=curval.columns.to_list()
  for curr_col in my_list:
    if curr_col in ('Stock market capitalization to GDP (%)'):
      df[curkey]=(currdf[curr_col])**(1./2)
    elif curr_col in ('Outstanding deposits with commercial banks','GNI per capita (constant 2010 US$)','Outstanding loans from commercial banks'):
      if currdf[curr_col].notnull:
        currdf[curr_col]=np.log(currdf[curr_col])   
    elif (curr_col not in ['OIC','OECD'])&(is_numeric_dtype(currdf[curr_col])):
      currdf[curr_col]=(currdf[curr_col])**(1./3)
'''

### TODO ###
# (-) Why are still nulls in Stock Market Cap?
# (-) Transform data at regression run-time using pycaret
# (-) Use 'region', '%muslim' in regression models (not charts)
# (-) use same variables as Ihsan's paper
# (-) review Ihsan's paper to see if I'm missing anything else
# (-) figure out best box plots (huge difference between knn and null)




#Run classification models
####using pycaret

##df_iset Classification for unprocessed data (has null values)

In [ ]:
#Classification model for unprocessed data (data has null values)
dfa=tmp_all.copy(deep=True) #has null values
dfa=dfa.merge(class_codes, on='country',how='inner')

#dfa=df_all_i #.fillna with KNN by inc_grp & OIC  

#dfa=df_all_o #.fillna with KNN by OIC

dfa=dfa.loc[~dfa.OIC.isnull()]
delcols=['country','FIPS','region','inc_grp','OECD']
dfa.drop(delcols,inplace=True,axis=1)

dftrain = dfa.copy(deep=True)
my_model = setup(data=dftrain, # using my training data partition
                 target='OIC', # variable of interest (my dependent variable)
                 silent=True
                 )

# create the best-fit model
top = compare_models() 
mymod=create_model(top)
best = tune_model(mymod)

# compare all baseline models and select top 3
'''
#if you want to blend the top 3 models...
top = compare_models(n_select = 3) 
# tune top base models
tuned_top = [tune_model(i) for i in top]
# ensemble top tuned models
bagged_top = [ensemble_model(i) for i in tuned_top]
# blend top base models 
blender = blend_models(estimator_list = top) 
# select best model 
best = automl(optimize = 'f1')
'''


In [ ]:
evaluate_model(best)

##Classification for data with .fillna(KNN) for OIC vs. notOIC (df_all_o)
#####removed the colums exhibiting multicollinearity: 
```python
col_to_delete = ['Made or received digital payments in the past year (% age 15+)', 
                 'Debit card (% age 15+)',
                 'BANK DEPOSITS to GDP (%)',
                 'Used a debit or credit card to make a purchase in the past year (% age 15+)',
                 'Outstanding deposits with commercial banks',
                 'Credit card (% age 15+)'
                ]
```

In [ ]:
dfa=df_all_o.copy(deep=True)

dfa=dfa.loc[~dfa.OIC.isnull()]
delcols=['country','FIPS','region','inc_grp','OECD']
dfa.drop(delcols,inplace=True,axis=1)
col_to_delete = ['Made or received digital payments in the past year (% age 15+)', 
                 'Debit card (% age 15+)',
                 'BANK DEPOSITS to GDP (%)',
                 'Used a debit or credit card to make a purchase in the past year (% age 15+)',
                 'Outstanding deposits with commercial banks',
                 'Credit card (% age 15+)'
                ]
dfa.drop(col_to_delete,inplace=True,axis=1)

dftrain = dfa.copy(deep=True)
my_model = setup(data=dftrain, # using my training data partition
                 target='OIC', # variable of interest (my dependent variable)
                 silent=True
                 )

# create the best-fit model
top = compare_models() 
mymod=create_model(top)
best = tune_model(mymod)


In [ ]:
evaluate_model(best)

##Classification model for df_ifdindex

In [ ]:
df =df_ifdindex.merge(class_codes, on='country',how='inner')
df1 =df.pivot_table(index=['country','OIC'], columns='desc', values='value')
df2 =df.pivot_table(index=['country','OECD'], columns='desc', values='value')
df1.reset_index(inplace = True)
df2.reset_index(inplace = True)

df_OI =df1.loc[df['OIC']==1]
df_N =df1.loc[df['OIC']!=1]
df_OE =df2.loc[df['OECD']==1]

dfa=df1.copy(deep=True)
print(dfa.head())

dfa=dfa.loc[~dfa.OIC.isnull()]
try:
  delcols=['country','FIPS','region','inc_grp','OECD']
  dfa.drop(delcols,inplace=True,axis=1)
except:
  True
  
dftrain = dfa.copy(deep=True)
my_model = setup(data=dftrain, # using my training data partition
                 target='OIC', # variable of interest (my dependent variable)
                 silent=True
                 )

# create the best-fit model
top = compare_models() 
mymod=create_model(top)
best = tune_model(mymod)


In [ ]:
evaluate_model(best)

#t-test on mean differences (df_iset)
##????Where is the VIF dataframe being used????
#VIF to check for multicollinearity (idenified several variables to drop)
######VIF of greater than 5 indicates problem (must elim one variable at a time, then re-run VIF calc). The resulting list of columns to delete is below. There might also be additional columns to delete.
```python
col_to_delete = ['Made or received digital payments in the past year (% age 15+)', 
                 'Debit card (% age 15+)',
                 'BANK DEPOSITS to GDP (%)',
                 'Used a debit or credit card to make a purchase in the past year (% age 15+)',
                 'Outstanding deposits with commercial banks',
                 'Credit card (% age 15+)'
                ]
```

In [ ]:
  
# Dataframes to use for statistical tests
df_OI=df_OIC_o.copy(deep=True)
df_N=df_not_o.copy(deep=True)

# Save exist
  
# 2-sample (based on OIC) t-test for each DV variable
collst=df_OI.columns.to_list()
for curr_col in collst:
    #  if ((curr_col not in ['OIC','OECD','FIPS'])&(is_numeric_dtype(df_OI[curr_col]))):
    if (is_numeric_dtype(df_OI[curr_col])):
        t_stat, p_val = stats.ttest_ind(df_OI[curr_col], df_N[curr_col], equal_var=True)
       ####### p-value ~since we are looking to see if OIC is less than the non-OIC, we only need one-tail. Since scipy give 2-tail, we can divide p by 2
        if p_val/2 < .001:
            print('***',curr_col)
        elif p_val/2 < .01:
            print('**',curr_col)
        elif p_val/2 < .05:
            print('*',curr_col)
        else:
            print('#',curr_col, 'is not significantly different')


###### calculating VIF for each feature

vif_data =pd.DataFrame()
vif_data =pd.concat([df_OIC_o,df_not_o], axis=0)
vif_data.reset_index()

vif_data = vif_data._get_numeric_data()
#df.select_dtypes(include=np.number)

try:
    delcols=['FIPS','OIC','OECD']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['Made or received digital payments in the past year (% age 15+)']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['Debit card (% age 15+)']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['BANK DEPOSITS to GDP (%)']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['Used a debit or credit card to make a purchase in the past year (% age 15+)']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['Outstanding deposits with commercial banks']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True

#### re-run VIF after col delete
try:
    delcols=['Credit card (% age 15+)']
    vif_data.drop(delcols,inplace=True,axis=1)
except:
    True


#Start of the VIF procedure...
# the independent variables set
mycols=vif_data.columns.to_list()
X = add_constant(vif_data)

# VIF dataframe
vif_out=pd.DataFrame()
vif_out["feature"] = X.columns
  
# calculation
vif_out["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
  
print('\n', vif_out)


# 2-sample (based on OIC) t-test for each DV variable
t=df_ifdindex.merge(df_incgrp,on='country',how='inner')
t=t.merge(df,on='country',how='inner')

print (t.info())
df_OI = t.loc[t['OIC']==1]
df_N = t.loc[t['OIC']!=1]
#df_OI=df_ifdindex.loc [t['OIC']==1]
#df_N=df_ifdindex.loc [t['OIC']!=1]

collst=df_OI.columns.to_list()
for curr_col in collst:
    #  if ((curr_col not in ['OIC','OECD','FIPS'])&(is_numeric_dtype(df_OI[curr_col]))):
    if (is_numeric_dtype(df_OI[curr_col])):
        t_stat, p_val = stats.ttest_ind(df_OI[curr_col], df_N[curr_col], equal_var=True)
       ####### p-value ~since we are looking to see if OIC is less than the non-OIC, we only need one-tail. Since scipy give 2-tail, we can divide p by 2
        if p_val/2 < .001:
            print('***',curr_col)
        elif p_val/2 < .01:
            print('**',curr_col)
        elif p_val/2 < .05:
            print('*',curr_col)
        else:
            print('#',curr_col, 'is not significantly different')





#t-test difference of means (df_ifdinex)

#EDA statistics used to identify data to be cleansed

###Heatmap... not used because it was not useful



In [ ]:
'''
# using the transformed dataframe 

plt.figure(figsize=(7,25))

#print(df_OIC.info(verbose=True),df_OIC0.info(verbose=True))

cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)

g = sns.heatmap(
    df_OI.iloc[:,0:28], 
    square=True, # make cells square
    cbar_kws={'fraction' : 0.01}, # shrink colour bar
    cmap=cmap, 
    linewidth=1 # space between cells
)
plt.show()
'''

##Univariate EDA statistics

In [ ]:
''' this code block is meant to test modality... it is not working
#### pretty sure the issue is that data must = one column (not whole df)
!pip install unidip
from unidip import UniDip
import unidip.dip as dip

data=df_univar_all.copy(deep=True)
data = np.msort(data)
print(dip.diptst(data))
intervals = UniDip(data).run()
print(intervals)
'''

import pandas as pd
import numpy as np
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson

# show all rows & columns (wrap, truncate, see all, view)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)

# ADD more stuff: 
#     unimodal?, z-value, potential outliers, boxplots? qqplots? scatterplots? histograms?...
# needs interpretation insights (how to read)

# good usage of append, dict, df, and multiIndex 
#        https://www.kaggle.com/dimitreoliveira/model-stacking-feature-engineering-and-eda 

# calculate statistic metrics for features
def univariate_metrics (df_passed, name_of_df):
    #create blank lists
    my_col_dtype = []
    my_col_cnt = []
    my_col_num_distinct = []
    my_col_perc_missing = []
    my_col_cnt_null = []
    my_col_min = []
    my_col_max = []
    my_col_range = []
    my_col_mean = []
    my_col_std = []
    my_col_median = []
    my_col_Q25 = []
    my_col_Q75 = []
    my_col_CV = []
    my_col_skew = []
    my_col_kurtosis = []
    my_col_interp_skew = []
    my_col_interp_kurtosis = []
    my_col_mode = []
    my_col_unique_vals = []
    my_col_shapiro_p = []
    my_col_shapiro_p_interp = []
    my_col_K2_p = []
    my_col_K2_p_interp = []

    for mycol in df_passed.columns:  #add values to lists
        my_col_dtype.append({'Col_Name' : mycol, 'DType': df_passed.dtypes[mycol]})
        my_col_cnt.append({'Col_Name' : mycol, 'NbrValues': len(df_passed)-df_passed[mycol].isnull().sum()})
        my_col_cnt_null.append({'Col_Name' : mycol, 'CntNull': df_passed[mycol].isnull().sum()})
        my_col_num_distinct.append({'Col_Name' : mycol,'NumUnique': df_passed[mycol].nunique()})
        my_col_perc_missing.append({'Col_Name' : mycol, 'PercNull': df_passed[mycol].isnull().mean()})
        my_col_cnt_null.append({'Col_Name' : mycol, 'CntNull': df_passed[mycol].isnull().sum()})
        
        if df_passed[mycol].dtype.kind in 'iuf': # these metrics return a single value per feature
            my_col_min.append({'Col_Name' : mycol, 'Min' : df_passed[mycol].min()})
            my_col_max.append({'Col_Name' : mycol,'Max' : df_passed[mycol].max()})
            my_col_range.append({'Col_Name' : mycol,'Range' : df_passed[mycol].max() - df_passed[mycol].min()})
            my_col_mean.append({'Col_Name' : mycol,'Mean' : df_passed[mycol].mean()})
            my_col_std.append({'Col_Name' : mycol,'Std' : df_passed[mycol].std()})
            my_col_median.append({'Col_Name' : mycol,'Median' : df_passed[mycol].median()})
            my_col_Q25.append({'Col_Name' : mycol,'Q25' : df_passed[mycol].quantile(q=.25)})
            my_col_Q75.append({'Col_Name' : mycol,'Q75' : df_passed[mycol].quantile(q=.75)})
            if df_passed[mycol].mean() != 0: 
                my_col_CV.append({'Col_Name' : mycol,'CV' : df_passed[mycol].std() / df_passed[mycol].mean()})
            my_col_skew.append({'Col_Name' : mycol,'Skew' : df_passed[mycol].skew()})  
            curr_skew=df_passed[mycol].skew()
            if curr_skew >= 1:
                my_col_interp_skew.append({'Col_Name' : mycol, 'Skew Interp' : 'Significant positive skew'})
            elif curr_skew > 0.5:
                my_col_interp_skew.append({'Col_Name' : mycol,'Skew Interp' : 'Positive skew'})
            elif curr_skew >= -0.5:
                my_col_interp_skew.append({'Col_Name' : mycol,'Skew Interp' : 'Not skewed'})
            elif curr_skew > -1:
                my_col_interp_skew.append({'Col_Name' : mycol,'Skew Interp' : 'Negative skew'})
            else:
                my_col_interp_skew.append({'Col_Name' : mycol,'Skew Interp' : 'Significant negative skew'})
            my_col_kurtosis.append({'Col_Name' : mycol,'Kurtosis' : df_passed[mycol].kurtosis()})            
            curr_kurt = df_passed[mycol].kurtosis()
            if curr_kurt >= 3:
                my_col_interp_kurtosis.append({'Col_Name' : mycol, 'kurtosis Interp' : 'Positive kurtosis'})
            elif curr_kurt <= -3:
                my_col_interp_kurtosis.append({'Col_Name' : mycol,'kurtosis Interp' : 'Negative kurtosis'})
            else:
                my_col_interp_kurtosis.append({'Col_Name' : mycol,'kurtosis Interp' : 'No kurtosis'})
            ''' Need to figure out why Normal tests are not working... 
            #######test normality
            stat, p = shapiro(df_passed[mycol])
            my_col_shapiro_p.append({'Col_Name' : mycol,'Shapiro p' : p})
            alpha = .05
            if p > alpha:
                my_col_shapiro_p_interp.append({'Col_Name' : mycol,'Shapiro Interp' : 'Appears Normal'})
            else:
                my_col_shapiro_p_interp.append({'Col_Name' : mycol,'Shapiro Interp' : 'Appears Not Normal'})
                
            stat, p = normaltest(df_passed[mycol])
            my_col_K2_p.append({'Col_Name' : mycol,'K2 p' : p})
            if p > alpha:
                my_col_K2_p_interp.append({'Col_Name' : mycol,'K2 Interp' : 'Appears Normal'})
            else:
                my_col_K2_p_interp.append({'Col_Name' : mycol,'K2 Interp' : 'Appears Not Normal'})
            '''
        if df_passed[mycol].dtype.kind not in 'bf': # these metrics return multiple values per feature
            my_col_mode.append({'Col_Name' : mycol,'Mode' : df_passed[mycol].mode()})
            my_col_unique_vals.append({'Col_Name' : mycol,'UniqueVals' : df_passed[mycol].unique()})
            
    # Merge dictionary-lists to dataframe on "Col_Name"
    mylist = [my_col_dtype, my_col_num_distinct, my_col_cnt, my_col_cnt_null, my_col_perc_missing, 
              my_col_min, my_col_max, my_col_range, my_col_mean, my_col_std, my_col_Q25, 
              my_col_median, my_col_Q75, my_col_CV, my_col_skew, my_col_interp_skew, 
              my_col_kurtosis, my_col_interp_kurtosis]
    df1 = []
    df2 = []
    df13 = []
    df14 = []
    for currlist in mylist:
        if currlist == my_col_dtype:
            df1 = pd.DataFrame.from_dict(currlist) #since my_col_dtype is the first element, create (rather than append) the df
        else:
            df2 = pd.DataFrame.from_dict(currlist)
            df1 = pd.merge(left = df1, right = df2, on = 'Col_Name')
    
    #eliminate duplicate columns
    df1 = df1.loc[:,~df1.columns.duplicated()]
    #eliminate duplicate rows
    df1.drop_duplicates(inplace=True, ignore_index=True)

    df13 = pd.DataFrame.from_dict(my_col_mode)
    df14 = pd.DataFrame.from_dict(my_col_unique_vals)

    #######Currently not printing the unique values or mode (too long)
    #print(df1.transpose(), '\n\nUnique Values:\n', df14.transpose(), '\n\nMode:\n', df13.transpose())
    print(df1.transpose())
    mypath = '/gdrive/MyDrive/Colab Notebooks/Research_FinAccessData/' 
    myfile = 'mycol_'+name_of_df+'.xlsx'
    joined_path = os.path.join(mypath, myfile)
    df1.to_excel(joined_path)



###################
#NEED to figure out:
# why non-Nan value = num_unique
# why K2 normaltest is NaN
# why Shapiro is alway 1
#
###################
print('\n\nOIC Univariate stats:')    
univariate_metrics (df_univar_OIC,'df_univar_OIC')  

print('\n\nNot OIC Univariate stats:')    
univariate_metrics (df_univar_not,'df_univar_not')  

print('\n\nAll Univariate stats:')    
univariate_metrics (df_univar_all,'df_univar_all')  

     
#---------------------
#  OR   print a threashold-percent of rows with zero/null/unique values
#---------------------
'''
for col in df.columns:
    pct_nan = df[col].isna().mean() 
    pct_zero = (df[col] == 0).mean()
    pct_uniq = df[col].nunique()/df.shape[0]
    if pct_nan > .3:
        print('{0}: NULL value\'s = {1:.2f}%'.format(col, pct_nan * 100))
    if pct_zero > .3:
        print('{0}: ZERO\'s = {1:.2f}%'.format(col, pct_zero * 100))
    if pct_uniq < .05:
            print('\nUnique values for feature: ', col, '\n', df[col].unique()) 
'''


##commented out... Outlier detection using pycaret


In [ ]:
'''#seek out anomalies for the not OIC countries
from pycaret.anomaly import *


#detecting anolalies is an unsupervised model --there is no response variable. You are simply inspecting the data to 
#determine if there are outliers (data that does not look/act like the rest of the data set). This is performed by 
#looking for clustering of the data points in n-dimensional space, where n = number of features.

dftrain =df_not_save.sample(frac=.9, random_state=111)
dftest = df_not_save.drop(dftrain.index)
mynomaly = setup(dftrain, normalize=True, session_id = 123)

# creating different models to find anomolies
iforest = create_model('iforest')
knn = create_model('knn')

# Comparing anomalies in models
iforest_results = assign_model(iforest)
knn_results = assign_model(knn)
iforest_anomaly=iforest_results[iforest_results['Anomaly']==1]
knn_anomaly=knn_results[knn_results['Anomaly']==1]

mylist=iforest_anomaly.append(knn_anomaly)

print(mylist.transpose())
'''

In [ ]:
#seek out anomalies for the OIC countries
'''
dftrain =df_OIC_save.sample(frac=.9, random_state=111)
dftest = df_OIC_save.drop(dftrain.index)
mynomaly = setup(dftrain, normalize=True, session_id = 123)

iforest = create_model('iforest')
knn = create_model('knn')

# Comparing anomalies in models
iforest_results = assign_model(iforest)
knn_results = assign_model(knn)
iforest_anomaly=iforest_results[iforest_results['Anomaly']==1]
knn_anomaly=knn_results[knn_results['Anomaly']==1]

mylist=iforest_anomaly.append(knn_anomaly)
print(mylist.transpose())
'''


#Regression models:
###Financial depth
###Financial breadth

In [ ]:
from sklearn import linear_model
import statsmodels.api
# Dataframes to use for regression models 
dfa=df_all_o.copy(deep=True)
#KNN imputation for missing values 
imputer = KNNImputer(n_neighbors= 5) #use the 5 closest neighbors



my_y= dfa[['Stock market capitalization to GDP (%)',
          'Outstanding loans from commercial banks', 
          'Number of loan accounts with commercial banks per 1,000 adults', 
          'Private credit by deposit money banks to GDP (%)', 
          'Number of commercial bank branches per 100,000 adults',
          'ATMs per 100,000 adults','Account at a formal financial institution (% age 15+)',
          'Deposit accounts per 1,000 adults',
          'Liquid liabilities to GDP (%)',
          'Made payment using a mobile phone or the internet (% age 15+)',
          'Number of ATMs per 1,000 km2',
          'Number of commercial bank branches per 1,000 km2']]

list_of_responses = my_y.columns.to_list()

my_y=imputer.fit_transform(my_y)

my_y=pd.DataFrame(my_y,columns=list_of_responses,index=dfa.index)
dfa=dfa.fillna(0)

# list of models
models = []
X=[]
Y=[]
for resp in list_of_responses:
    '''    
    formula = resp + " ~  + 'OIC' + 'GNI per capita (constant 2010 US$)'"
    models.append(sm.OLS.from_formula(formula, data = dfa).fit())

    #  can access params for each element in model
    models[0].params
    '''
    print(resp)
    X = dfa[['OIC', 'GNI per capita (constant 2010 US$)']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y = my_y[resp]

    ''' 
    # with sklearn
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)

    print('Intercept: \n', regr.intercept_)
    print('Coefficients: \n', regr.coef_)

    '''
    '''
    # example prediction with sklearn
    New_Interest_Rate = 2.75
    New_Unemployment_Rate = 5.3
    print ('Predicted Stock Index Price: \n', regr.predict([[New_Interest_Rate ,New_Unemployment_Rate]]))
    '''
    # with statsmodels
    X = statsmodels.api.add_constant(X) # adding a constant
    
    model = statsmodels.api.OLS(Y, X).fit()
    predictions = model.predict(X) 
    
    #print('R^2:', model.rsquared, 'adj R^2:', model.rsquared_adj)
    print_model = model.summary()
    print('\nstatmodels.api:\n', print_model,'\n\n')

#Visualizations
####bar charts
####boxplots
####ridge plots

In [ ]:
# Dataframes to use for statistical tests 
df_OI=df_OIC_o
df_N=df_not_o
df_OE=df_OECD_o
  
# 2-sample (based on OIC) t-test for each DV variable
collst=df_OI.columns.to_list()
for curr_col in collst:
  if (curr_col not in ['OIC','OECD'])&(is_numeric_dtype(df_OI[curr_col])):
    t_stat, p_val = stats.ttest_ind(df_OI[curr_col], df_N[curr_col], equal_var=True)
    ####### p-value ~since we are looking to see if OIC is less than the non-OIC, we only need one-tail. Since scipy give 2-tail, we can divide p by 2
    if p_val/2 < .001:
      print('***',curr_col)
    elif p_val/2 < .01:
      print('**',curr_col)
    elif p_val/2 < .05:
      print('*',curr_col)
    else:
      print('#',curr_col, 'is not significantly different')

# Dataframes to use for statistical tests 
#build dataframes for df_ifdindex

df =df_ifdindex.merge(class_codes, on='country',how='inner')
df1 =df.pivot_table(index=['country','OIC'], columns='desc', values='value')
df2 =df.pivot_table(index=['country','OECD'], columns='desc', values='value')
df1.reset_index(inplace = True)
df2.reset_index(inplace = True)

df_OI =df1.loc[df['OIC']==1]
df_N =df1.loc[df['OIC']!=1]
df_OE =df2.loc[df['OECD']==1]

# 2-sample (based on OIC) t-test for each DV variable
collst=df_OI.columns.to_list()
for curr_col in collst:
  if (curr_col not in ['OIC','OECD'])&(is_numeric_dtype(df_OI[curr_col])):
    t_stat, p_val = stats.ttest_ind(df_OI[curr_col], df_N[curr_col], equal_var=True)
    ####### p-value ~since we are looking to see if OIC is less than the non-OIC, we only need one-tail. Since scipy give 2-tail, we can divide p by 2
    if p_val/2 < .001:
      print('***',curr_col)
    elif p_val/2 < .01:
      print('**',curr_col)
    elif p_val/2 < .05:
      print('*',curr_col)
    else:
      print('#',curr_col, 'is not significantly different')



##Bar charts by country (df_iset & df_ifdindex)
###excludes multicollinearity columns#
```python
col_to_delete = ['Made or received digital payments in the past year (% age 15+)', 
                 'Debit card (% age 15+)',
                 'BANK DEPOSITS to GDP (%)',
                 'Used a debit or credit card to make a purchase in the past year (% age 15+)',
                 'Outstanding deposits with commercial banks',
                 'Credit card (% age 15+)'
                ]
```

In [ ]:
# Dataframes to use for barplots
df_OI=df_OIC_o
df_N=df_not_o
df_OE=df_OECD_o

my_colors = ['#BDFCC9', '#00CDCD']
sns.set_palette(sns.color_palette(my_colors))

# Print set of bar chart plots
col_out=[['GNI per capita (constant 2010 US$)','Stock market capitalization to GDP (%)',
          'Outstanding loans from commercial banks', 'Number of loan accounts with commercial banks per 1,000 adults'], 
         ['Private credit by deposit money banks to GDP (%)', 'Number of commercial bank branches per 100,000 adults',
          'ATMs per 100,000 adults','Account at a formal financial institution (% age 15+)'],
         ['Deposit accounts per 1,000 adults',
          "Deposit money banks' assets to GDP (%)",
          'Liquid liabilities to GDP (%)'],
         ['Made payment using a mobile phone or the internet (% age 15+)',
          'Number of ATMs per 1,000 km2',
          'Number of commercial bank branches per 1,000 km2']
        ]

mygrp=0
while mygrp < len(col_out):
  fig = plt.figure(figsize=[14,14]) #width by height
  mycnt=0 #set temp variable to index the column variables
  ax=pd.Series(range(0,50))
  mycnt=0
  for mycol in col_out[mygrp]:
    #df_OI = df_OI_save    -----not sure why I did this
    try:
      df_OI = df_OI[df_OI[mycol]>0] # eliminate null and zero value rows from the plot
      plt_cols=len(col_out[mygrp])
      df_OI_sort = df_OI.reindex(df_OI[mycol].sort_values(ascending=True).index)
      # figure-out how to eliminate nan & 0 values from the dataframe
      ax[mycnt]=fig.add_subplot(1,plt_cols,mycnt+1)
      ax[mycnt].barh(df_OI_sort.country,df_OI_sort[mycol],height=.5, color='limegreen', alpha=.4)

      #  Add line plots (on top of bar) for averages: OIC, non-OIC, OECD (OECD not calc yet)
      ax[mycnt].axvline(df_OI[mycol].mean(), color='green', linewidth=2, alpha=.8, linestyle=':', label='OIC') 
      ax[mycnt].axvline(df_N[mycol].mean(), color='orange', linewidth=2, alpha=.3, linestyle='dashdot', label='Not OIC')
      ax[mycnt].axvline(df_OE[mycol].mean(), color='lightblue', linewidth=2, linestyle='--', label='OECD')
    
      #  Adjust the spines & axis labels
      ax[mycnt].set_xlabel(mycol, rotation=5, fontweight='light')
      ax[mycnt].tick_params(bottom=False)
      ax[mycnt].spines['top'].set_visible(False)
      ax[mycnt].spines['bottom'].set_visible(False)
      ax[mycnt].spines['left'].set_visible(True)
      ax[mycnt].spines['right'].set_visible(False)
      ax[mycnt].set(xticklabels=[])  
      ax[mycnt].grid(False)
      mycnt+=1
    except:
      True
  plt.axis('tight')
  plt.legend(bbox_to_anchor=(1.04,.1), borderaxespad=0)
  fig.tight_layout()
  fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
  


  #save the plot into a file
  mypath = '/gdrive/MyDrive/Colab Notebooks/FinAccessData' 
  myfile = 'barplot_df_iset_'+str(mygrp)+'.jpg'
  joined_path = os.path.join(mypath, myfile)
  #plt.savefig(joined_path,bbox_inches = 'tight', pad_inches = 0)

  fig.show()
  mygrp+=1




In [ ]:
#build barplots df_ifdindex

df =df_ifdindex.merge(class_codes, on='country',how='inner')
df1 =df.pivot_table(index=['country','OIC'], columns='desc', values='value')
df2 =df.pivot_table(index=['country','OECD'], columns='desc', values='value')
df1.reset_index(inplace = True)
df2.reset_index(inplace = True)

df_OI =df1.loc[df['OIC']==1]
df_N =df1.loc[df['OIC']!=1]
df_OE =df2.loc[df['OECD']==1]

col_out=[['FinDev','FinInstit','FinMarkets'],
         ['DepthFinInstit','AccFinInstit','EffFinInstit'],
         ['DepthFinMarkets','AccFinMarkets','EffFinMarkets']
        ]

mygrp=0
while mygrp < len(col_out):
  fig = plt.figure(figsize=[12,12]) #width by height
  mycnt=0 #set temp variable to index the column variables
  ax=pd.Series(range(0,50))
  mycnt=0
  for mycol in col_out[mygrp]:
    #df_OI = df_OI_save
    df_OI = df_OI[df_OI[mycol]>0] # eliminate null and zero value rows from the plot
    plt_cols=len(col_out[mygrp])
    df_OI_sort = df_OI.reindex(df_OI[mycol].sort_values(ascending=True).index)
    # figure-out how to eliminate nan & 0 values from the dataframe
    ax[mycnt]=fig.add_subplot(1,plt_cols,mycnt+1)
    ax[mycnt].barh(df_OI_sort.country,df_OI_sort[mycol],height=.5, color='limegreen', alpha=.4)

    #  Add line plots (on top of bar) for averages: OIC, non-OIC, OECD (OECD not calc yet)
    ax[mycnt].axvline(df_OI[mycol].mean(), color='green', linewidth=2, alpha=.8, linestyle=':', label='OIC') 
    ax[mycnt].axvline(df_N[mycol].mean(), color='orange', linewidth=2, alpha=.3, linestyle='dashdot', label='Not OIC')
    ax[mycnt].axvline(df_OE[mycol].mean(), color='lightblue', linewidth=2, linestyle='--', label='OECD')
  
    #  Adjust the spines & axis labels
    ax[mycnt].set_xlabel(mycol, rotation=5, fontweight='light')
    ax[mycnt].tick_params(bottom=False)
    ax[mycnt].spines['top'].set_visible(False)
    ax[mycnt].spines['bottom'].set_visible(False)
    ax[mycnt].spines['left'].set_visible(True)
    ax[mycnt].spines['right'].set_visible(False)
    ax[mycnt].set(xticklabels=[])  
    ax[mycnt].grid(False)
    mycnt+=1
  plt.axis('tight')
  plt.legend(bbox_to_anchor=(1.04,.1), borderaxespad=0)
  fig.tight_layout()
  fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
  
  #save the plot into a file
  mypath = '/gdrive/MyDrive/Colab Notebooks/FinAccessData' 
  myfile = 'barplot_df_ifdindex_'+str(mygrp)+'.jpg'
  joined_path = os.path.join(mypath, myfile)
  #plt.savefig(joined_path,bbox_inches = 'tight', pad_inches = 0)

  fig.show()
  mygrp+=1


##Seaborn boxplot

In [ ]:
# plotting boxplots (a different approach)
#////////////////////// df_OI & df_N are transformed
#////// plots are slightly different for transformed data

#dfa=pd.concat([df_OI,df_N])
dfa=tmp_all
dfa=dfa.merge(class_codes, on='country',how='inner')

my_colors = ['#BDFCC9', '#00CDCD']
sns.set_palette(sns.color_palette(my_colors))

try:
  dfa.loc[dfa['OIC']>0.,'OIC']='Muslim'
  dfa.loc[dfa['OIC']!='Muslim','OIC']='Not_Muslim'
except:
  True
col_out=['ATMs per 100,000 adults',
       'Account at a formal financial institution (% age 15+)',
       'BANK DEPOSITS to GDP (%)', 'Credit card (% age 15+)',
       'Debit card (% age 15+)', 'Deposit accounts per 1,000 adults',
       "Deposit money banks' assets to GDP (%)",
       'GNI per capita (constant 2010 US$)', 'Liquid liabilities to GDP (%)',
       'Made or received digital payments in the past year (% age 15+)',
       'Made payment using a mobile phone or the internet (% age 15+)',
       'Number of ATMs per 1,000 km2',
       'Number of commercial bank branches per 1,000 km2',
       'Number of commercial bank branches per 100,000 adults',
       'Number of loan accounts with commercial banks per 1,000 adults',
       'Outstanding deposits with commercial banks',
       'Outstanding loans from commercial banks',
       'Private credit by deposit money banks to GDP (%)',
       'Stock market capitalization to GDP (%)',
       'Used a debit or credit card to make a purchase in the past year (% age 15+)'
        ] 

from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(35,42))

my_ncols = 3
if (len(col_out)%my_ncols) == 0:
  my_nrows = len(col_out)//my_ncols
else:
  my_nrows = (len(col_out)//my_ncols)+1

gs = GridSpec(nrows=my_nrows, ncols=my_ncols)
mycnt=0
for my_col in col_out:
  myaxcol=mycnt%my_ncols
  myaxrow=mycnt//my_ncols
  ax=fig.add_subplot(gs[myaxrow,myaxcol])

  g1=sns.boxplot(x='inc_grp', y=my_col, showmeans=True, ax=ax, showcaps=False, showfliers=False, hue='OIC', 
                 data=dfa, order=['Low income', 'Lower middle income', 'Upper middle income','High income'])
  #               palette='RdGn')
  #ax = sns.swarmplot(x='inc_grp', y='AccFinInstit', data=dfa, color="k", alpha=0.4)
  #ax = sns.boxplot(x='inc_grp', y='AccFinInstit', data=dfa_OIC, )
  g1.set_axis_on()
  g1.grid(False)
  g1.legend_.remove()
  g1.tick_params(bottom=False, top=False, left=False, right=False)
  g1.set(yticklabels=[]) 
  g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
  g1.set_xlabel(None)
  mycnt+=1
  #add text
  x_offset=0.05 
  x_loc=0
  y_loc=myaxcol
  text_offset_x=35
  text_offset_y=20

  annotate_params = dict(xytext=(text_offset_x, text_offset_y), textcoords='offset points', arrowprops={'arrowstyle':'->'})

#save the plot into a file
mypath = '/gdrive/MyDrive/Colab Notebooks/FinAccessData' 
myfile = 'boxplot_df_iset.jpg'
joined_path = os.path.join(mypath, myfile)
#plt.savefig(joined_path,bbox_inches = 'tight', pad_inches = 0)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

'''  ax[myaxrow,myaxcol].annotate('Median', (x_loc + 1 + x_offset, g1['medians'][x_loc].get_ydata()[0]), **annotate_params)
  ax[myaxrow,myaxcol].annotate('25%', (x_loc + 1 + x_offset, g1['boxes'][x_loc].get_ydata()[0]), **annotate_params)
  ax[myaxrow,myaxcol].annotate('75%', (x_loc + 1 + x_offset, g1['boxes'][x_loc].get_ydata()[2]), **annotate_params)
  ax[myaxrow,myaxcol].annotate('5%', (x_loc + 1 + x_offset, g1['caps'][x_loc*2].get_ydata()[0]), **annotate_params)
  ax[myaxrow,myaxcol].annotate('95%', (x_loc + 1 + x_offset, g1['caps'][(x_loc*2)+1].get_ydata()[0]), **annotate_params)
'''



##Ridgeplot 


In [ ]:
#setup dataframes to plot
dfa=tmp_all
dfa=dfa.merge(class_codes, on='country',how='inner')

try:
    dfa.loc[dfa['OIC']>0.,'OIC']='Muslim'
    dfa.loc[dfa['OIC']!='Muslim','OIC']='Not_Muslim'
except:
    True

dfm=dfa.loc[dfa['OIC']=='Muslim']
dfnot=dfa.loc[dfa['OIC']=='Not_Muslim']

#list of columns to plot
collst=['ATMs per 100,000 adults',
       'Account at a formal financial institution (% age 15+)',
       'BANK DEPOSITS to GDP (%)', 'Credit card (% age 15+)',
       'Debit card (% age 15+)', 'Deposit accounts per 1,000 adults',
       "Deposit money banks' assets to GDP (%)",
       'GNI per capita (constant 2010 US$)', 'Liquid liabilities to GDP (%)',
       'Made or received digital payments in the past year (% age 15+)',
       'Made payment using a mobile phone or the internet (% age 15+)',
       'Number of ATMs per 1,000 km2',
       'Number of commercial bank branches per 1,000 km2',
       'Number of commercial bank branches per 100,000 adults',
       'Number of loan accounts with commercial banks per 1,000 adults',
       'Outstanding deposits with commercial banks',
       'Outstanding loans from commercial banks',
       'Private credit by deposit money banks to GDP (%)',
       'Stock market capitalization to GDP (%)',
       'Used a debit or credit card to make a purchase in the past year (% age 15+)'
        ] 
        
from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(25,32))

#enter number of desired columns of plots in output, then calculate the number of rows (% shows remainder; // operator is modulo function)
my_ncols = 1
if (len(collst)%my_ncols) == 0:
    my_nrows = len(collst)//my_ncols
else:
    my_nrows = (len(collst)//my_ncols)+1

gs = GridSpec(nrows=my_nrows, ncols=my_ncols, hspace=-0.2)

#loop through each column in collst
mycnt=0
for my_col in collst:
    #use number of columns to determine relative ax position
    myaxcol=mycnt%my_ncols
    myaxrow=mycnt//my_ncols
    ax=fig.add_subplot(gs[myaxrow,myaxcol])
    g1=sns.kdeplot(data=dfm[my_col].squeeze(), color='red', fill=True, label='Muslim', alpha=.1)
    g1=sns.kdeplot(data=dfnot[my_col].squeeze(), color='green', fill=True, label='Not Muslim', alpha=.1)
    g1.set_axis_on()
    g1.grid(False)
    #g1.legend_.remove()
    g1.tick_params(bottom=False, top=False, left=False, right=False)
    #g1.set(yticklabels=[my_col]) 
    g1.set_xlabel(my_col)
    g1.axes.get_yaxis().set_visible(False)
    rect = g1.patch
    rect.set_alpha(0) #make background transparent
    mycnt+=1
    #add text
    x_offset=0.05 
    x_loc=0
    y_loc=myaxcol
    text_offset_x=35
    text_offset_y=20
    sns.despine(left=True,right=True,top=True)

    annotate_params = dict(xytext=(text_offset_x, text_offset_y), textcoords='offset points', arrowprops={'arrowstyle':'->'})

#save the plot into a file
mypath = '/gdrive/MyDrive/Colab Notebooks/FinAccessData' 
myfile = 'ridgeplot_df_iset_'+str(mygrp)+'.jpg'
joined_path = os.path.join(mypath, myfile)
#plt.savefig(joined_path,bbox_inches = 'tight', pad_inches = 0)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()


##Seaborn boxplots
####approach #1 --not used

In [ ]:
####################################
#
#
#  ???Should i plot transformed data  
#  ???Will boxplot ignore null values 
#  Must add by region plots (possible to have region, incGrp, & OIC)
#
###################################

# Use df_OIC or df_OIC_save instead of demo_all
# need to merge with inc_grp & region data though

'''def annotate_boxplot(bpdict, annotate_params=None,
                     x_offset=0.05, x_loc=0,
                     text_offset_x=35,
                     text_offset_y=20):
    """Annotates a matplotlib boxplot with labels marking various centile levels.

    Parameters:
    - bpdict: The dict returned from the matplotlib `boxplot` function. If you're using pandas you can
    get this dict by setting `return_type='dict'` when calling `df.boxplot()`.
    - annotate_params: Extra parameters for the plt.annotate function. The default setting uses standard arrows
    and offsets the text based on other parameters passed to the function
    - x_offset: The offset from the centre of the boxplot to place the heads of the arrows, in x axis
    units (normally just 0-n for n boxplots). Values between around -0.15 and 0.15 seem to work well
    - x_loc: The x axis location of the boxplot to annotate. Usually just the number of the boxplot, counting
    from the left and starting at zero.
    text_offset_x: The x offset from the arrow head location to place the associated text, in 'figure points' units
    text_offset_y: The y offset from the arrow head location to place the associated text, in 'figure points' units
    """
    if annotate_params is None:
        annotate_params = dict(xytext=(text_offset_x, text_offset_y), textcoords='offset points', arrowprops={'arrowstyle':'->'})

    plt.annotate('Median', (x_loc + 1 + x_offset, bpdict['medians'][x_loc].get_ydata()[0]), **annotate_params)
    plt.annotate('25%', (x_loc + 1 + x_offset, bpdict['boxes'][x_loc].get_ydata()[0]), **annotate_params)
    plt.annotate('75%', (x_loc + 1 + x_offset, bpdict['boxes'][x_loc].get_ydata()[2]), **annotate_params)
    plt.annotate('5%', (x_loc + 1 + x_offset, bpdict['caps'][x_loc*2].get_ydata()[0]), **annotate_params)
    plt.annotate('95%', (x_loc + 1 + x_offset, bpdict['caps'][(x_loc*2)+1].get_ydata()[0]), **annotate_params)


# The following section is to setup un-transformed data
"""
try: 
  df_demo_all['inc_grp']=df_demo_all['inc_grp'].fillna(value='other')
except:
  True

#discard the rows with no income group
df_demo_all=df_demo_all.loc[df_demo_all['inc_grp']!='other']

unique_inc_grp = ['High income', 'Upper middle income', 'Lower middle income', 'Low income']

df_dem_all = df_demo_all
dfa=df_dem_all

try:
  dfa.loc[dfa['OIC']>0.,'OIC']='Muslim'
  dfa.loc[dfa['OIC']!='Muslim','OIC']='Not_Muslim'
except:
  True

dfa_OIC=dfa.loc[dfa['OIC']=='Muslim']
"""

#////////////////////// df_OI & df_N are transformed
#////// plots are slightly different for transformed data

#dfa=pd.concat([df_OI,df_N])
dfa=tmp_all
dfa=dfa.merge(class_codes, on='country',how='inner')

try:
  dfa.loc[dfa['OIC']>0.,'OIC']='Muslim'
  dfa.loc[dfa['OIC']!='Muslim','OIC']='Not_Muslim'
except:
  True
col_out=['GNI per capita (constant 2010 US$)','Stock market capitalization to GDP (%)',
         'Outstanding deposits with commercial banks','Outstanding loans from commercial banks', 
         'Private credit by deposit money banks to GDP (%)'
        ] 


#  specify number of columns in figure
my_ncols = 2
if (len(col_out)%my_ncols) == 0:
  my_nrows = len(col_out)//my_ncols
else:
  my_nrows = (len(col_out)//my_ncols)+1

fig, ax = plt.subplots(nrows=my_nrows,ncols=my_ncols, sharex=True)
fig.set_size_inches(15,22)

# hide all subplots in figure (show subplot after it's populated with data)
my_rcnt=0
my_ccnt=0
while my_rcnt < my_nrows:
  while my_ccnt < my_ncols:  
    ax[my_rcnt, my_ccnt].set_axis_off()
    my_ccnt+=1
  my_ccnt=0
  my_rcnt+=1

mycnt=0
for my_col in col_out:
  myaxcol=mycnt%my_ncols
  myaxrow=mycnt//my_ncols
  g1=sns.boxplot(x='inc_grp', y=my_col, showcaps=False, showmeans=True, ax=ax[myaxrow,myaxcol], showfliers=False, 
                 palette="GnBu", hue='OIC', data=dfa, order=['Low income', 'Lower middle income', 'Upper middle income','High income'])
  #ax = sns.swarmplot(x='inc_grp', y='AccFinInstit', data=dfa, color="k", alpha=0.4)
  #ax = sns.boxplot(x='inc_grp', y='AccFinInstit', data=dfa_OIC, )
  g1.set_axis_on()
  g1.grid(False)
  g1.legend_.remove()
  g1.tick_params(bottom=False, top=False, left=False, right=False)
  g1.set(yticklabels=[]) 
  g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
  g1.set_xlabel(None)

  mycnt+=1
  #add text
  """mytext=['a','b']
  for i in range(len(x)):
    plt.annotate(mytext[i], (x[i], y[i] + 0.2))"""
 
  # get bpdict by setting `return_type='dict'` when calling `df.boxplot()`
  #annotate_boxplot(bpdict)
  
  """for curr_box in g1.patches:
    height = curr_box.get_height()
    ax[myaxrow,myaxcol].text(curr_box.get_x() + curr_box.get_width() / 2., 0.5 * height, int(height),
                ha='center', va='center', color='black')"""

handles, labels = ax[myaxrow,myaxcol].get_legend_handles_labels()
plt.legend(handles[0:2], labels[0:2], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

sns.despine(left=True)
plt.savefig('/gdrive/MyDrive/Colab Notebooks/box0.jpg')
plt.show()

'''  




##Plotly Split Violin
## NOT USED because unable to sort the income group categories


In [ ]:
#############
# need to create a dataframe for the ridgeline plots: 
#       Rows will be the plots to chart (e.g., ATMs, Branches,...); 
#       2 shapes will be (OIC vs. Not); 
#       x-axis will be country
#############

#Make column a categorical datatype
''' COMMENT-out whole code block
import plotly.graph_objects as go
import plotly.express as px

#voilin showing muliple categories
show_legend = [True,False,False,False]
#category_orders={'inc_grp': ['High income', 'Upper middle income', 'Lower middle income', 'Low income']}, 

fig = go.Figure()
curdf=df_all_o

for i in range(0,len(pd.unique(curdf['inc_grp']))):
 
  fig.add_trace(go.Violin(x=curdf['inc_grp'][(curdf['OIC'] == 1) &
                                      (curdf['inc_grp'] == pd.unique(curdf['inc_grp'])[i])],
                          y=curdf['AccFinInstit'][(curdf['OIC'] == 1)&
                                      (curdf['inc_grp'] == pd.unique(curdf['inc_grp'])[i])],
                          legendgroup='Muslim', scalegroup='Muslim', name='Muslim',
                          side='negative',
                          line_color='lightblue',
                          showlegend=show_legend[i])
               )
  fig.add_trace(go.Violin(x=curdf['inc_grp'][(curdf['OIC'] != 1) &
                                        (curdf['inc_grp'] == pd.unique(curdf['inc_grp'])[i])],
                            y=curdf['AccFinInstit'][(curdf['OIC'] != 1)&
                                               (curdf['inc_grp'] == pd.unique(curdf['inc_grp'])[i])],
                            legendgroup='Not Muslim', scalegroup='Not Muslim', name='Not Muslim',
                            side='positive',
                            line_color='green',
                            showlegend=show_legend[i])
             )
    
# update characteristics shared by all traces

fig.update_layout(
    title_text="IMF Composition Metric",
    violingap=0, violingroupgap=0, violinmode='overlay', template='simple_white', xaxis_title='Income Group', yaxis_title='Access to Financial Institutions')

fig.show()


'''

##Sparkline type charts
#####Currently not working

In [ ]:
#sparkline type report
'''
data = np.cumsum(np.random.rand(1000)-0.5)
data = data - np.mean(data)

fig = plt.figure()
ax1 = fig.add_subplot(411) # nrows, ncols, plot_number, top sparkline
ax1.plot(data, 'b-')
ax1.axhline(c='grey', alpha=0.5)

ax2 = fig.add_subplot(412, sharex=ax1) 
ax2.plot(data, 'g-')
ax2.axhline(c='grey', alpha=0.5)

ax3 = fig.add_subplot(413, sharex=ax1)
ax3.plot(data, 'y-')
ax3.axhline(c='grey', alpha=0.5)

ax4 = fig.add_subplot(414, sharex=ax1) # bottom sparkline
ax4.plot(data, 'r-')
ax4.axhline(c='grey', alpha=0.5)


for axes in [ax1, ax2, ax3, ax4]: # remove all borders
    plt.setp(axes.get_xticklabels(), visible=False)
    plt.setp(axes.get_yticklabels(), visible=False)
    plt.setp(axes.get_xticklines(), visible=False)
    plt.setp(axes.get_yticklines(), visible=False)
    plt.setp(axes.spines.values(), visible=False)


# bottom sparkline
plt.setp(ax4.get_xticklabels(), visible=True)
plt.setp(ax4.get_xticklines(), visible=True)
ax4.xaxis.tick_bottom() # but onlyt the lower x ticks not x ticks at the top

plt.tight_layout()
plt.show()'''

#not used 'potential routines'



In [ ]:

''' 
#This works! Saving df info in a loop
new_df = []
for curr_df in my_df_list:
    new_df.append(curr_df)
#outside of loop ...concat
large_df = pd.concat(new_df, ignore_index=True)

#sample code saving df values in a loop (untested)
df = pd.DataFrame(columns=["A", "B"])
for i in len(df):
    this_column = df.columns[i]
    df[this_column] = [i, i+1]

#sample code to save data to a file
df.to_csv(file_name, encoding='utf-8', index=False)                                

'''

'''
imputer = KNNImputer(n_neighbors= 5)
impute_with_3 = imputer.fit_transform(OIC)
'''

'''collst=df_OIC_l.columns.to_list()
lastcol=len(collst)-1
mydfcnt=0
for currdf in dfs_vals:
  search_values = ['_ot','_it']
  if dfs_keys[mydfcnt].str.contains('|'.join(search_values))]:
    break
  mycnt=0
  collst=currdf.columns.to_list() # since cols are incorrectly different build new list
  lastcol=len(collst)-1
  for currcol in collst:
    if is_numeric_dtype(currdf[currcol]):
      currdf[currcol].fillna(currdf[currcol].median()).reset_index(drop=True)
    if mycnt ==lastcol:
      name = dfs_keys[mydfcnt]
      print('name',name)
      currdf.to_csv('%s.csv' % name, encoding='utf-8', index=False)  
    mycnt+=1
  mydfcnt+=1

df_OIC_i = pd.concat([df_OIC_l, df_OIC_lm, df_OIC_um, df_OIC_h], join='inner')
df_not_i = pd.concat([df_not_l, df_not_lm, df_not_um, df_not_h], join='inner')

pdList = [df_not_i, df_OIC_i]
df_all_i= pd.concat(pdList, join='inner')

'''

'''print('The single Null value imputation method – unconditional mean imputation has been used to \n', 
      'determine the missing values in terms of the OIC classification scheme.\n',
      'Natural log data transformation or cube-root transformations used to reduce skewness within data.')
'''
'''
sample code:
# to slice a dataframe based on values in a string
search_values = ['m','ia']
df0=df0[df0.index.str.contains('|'.join(search_values ))]
'''


'''for dfname, dikt in zip(dfs.keys(), dicts):
    dfs[dfname] = dfs[dfname].from_dict(dikt, orient='columns', dtype=None)
    dfs['df_OIC_l']=df_all_it.pivot_table(index='country', columns='desc', values='value')
'''

'''////outdated
#overall median
collst=df_OIC_h.columns.to_list()
for currcol in collst:
  if (currcol not in ['OIC','OECD'])&(is_numeric_dtype(currdf[currcol])):
    tmp_all[currcol].fillna(tmp_all[currcol].median())
'''

'''
# sample code to slice a dataframe based on values in a string
search_values = ['m','ia']
df0=df0[df0.index.str.contains('|'.join(search_values ))]
'''

